In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# ===============================
# BEGIN ACQUISITION METHODS
# ===============================

# TODO Need to update build_active_MP_for_outages_df in other notebooks, as the method below is the most recent
Also need to add build_active_MP_for_xfmrs_in_outages_df (below)

In [2]:
def reduce_active_MP_for_outages(
    df_outage, 
    prem_nb_col, 
    df_mp_curr, 
    df_mp_hist, 
    drop_inst_rmvl_cols=False, 
    outg_rec_nb_col='OUTG_REC_NB',  #TODO!!!!!!!!!!!!!!!!!!!!!!! what if index?!
    is_slim=False, 
    drop_approx_duplicates=True, 
    drop_approx_duplicates_args=None, 
    dt_on_ts_col='DT_ON_TS', 
    df_off_ts_full_col='DT_OFF_TS_FULL', 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb'
):
    r"""
    Intended to be used within build_active_MP_for_outages_df and build_active_MP_for_xfmrs_in_outages_df.
    This does not mean it cannot be used elsewhere, but use with caution (understand functionality before use!)
    """
    # Only reason for making dict is to ensure outg_rec_nbs are not repeated 
    active_SNs_in_outgs_dfs_dict = {}

    if not is_slim:
        for outg_rec_nb_i, df_i in df_outage.groupby(outg_rec_nb_col):
            # Don't want to include outg_rec_nb_i=-2147483648
            if int(outg_rec_nb_i) < 0:
                continue
            # There should only be a single unique dt_on_ts and dt_off_ts_full for each outage
            if(df_i[dt_on_ts_col].nunique()!=1 or 
               df_i[df_off_ts_full_col].nunique()!=1):
                print(f'outg_rec_nb_i = {outg_rec_nb_i}')
                print(f'df_i[dt_on_ts_col].nunique()       = {df_i[dt_on_ts_col].nunique()}')
                print(f'df_i[df_off_ts_full_col].nunique() = {df_i[df_off_ts_full_col].nunique()}')
                print('CRASH IMMINENT!')
                assert(0)
            # Grab power out/on time and PNs from df_i
            dt_on_ts_i       = df_i[dt_on_ts_col].unique()[0]
            df_off_ts_full_i = df_i[df_off_ts_full_col].unique()[0]
            PNs_i            = df_i[prem_nb_col].unique().tolist()

            # Just as was done above for PNs, NaN values must be removed from PNs_i
            #   The main purpose here is to remove instances where PNs_i = [nan]
            #   NOTE: For case of slim df, the NaNs should already be removed
            # After removal, if len(PNs_i)==0, contine
            PNs_i = [x for x in PNs_i if pd.notna(x)]
            if len(PNs_i)==0:
                continue
            
            # Build active_SNs_df_i and add it to active_SNs_in_outgs_dfs_dict
            # NOTE: assume_one_xfmr_per_PN=True above in MeterPremise.build_mp_df_curr_hist_for_PNs,
            #       so does not need to be set again (i.e., assume_one_xfmr_per_PN=False below)
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=PNs_i,
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=df_off_ts_full_i,
                dt_1=dt_on_ts_i,
                addtnl_mp_df_curr_cols=None, 
                addtnl_mp_df_hist_cols=None,
                assume_one_xfmr_per_PN=False, 
                output_index=None,
                output_groupby=None, 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False, 
                drop_approx_duplicates=drop_approx_duplicates, 
                drop_approx_duplicates_args=drop_approx_duplicates_args, 
                df_mp_serial_number_col=df_mp_serial_number_col, 
                df_mp_prem_nb_col=df_mp_prem_nb_col, 
                df_mp_install_time_col=df_mp_install_time_col, 
                df_mp_removal_time_col=df_mp_removal_time_col, 
                df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
            )
            active_SNs_df_i[outg_rec_nb_col] = outg_rec_nb_i
            assert(outg_rec_nb_i not in active_SNs_in_outgs_dfs_dict)
            active_SNs_in_outgs_dfs_dict[outg_rec_nb_i] = active_SNs_df_i
    else:
        for outg_rec_nb_i, row_i in df_outage.iterrows():
            # NOTE: assume_one_xfmr_per_PN=True above in MeterPremise.build_mp_df_curr_hist_for_PNs,
            #       so does not need to be set again (i.e., assume_one_xfmr_per_PN=False below)
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=row_i[prem_nb_col],
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=row_i[df_off_ts_full_col],
                dt_1=row_i[dt_on_ts_col],
                addtnl_mp_df_curr_cols=None, 
                addtnl_mp_df_hist_cols=None,
                assume_one_xfmr_per_PN=False, 
                output_index=None,
                output_groupby=None, 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False, 
                drop_approx_duplicates=drop_approx_duplicates, 
                drop_approx_duplicates_args=drop_approx_duplicates_args, 
                df_mp_serial_number_col=df_mp_serial_number_col, 
                df_mp_prem_nb_col=df_mp_prem_nb_col, 
                df_mp_install_time_col=df_mp_install_time_col, 
                df_mp_removal_time_col=df_mp_removal_time_col, 
                df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col                
            )
            active_SNs_df_i[outg_rec_nb_col] = outg_rec_nb_i
            assert(outg_rec_nb_i not in active_SNs_in_outgs_dfs_dict)
            active_SNs_in_outgs_dfs_dict[outg_rec_nb_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_outgs_dfs_dict.values()))
    #-------------------------
    if drop_inst_rmvl_cols:
        active_SNs_df = active_SNs_df.drop(columns=[df_mp_install_time_col, df_mp_removal_time_col])
    #-------------------------
    return active_SNs_df

In [3]:
"""
In MeterPremise, can I create a function which will build mp_df_hist given mp_df_curr?
In DOVSOutages I need to build new build_mp_for_outg (and update its use throughout)
    APPARENTLY I already have this, build_active_MP_for_outages or one of other similar functions
    
I need to basically replace everything in DOVSOutages which uses build_mp_for_outg
"""


def build_active_MP_for_outages_df(
    df_outage, 
    prem_nb_col, 
    df_mp_curr=None, 
    df_mp_hist=None, 
    assert_all_PNs_found=True, 
    drop_inst_rmvl_cols=False, 
    outg_rec_nb_col='OUTG_REC_NB',  #TODO!!!!!!!!!!!!!!!!!!!!!!! what if index?!
    is_slim=False, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    dt_on_ts_col='DT_ON_TS', 
    df_off_ts_full_col='DT_OFF_TS_FULL', 
    consolidate_PNs_batch_size=1000, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    early_return=False
):
    r"""
    Similar to build_active_MP_for_outages
    
    If addtnl_mp_df_curr_cols and addtnl_mp_df_hist_cols are included (i.e., are not None), their intersection will be added
      as addtnl_groupby_cols input argument to drop_approx_duplicates (otherwise, they would be returned in the DF as list
      objects, and would likely need to be exploded)
    """
    #-------------------------
    assert(prem_nb_col in df_outage.columns and 
           dt_on_ts_col in df_outage.columns and 
           df_off_ts_full_col in df_outage.columns)
    #-------------------------
    if not is_slim:
        PNs = df_outage[prem_nb_col].unique().tolist()
    else:
        PNs = Utilities_df.consolidate_column_of_lists(
            df=df_outage, 
            col=prem_nb_col, 
            sort=True,
            include_None=False,
            batch_size=consolidate_PNs_batch_size, 
            verbose=False
        )
    #-----
    PNs = [x for x in PNs if pd.notna(x)]
    PNs_type = type(PNs[0])
    assert(Utilities.are_all_list_elements_of_type(PNs, PNs_type))
    #-------------------------
    if addtnl_mp_df_curr_cols is not None and addtnl_mp_df_hist_cols is not None:
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_curr_cols, [list, tuple]))
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_hist_cols, [list, tuple]))
        drop_approx_duplicates_args = dict(
            addtnl_groupby_cols=list(set(addtnl_mp_df_curr_cols).intersection(set(addtnl_mp_df_hist_cols)))
        )
    else:
        drop_approx_duplicates_args=None
    #-----
    mp_df_curr_hist_dict = MeterPremise.build_mp_df_curr_hist_for_PNs(
        PNs=PNs, 
        mp_df_curr=df_mp_curr,
        mp_df_hist=df_mp_hist, 
        join_curr_hist=False, 
        addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
        addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
        assert_all_PNs_found=assert_all_PNs_found, 
        assume_one_xfmr_per_PN=True, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    df_mp_curr = mp_df_curr_hist_dict['mp_df_curr']
    df_mp_hist = mp_df_curr_hist_dict['mp_df_hist']
    if early_return:
        return df_mp_curr, df_mp_hist
    print(f'df_mp_curr.shape = {df_mp_curr.shape}')
    print(f'df_mp_hist.shape = {df_mp_hist.shape}')
    #-------------------------
    active_SNs_df = reduce_active_MP_for_outages(
        df_outage=df_outage, 
        prem_nb_col=prem_nb_col, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        drop_inst_rmvl_cols=drop_inst_rmvl_cols, 
        outg_rec_nb_col=outg_rec_nb_col, 
        is_slim=is_slim, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        dt_on_ts_col=dt_on_ts_col, 
        df_off_ts_full_col=df_off_ts_full_col, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    #-------------------------
    return active_SNs_df

In [4]:
#TODO !!!!!!!!!!!! IMPORTANT !!!!!!!!!!!!!!
# In order for this to function as desired, reduce_active_MP_for_outages must group by transformer pole number
#   for this case, not outage number as in the build_active_MP_for_outages_df case
# Some other adjustment may be needed as well!
# NOTE: There are some instances where build_active_MP_for_outages_df will find PNs not found by 
#         build_active_MP_for_xfmrs_in_outages_df.
#       The reason being that the former doesn't care whether or not the trsf_pole_nb is NaN, but this function does!
# May want to, e.g., add in some sort of dropna option
# However, what would one do in such a situation?  I guess you would have to make an or statement, allowing
#   a PN to have the correct transformer OR be listed in df_outage is partaking in the outage?
# Not too sure about this, but it will obviously need some work.
# For example where build_active_MP_for_outages_df finds additional PN, see outg_rec_nb 13297062, where
#   mfr_devc_ser_nbr=995582422 is found by build_active_MP_for_outages_df but not by build_active_MP_for_xfmrs_in_outages_df
#   (because only found in historic, and therefore the trsf_pole_nb could not be determined!)
#
# May need to combine those with trsf_pole_nb=na in mp to those in mp_df_curr_hist_dict
#   One could, e.g., call MeterPremise.build_mp_df_curr_hist_for_PNs for any PNs without a transformer 
#   pole number, and still use MeterPremise.build_mp_df_curr_hist_for_xfmrs for others
def build_active_MP_for_xfmrs_in_outages_df(
    df_outage, 
    prem_nb_col, 
    df_outage_trsf_pole_nb_col=None, 
    drop_inst_rmvl_cols=False, 
    outg_rec_nb_col='OUTG_REC_NB',  #TODO!!!!!!!!!!!!!!!!!!!!!!! what if index?!
    is_slim=False, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    dt_on_ts_col='DT_ON_TS', 
    df_off_ts_full_col='DT_OFF_TS_FULL', 
    consolidate_PNs_batch_size=1000, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    early_return=False
):
    r"""
    Similar to build_active_MP_for_outages/build_active_MP_for_outages_df
    
    df_outage_trsf_pole_nb_col:
        If trsf_pole_nb already present in df_outage, set this equal to the column.
        Otherwise, the trsf_pole_nbs will be acquired from the current Meter Premise table
        
    If addtnl_mp_df_curr_cols and addtnl_mp_df_hist_cols are included (i.e., are not None), their intersection will be added
      as addtnl_groupby_cols input argument to drop_approx_duplicates (otherwise, they would be returned in the DF as list
      objects, and would likely need to be exploded)
    """
    #-------------------------
    assert(prem_nb_col in df_outage.columns and 
           dt_on_ts_col in df_outage.columns and 
           df_off_ts_full_col in df_outage.columns)
    #-------------------------
    # If trsf_pole_nbs present in df_outage, simply grab.
    # Otherwise, use current Meter Premise table together with PNs from df_outage to acquire trsf_pole_nbs
    if df_outage_trsf_pole_nb_col is not None and df_outage_trsf_pole_nb_col in df_outage.columns:
        trsf_pole_nbs = df_outage[df_outage_trsf_pole_nb_col].unique().tolist()
    else:
        if not is_slim:
            PNs = df_outage[prem_nb_col].unique().tolist()
        else:
            PNs = Utilities_df.consolidate_column_of_lists(
                df=df_outage, 
                col=prem_nb_col, 
                sort=True,
                include_None=False,
                batch_size=consolidate_PNs_batch_size, 
                verbose=False
            )
        #-----
        PNs = [x for x in PNs if pd.notna(x)]
        PNs_type = type(PNs[0])
        assert(Utilities.are_all_list_elements_of_type(PNs, PNs_type))
        #-------------------------
        mp_cols_of_interest=[df_mp_prem_nb_col, df_mp_trsf_pole_nb_col]
        mp = MeterPremise(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            init_df_in_constructor=True, 
            build_sql_function=MeterPremise.build_sql_meter_premise, 
            build_sql_function_kwargs=dict(
                cols_of_interest=mp_cols_of_interest, 
                premise_nbs=PNs, 
                table_name='meter_premise'
            )
        )
        mp_df = mp.df
        #-------------------------
        trsf_pole_nbs = mp_df[df_mp_trsf_pole_nb_col].unique().tolist()
    #--------------------------------------------------
    if addtnl_mp_df_curr_cols is not None and addtnl_mp_df_hist_cols is not None:
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_curr_cols, [list, tuple]))
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_hist_cols, [list, tuple]))
        drop_approx_duplicates_args = dict(
            addtnl_groupby_cols=list(set(addtnl_mp_df_curr_cols).intersection(set(addtnl_mp_df_hist_cols)))
        )
    else:
        drop_approx_duplicates_args=None
    #-----
    mp_df_curr_hist_dict = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        join_curr_hist=False, 
        addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
        addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
        assume_one_xfmr_per_PN=True, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    df_mp_curr = mp_df_curr_hist_dict['mp_df_curr']
    df_mp_hist = mp_df_curr_hist_dict['mp_df_hist']
    if early_return:
        return df_mp_curr, df_mp_hist
    print(f'df_mp_curr.shape = {df_mp_curr.shape}')
    print(f'df_mp_hist.shape = {df_mp_hist.shape}')
    #-------------------------
    active_SNs_df = reduce_active_MP_for_outages(
        df_outage=df_outage, 
        prem_nb_col=prem_nb_col, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        drop_inst_rmvl_cols=drop_inst_rmvl_cols, 
        outg_rec_nb_col=outg_rec_nb_col, 
        is_slim=is_slim, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        dt_on_ts_col=dt_on_ts_col, 
        df_off_ts_full_col=df_off_ts_full_col, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    #-------------------------
    return active_SNs_df

# ===============================
# END ACQUISITION METHODS
# ===============================

In [5]:
# # MOVED TO Utilities_df
# import statistics
# def determine_freq_in_df_col(
#     df, 
#     groupby_SN=True, 
#     return_val_if_not_found='error', 
#     assert_single_freq=True, 
#     assert_expected_freq_found=False, 
#     expected_freq=pd.Timedelta('15 minutes'), 
#     time_col='starttimeperiod_local', 
#     SN_col='serialnumber'
# ):
#     r"""
#     Determine the frequency of the data in df[time_col]
#     If the df contains multiple serial numbers, IT IS HIGHLY recommended that one groups by serialnumber,
#       finding the frequency of each subset, and inferring the overall frequency of the data from that collection.
#         In most cases, not grouping the data for, e.g., and outage by serial number will be fine.
#         BUT, there are some instances where this will give unexpected results (more info below).
#         THUS, to be safe, follow the recommendation above.
        
#     return_val_if_not_found:
#         This directs what occurs if no frequency is determined.
#         Possible values (case insensitive):
#             NaN:      return np.nan (which is converted by Python/pandas to NaT)
#             expected: return input argument expected_freq
#             error:    throw an error
#         This is especially useful when finding the frequency for a group of SNs. 
#             For this case, return_val_if_not_found is set to NaN within the lambda function call to 
#               determine_freq_in_df_col, allowing some flexibility of not every single SN needing to return 
#               a frequency for the overall frequency to be determined.
#             However, after the groupby method, the overall returned result behaves according to the value of
#               return_val_if_not_found, as stated above.
        
#     assert_single_freq:
#         NOTE: This is only enforced if frequencies are found; i.e., if no frequency is determined, the function
#               progressed according to the return_val_if_not_found argument.
#         -----
#         For a single SN level or groupby_SN==False:
#             Self-explanatory; if True, assert only a single frequency is found.
#             If False (not recommended), and multiple frequencies are found, take the first
#         -----
#         For multiple SNs with groupby_SN==True:
#             True:
#                 Enforce a single frequency is found for each SN
#                 Enforce all SNs share the same frequency
#             False:
#                 Do not enforce single frequency found for each SN (i.e., if multiple are found for a single SN, 
#                   the first is returned).
#                 If the collection of frequencies found from SNs contains multiple values, take the mode (if multiple
#                   modes found, take the first one)
                  
#     assert_expected_freq_found:
#         Assert the frequency found equals that which is expected.
#         NOTE: This is only enforced if a frequency is found; i.e., if no frequency is determined, the function
#               progressed according to the return_val_if_not_found argument.        
        
#     -----
#     Potential for unexpected results:
#         Consider the following simple example of a pd.DataFrame (ex_df) containing two serial numbers whose times are offset 
#           relative to each other by one-minute (BTW, this is not some made-up scenario, this is observed in the data)
#                 -------------------------
#                     serialnumber starttimeperiod_local
#                 0               1   2023-01-14 20:45:00
#                 1               2   2023-01-14 20:46:00
#                 2               1   2023-01-14 21:00:00
#                 3               2   2023-01-14 21:01:00
#                 4               1   2023-01-14 21:15:00
#                 5               2   2023-01-14 21:16:00
#                 6               1   2023-01-14 21:30:00
#                 7               2   2023-01-14 21:31:00
#                 8               1   2023-01-14 21:45:00
#                 9               2   2023-01-14 21:46:00
#                 10              1   2023-01-14 22:00:00
#                 -------------------------                
#         Pandas has no idea what the frequency of the data is, and the call 
#           freq = pd.infer_freq(ex_df['starttimeperiod_local'].unique()) will return a value of None
#         Therefore, one falls back to the secondary method of looking for the most frequent difference in the data (i.e.,
#           the method freq=stats.mode(np.diff(natsorted(df[time_col].unique()))).mode[0])
#         However, the call np.diff(natsorted(ex_df['starttimeperiod_local'].unique())) will return:
#           (NOTE: Actually, to make numbers easy to interpret, the call was: 
#                  [pd.Timedelta(x) for x in np.diff(natsorted(ex_df['starttimeperiod_local'].unique()))])
#             -------------------------                  
#             [Timedelta('0 days 00:01:00'),
#              Timedelta('0 days 00:14:00'),
#              Timedelta('0 days 00:01:00'),
#              Timedelta('0 days 00:14:00'),
#              Timedelta('0 days 00:01:00'),
#              Timedelta('0 days 00:14:00'),
#              Timedelta('0 days 00:01:00'),
#              Timedelta('0 days 00:14:00'),
#              Timedelta('0 days 00:01:00'),
#              Timedelta('0 days 00:14:00')]
#             -------------------------
#         The example I have chosen actually has two modes (1 minutes and 14 minutes), both of which are incorrect!
#         Using scipy.stats.mode, only a single value is returned, and this appears to be the first, i.e. 1 minute
#             ==> freq = stats.mode(np.diff(natsorted(ex_df['starttimeperiod_local'].unique()))).mode[0]
#                 freq = pd.Timedelta(freq)
#                 GIVES THE RESULT: freq = 1 minute!
#         We only expect there to be one frequency of the data, but one could also use statistics.multimode to obtain:
#             ==> freq = statistics.multimode(np.diff(natsorted(ex_df['starttimeperiod_local'].unique())))
#                 freq = [pd.Timedelta(x) for x in freq]
#                 GIVES THE RESULT: freq = [1 minute, 14 minutes]
                
#         ----------
#         SOLUTION: Group the data by serial number, and find the frequency wrt each SN
#             ------------------------- 
#             SN==1
#             -----
#                 serialnumber starttimeperiod_local
#             0              1   2023-01-14 20:45:00
#             2              1   2023-01-14 21:00:00
#             4              1   2023-01-14 21:15:00
#             6              1   2023-01-14 21:30:00
#             8              1   2023-01-14 21:45:00
#             10             1   2023-01-14 22:00:00
#             ------------------------- 
#             ------------------------- 
#             SN==2
#             -----
#                serialnumber starttimeperiod_local
#             1             2   2023-01-14 20:46:00
#             3             2   2023-01-14 21:01:00
#             5             2   2023-01-14 21:16:00
#             7             2   2023-01-14 21:31:00
#             9             2   2023-01-14 21:46:00
#             ------------------------- 
#         ==> freq_1 == freq_2 == 15 minutes! (with pd.infer_freq or secondary method!)
#         ----------
#     """
#     #-------------------------
#     if not groupby_SN and df[SN_col].nunique()>1:
#         print('Multiple SNs exist in df.\nGrouping by SN is HIGHLY RECOMMENDED!')
#     #-------------------------
#     return_val_if_not_found = return_val_if_not_found.lower()
#     assert(return_val_if_not_found in ['nan', 'expected', 'error'])
#     #-------------------------
#     # Strategy below will be to develop the code as if being written for case where df has a single SN (or, the case
#     #   where groupby_SN==False).
#     # Then, if groupby_SN is True, one can call the function from inside a groupby lambda function (with groupby_SN
#     #   set to False within the lambda call)
#     #----------------------------------------------------------------------------------------------------
#     if not groupby_SN:
#         # First, try pd.infer_freq method
#         # NOTE: infer_freq returns None if no frequency could be determined and raises a TypeError 
#         #         if data are not datetime-like and ValueError is few than 3 values
#         #       try/except below eliminates errors from being raise, setting freq to None in such cases instead.
#         try:
#             freq=pd.infer_freq(natsorted(df[time_col].unique()))
#             # If freq is, e.g., 1 hour, this will return 'H', whereas pd.Timedelta wants
#             #   its input to be '1H' and will complain if fed 'H'
#             # Thus, check if string starts with digit.  It it doesn't, prepend '1' to it
#             if not freq[0].isdigit():
#                 freq = '1'+freq
#         except:
#             freq=None
#         #-------------------------
#         # If pd.infer_freq was unsuccessful, use the secondary method of looking for the most frequent 
#         #   difference in the data
#         # NOTE!: Cannot enforce assert_single_freq in the try block below, as if the assertion is tripped this
#         #        will simply switch the code to the except block!!!!!
#         if freq is None:
#             try:
#                 # NOTE: Below, freq will always be a list
#                 freq=statistics.multimode(np.diff(natsorted(df[time_col].unique())))
#             except:
#                 freq=[]
#             #-----
#             if len(freq)==0:
#                 if return_val_if_not_found=='nan':
#                     return np.nan
#                 elif return_val_if_not_found=='expected':
#                     return pd.Timedelta(expected_freq)
#                 elif return_val_if_not_found=='error':
#                     assert(0)
#                 else:
#                     assert(0)
#             else:
#                 # Enforce assertion and/or set freq equal to first value (as 
#                 #   freq returned from statistics.multimode is a list)
#                 if assert_single_freq:
#                     assert(len(freq)==1)
#                 freq=freq[0]
#         #-------------------------
#         freq=pd.Timedelta(freq)
#         if assert_expected_freq_found:
#             assert(freq==pd.Timedelta(expected_freq))
#         #-------------------------
#         return freq
#     #----------------------------------------------------------------------------------------------------
#     # Now, for the more typical case when groupby_SN is True
#     # In this case, we essentially call the above methods from within a groupby lambda function
#     #-------------------------
#     # Get the frequency for each serial number, which will be a pd.Series object
#     # NOTE: groupby_SN MUST BE FALSE in the lambda function below, otherwise infinite loop!!!!!
#     freqs_srs = df.groupby([SN_col]).apply(
#         lambda x: determine_freq_in_df_col(
#             df=x, 
#             groupby_SN=False, 
#             assert_single_freq=assert_single_freq, 
#             expected_freq=expected_freq, 
#             return_val_if_not_found='nan', 
#             assert_expected_freq_found=assert_expected_freq_found, 
#             time_col=time_col, 
#             SN_col=SN_col
#         )
#     )
#     #-------------------------
#     freqs_unq_vals = freqs_srs.dropna().unique()
#     freqs_modes = statistics.multimode(freqs_unq_vals)
#     #-----        
#     if len(freqs_modes)==0:
#         if return_val_if_not_found=='nan':
#             return np.nan
#         elif return_val_if_not_found=='expected':
#             return pd.Timedelta(expected_freq)
#         elif return_val_if_not_found=='error':
#             assert(0)
#         else:
#             assert(0)
#     #-----
#     if assert_single_freq:
#         assert(len(freqs_modes)==1)
#     freq=freqs_modes[0]
#     freq=pd.Timedelta(freq)
#     #-------------------------
#     return freq

In [6]:
def get_pd_pu_times_for_meter(
    ede_df_i, 
    t_search_min_max=None, 
    pd_ids=['3.26.0.47', '3.26.136.47'], 
    pu_ids=['3.26.0.216', '3.26.136.216'], 
    SN_col='serialnumber', 
    valuesinterval_col='valuesinterval_local', 
    edetypeid_col='enddeviceeventtypeid'
):
    r"""
    Get the power-down (pd) and power-up (pu) events from ede_df_i within t_search_min_max.
    ede_df_i must be a meter events DataFrame containing data from a single meter.
    
    Returns a dict with two keys, pd_times and pu_times, the values of which are lists containing
      the associated times.
    """
    #-------------------------
    assert(ede_df_i[SN_col].nunique()==1)
    #-------------------------
    # Make sure valuesinterval_col contains time data
    if not is_datetime64_dtype(ede_df_i[valuesinterval_col]):
        ede_df_i[valuesinterval_col] = pd.to_datetime(ede_df_i[valuesinterval_col])
    ede_df_i = ede_df_i.sort_values(by=[valuesinterval_col])

    #------------------------- 
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        if t_search_min_max[0] is None:
            t_search_min_max[0] = pd.Timestamp.min
        if t_search_min_max[1] is None:
            t_search_min_max[1] = pd.Timestamp.max
        #-----
        ede_df_i = ede_df_i[
            (ede_df_i[valuesinterval_col]>=t_search_min_max[0]) & 
            (ede_df_i[valuesinterval_col]<=t_search_min_max[1])
        ]
        
    #------------------------- 
    ede_df_i = ede_df_i.sort_values(by=valuesinterval_col)
    #-----
    pd_times = ede_df_i[ede_df_i[edetypeid_col].isin(pd_ids)][valuesinterval_col].tolist()
    pu_times = ede_df_i[ede_df_i[edetypeid_col].isin(pu_ids)][valuesinterval_col].tolist()
    #------------------------- 
    return dict(
        pd_times=pd_times, 
        pu_times=pu_times
    )

In [7]:
# This function is not used much anymore
# Typically, I just use best_ests_df to grab which PNs/SNs participate in outage and which do not.
def get_full_part_not_outage_SNs(
    df, 
    out_t_beg,
    out_t_end, 
    pct_time_out_required_for_outage=25, 
    exclusive_partials=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber'    
):
    r"""
    During the outage time (typically taken from DOVS and defined by out_t_beg and out_t_end), find the serial numbers which were
      out (value==0) for the entire period, for part of the period (threshold set by pct_time_out_required_for_outage), 
      and for none of the period.
      
    NOTE: By default, the partial outage SNs will include the full outage SNs.
          See exclusive_partials below
      
    pct_time_out_required_for_outage:
        In order for a SN to be considered partially out, it must be out (i.e., have value==0) for at least 
          pct_time_out_required_for_outage percent of the time.
          
    exclusive_partials:
        By default, the partial outage SNs will include the full outage SNs.
        If this functionality is not desired, and one wants the two sets to be mutually exclusive, set
          exclusive_partials=True
    """
    #-------------------------
    # First, grab the subset of df occurring during the outage
    df_sub = df[
        (df[t_int_beg_col]>=out_t_beg) & 
        (df[t_int_end_col]<out_t_end)
    ]
    #-------------------------
    full_SNs = df_sub.groupby([SN_col]).filter(lambda x: (x[value_col]==0).all())[SN_col].unique().tolist() 
    #-----
    not_SNs = df_sub.groupby([SN_col]).filter(lambda x: (x[value_col]>0).all())[SN_col].unique().tolist()
    #-----
    part_SNs = df_sub.groupby([SN_col]).filter(lambda x: 100*(x[value_col]==0).sum()/x.shape[0]>pct_time_out_required_for_outage)[SN_col].unique().tolist()
    if exclusive_partials:
        part_SNs = list(set(part_SNs).difference(set(full_SNs)))
    #-------------------------
    return dict(
        full_SNs=full_SNs, 
        part_SNs=part_SNs, 
        not_SNs=not_SNs
    )
    

# This function is not used much anymore
# Typically, I just use best_ests_df to grab which PNs/SNs participate in outage and which do not.
def get_full_part_not_outage_subset_dfs(
    df, 
    out_t_beg,
    out_t_end, 
    pct_time_out_required_for_outage=0, 
    return_SNs=False, 
    exclusive_partials=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber'    
):
    r"""
    The full, partial, and not outage SNs are determined in the outage period defined by out_t_beg and out_t_end, but the returned
      subsets are for all times in df (i.e., not restricted to outage time)
    i.e., the SNs for the full, partial, and not are found using get_full_part_not_outage_SNs during outage times, and then
      the subsets for df containing those SNs are projected out and returned.
    """
    #-------------------------
    full_part_not_dict = get_full_part_not_outage_SNs(
        df=df, 
        out_t_beg=out_t_beg,
        out_t_end=out_t_end, 
        pct_time_out_required_for_outage=pct_time_out_required_for_outage, 
        exclusive_partials=exclusive_partials, 
        t_int_beg_col=t_int_beg_col, 
        t_int_end_col=t_int_end_col, 
        value_col=value_col, 
        SN_col=SN_col   
    )
    #-------------------------
    full_df = df[df[SN_col].isin(full_part_not_dict['full_SNs'])].copy()
    part_df = df[df[SN_col].isin(full_part_not_dict['part_SNs'])].copy()
    not_df  = df[df[SN_col].isin(full_part_not_dict['not_SNs'])].copy()
    #-------------------------
    return_dict = dict(
        full_df=full_df, 
        part_df=part_df, 
        not_df=not_df
    )
    if return_SNs:
        return_dict = return_dict|full_part_not_dict
    return return_dict

In [8]:
def estimate_broad_outage_time_using_ede(
    ede_df_i, 
    dovs_out_t_beg, 
    dovs_out_t_end, 
    
    pd_ids=['3.26.0.47', '3.26.136.47'], 
    pu_ids=['3.26.0.216', '3.26.136.216'], 
    
    return_pct_SNs_close=False, 
    out_time_thresh=pd.Timedelta('10 minutes'), 
    
    SN_col='serialnumber', 
    valuesinterval_col='valuesinterval_local', 
    edetypeid_col='enddeviceeventtypeid', 
    verbose=False
):
    r"""
    This function essentially finds the closest power-down event to dovs_out_t_beg and the 
      closest power-up event to dovs_out_t_end
    NOTE: I use the terms power-up and power-down events broadly here.  
          In general, these also contain reasons such as last gasps, power restore, etc.
          What is considered power-up is defined by pu_ids, and what is considered power-down is defined by pd_ids,
            where pu_ids and pd_ids contain enddeviceeventtypeids.
          Might want to use 'reason' column and run a further reduce method, but it seems enddeviceeventtypeids will
            work well for this purpose.
          
    This can be used with an ede_df_i containing data for all meters in an outage, or for an ede_df_i containing
      events for a single meter.
    If ede_df_i contains data from multiple meters, one can check the percentage of SNs with a power-down close to the
      found outage begin time and the percentage with a power-up close to the found outage end time.
      The definition of 'close' is set by out_time_thresh
    """
    #-------------------------
    # The methodology relies on ede_df_i being sorted by time
    # But, first, make sure valuesinterval_col contains time data
    if not is_datetime64_dtype(ede_df_i[valuesinterval_col]):
        ede_df_i[valuesinterval_col] = pd.to_datetime(ede_df_i[valuesinterval_col])
    ede_df_i = ede_df_i.sort_values(by=[valuesinterval_col])
    #-------------------------
    # Get the power-up and power-down subsets from ede_df_i
    ede_df_i_pu = ede_df_i[ede_df_i[edetypeid_col].isin(pu_ids)]
    ede_df_i_pd = ede_df_i[ede_df_i[edetypeid_col].isin(pd_ids)]
    if ede_df_i_pd.shape[0]==0 or ede_df_i_pu.shape[0]==0:
        if return_pct_SNs_close:
            return [], 0., 0.
        else:
            return []
    #-----
    # Get the time values for the power-up and power-down events
    vals_pu = ede_df_i_pu[valuesinterval_col].drop_duplicates().sort_values().tolist()
    vals_pd = ede_df_i_pd[valuesinterval_col].drop_duplicates().sort_values().tolist()
    
    #--------------------------------------------------
    # Estimate beginning of outage, out_t_beg
    #--------------------------------------------------
    try:
        out_t_beg_le = Utilities.find_le(vals_pd, dovs_out_t_beg)
    except:
        if verbose:
            print('Warning: Unable to find out_t_beg_le, so using vals_pd[0]')
        out_t_beg_le = vals_pd[0]
    #-----
    try:
        out_t_beg_ge = Utilities.find_ge(vals_pd, dovs_out_t_beg)
    except:
        if verbose:
            print('Warning: Unable to find out_t_beg_ge, so using out_t_beg_le')
        out_t_beg_ge = out_t_beg_le
    #-----
    if out_t_beg_le==out_t_beg_ge:
        out_t_beg = out_t_beg_le
    else:
        # Note: If out_t_beg_le and out_t_beg_ge are equidistant from dovs_out_t_beg
        #         out_t_beg_le is favored (through <= operator)
        if (dovs_out_t_beg-out_t_beg_le) <= (out_t_beg_ge-dovs_out_t_beg):
            out_t_beg = out_t_beg_le
        else:
            out_t_beg = out_t_beg_ge
            
    #--------------------------------------------------
    # Estimate end of outage, out_t_end
    #--------------------------------------------------
    try:
        out_t_end_ge = Utilities.find_ge(vals_pu, dovs_out_t_end)
    except:
        if verbose:
            print('Warning: Unable to find out_t_end_ge, so using vals_pu[-1]')
        out_t_end_ge = vals_pu[-1]
    #-----
    try:
        out_t_end_le = Utilities.find_le(vals_pu, dovs_out_t_end)
    except:
        if verbose:
            print('Warning: Unable to find out_t_end_le, so using out_t_end_ge')
        out_t_end_le = out_t_end_ge
    #-----
    if out_t_end_le==out_t_end_ge:
        out_t_end = out_t_end_le
    else:
        # Note: If out_t_end_le and out_t_end_ge are equidistant from dovs_out_t_end
        #         out_t_beg_ge is favored (through <= operator)
        if (out_t_end_ge-dovs_out_t_end) <= (dovs_out_t_end-out_t_end_le):
            out_t_end = out_t_end_ge
        else:
            out_t_end = out_t_end_le
            
    #--------------------------------------------------
    # If out_t_beg is greater than or equal to out_t_end, try changing out_t_beg to out_t_beg_le and/or 
    #   out_t_end to out_t_end_ge
    #-----
    # Before vals were split into vals_pd and vals_pu, it seemed like the only way this could happen was if
    #   out_t_beg equals out_t_end, but now I supposed it could happen in general?
    #   In any case, if the assertion hits below, then I should investigate further
    #-----
    if out_t_beg>=out_t_end:
#         assert(out_t_end==out_t_beg)
        # Change either out_t_beg to out_t_beg_le or out_t_end to out_t_end_ge first according to which 
        #   is closer (to make minimal adjustment first)
        # Note: abs calls below should really be necessary, but also no harm
        if np.abs(out_t_beg-out_t_beg_le) <= np.abs(out_t_end_ge-out_t_end):
            out_t_beg = out_t_beg_le
        else:
            out_t_end = out_t_end_ge
        #-----
        # If out_t_beg still >= out_t_end, set both out_t_beg=out_t_beg_le and out_t_end=out_t_end_ge
        # Note: Don't know which was set in the above if/else, so simply set both
        if out_t_beg>=out_t_end:
            out_t_beg = out_t_beg_le
            out_t_end = out_t_end_ge
    #-------------------------
    if out_t_beg>=out_t_end:
        if verbose:
            print('In estimate_broad_outage_time_using_ede, found out_t_beg<out_t_end.')
            print('Returning []')
        return ()
    assert(out_t_beg<out_t_end)
    #--------------------------------------------------
    if not return_pct_SNs_close:
        return (out_t_beg, out_t_end)
    #--------------------------------------------------
    # Find percentage of SNs with PD close to out_t_beg and percentage with PU close to out_t_end
    n_SNs_w_pd_close_to_beg = ede_df_i_pd[np.abs(ede_df_i_pd[valuesinterval_col]-out_t_beg)<out_time_thresh][SN_col].nunique()
    pct_SNs_w_pd_close_to_beg = 100*n_SNs_w_pd_close_to_beg/ede_df_i[SN_col].nunique()
    #-----
    n_SNs_w_pu_close_to_end = ede_df_i_pu[np.abs(ede_df_i_pu[valuesinterval_col]-out_t_end)<out_time_thresh][SN_col].nunique()
    pct_SNs_w_pu_close_to_end = 100*n_SNs_w_pu_close_to_end/ede_df_i[SN_col].nunique()
    #-------------------------
    return (out_t_beg, out_t_end), pct_SNs_w_pd_close_to_beg, pct_SNs_w_pu_close_to_end


def estimate_outage_times_using_ede_for_meter(
    ede_df_i, 
    broad_out_t_beg, 
    broad_out_t_end, 
    expand_search_time=pd.Timedelta('10 minutes'), 
    
    pd_ids=['3.26.0.47', '3.26.136.47'], 
    pu_ids=['3.26.0.216', '3.26.136.216'], 
    
    SN_col='serialnumber', 
    valuesinterval_col='valuesinterval_local', 
    edetypeid_col='enddeviceeventtypeid', 
    verbose=False
):
    r"""
    Given the broad outage time, find any times during the main outage time when power was restored for a single meter.
    NOTE: ede_df_i must be a meter events DataFrame containing data from a single meter.

    Essentially, I am finding any sub-outages within the outage as a whole
    This only makes sense at a serial number level (i.e., ede_df_i must contain events for only a single SN)
    This method, in contrast to the work I put together for Patrick, doesn't care about mismatched power-up/power-down 
      pairs (i.e., e.g., multiple power down events can occur in a row)
    Basically, first find a power down event, then find the next power up.  Call this the first sub-outage
    Then, find the next power down after the first sub-outage, then find the next power up.  Call this the second sub-outage.
    Repeat.
    -----    
    """
    #-------------------------
    # As described above, designed to work for DF containing data from a single meter.
    assert(ede_df_i[SN_col].nunique()==1)
    #-------------------------
    # First, get a better estimate for the broad outage times (which are usually estimated from all meters in an outage 
    #  or simply taken from DOVS)
    out_t_beg_end = estimate_broad_outage_time_using_ede(
        ede_df_i=ede_df_i, 
        dovs_out_t_beg=broad_out_t_beg-expand_search_time, 
        dovs_out_t_end=broad_out_t_end+expand_search_time, 

        pd_ids=pd_ids, 
        pu_ids=pu_ids, 

        return_pct_SNs_close=False, 
        out_time_thresh=pd.Timedelta('10 minutes'), 

        SN_col=SN_col, 
        valuesinterval_col=valuesinterval_col, 
        edetypeid_col=edetypeid_col, 
        verbose=verbose
    )
    if out_t_beg_end:
        assert(len(out_t_beg_end)==2)
        out_t_beg = out_t_beg_end[0]
        out_t_end = out_t_beg_end[1]
    else:
        return []
#     return [[out_t_beg, out_t_end]]
    #-------------------------
    # Get the subset of ede_df_i within the broad outage time (including the outage beginning and end) having power-up
    #   or power-down type IDs.
    ede_df_i_sub = ede_df_i[
        (ede_df_i[valuesinterval_col]>=out_t_beg) & 
        (ede_df_i[valuesinterval_col]<=out_t_end) &
        (ede_df_i[edetypeid_col].isin(pd_ids+pu_ids))
    ].copy()
    #-------------------------
    # The methodology relies on ede_df_i_sub being sorted by time
    # But, first, make sure valuesinterval_col contains time data
    if not is_datetime64_dtype(ede_df_i_sub[valuesinterval_col]):
        ede_df_i_sub[valuesinterval_col] = pd.to_datetime(ede_df_i_sub[valuesinterval_col])
    ede_df_i_sub = ede_df_i_sub.sort_values(by=[valuesinterval_col])
    #-------------------------
    # The procedure is to essentially mark and power-ups with a value of 1 and any power-downs with a value of 0
    # These values are stored in a temporary column (tmp_id_type_col).
    # Then, .diff can be called on this column, and the results (stored in tmp_diff_col) can be used to identify
    #   the beginning and end of the sub-outages.
    #-----
    tmp_id_type_col = Utilities.generate_random_string()
    ede_df_i_sub[tmp_id_type_col] = np.nan
    ede_df_i_sub.loc[ede_df_i_sub[edetypeid_col].isin(pu_ids), tmp_id_type_col] = 1
    ede_df_i_sub.loc[ede_df_i_sub[edetypeid_col].isin(pd_ids), tmp_id_type_col] = 0
    #-----
    tmp_diff_col = Utilities.generate_random_string()
    ede_df_i_sub[tmp_diff_col] = ede_df_i_sub[tmp_id_type_col].diff()
    #-------------------------
    # By definition, the first entry should be a power down event and the last should be a power up event
    #  (because of the use of >= and <= in ede_df_i_sub declaration above)
    assert(ede_df_i_sub.shape[0]>=2)
    #-----
    # If first value is not in pd_ids, remove until found
    if ede_df_i_sub.iloc[0][edetypeid_col] not in pd_ids:
        if verbose:
            print('Warning, first value of ede_df_i_sub not in pd_ids')
        while ede_df_i_sub.iloc[0][edetypeid_col] not in pd_ids:
            ede_df_i_sub = ede_df_i_sub.iloc[1:]
    # If last value is not in pu_ids, remove until found
    if ede_df_i_sub.iloc[-1][edetypeid_col] not in pu_ids:
        if verbose:
            print('Warning, last value of ede_df_i_sub not in pu_ids')
        while ede_df_i_sub.iloc[-1][edetypeid_col] not in pu_ids:
            ede_df_i_sub = ede_df_i_sub.iloc[:-1]
    #-----
    if ede_df_i_sub.shape[0]<2:
        print(f'Could not determine outage times for SN={ede_df_i[SN_col].unique()[0]}')
        return []
    assert(ede_df_i_sub.shape[0]>=2)
    assert(ede_df_i_sub.iloc[0][edetypeid_col] in pd_ids)
    assert(ede_df_i_sub.iloc[-1][edetypeid_col] in pu_ids)
    if ede_df_i_sub.shape[0]==2:
        outg_times = [(ede_df_i_sub.iloc[0][valuesinterval_col], ede_df_i_sub.iloc[-1][valuesinterval_col])]
        # return outg_times
    #-------------------------
    # The .diff operation above leaves the first entry (which is a power-down event) as NaN
    # Make it equal to -1 instead, like all other power-down events
    ede_df_i_sub.loc[ede_df_i_sub.index[0], tmp_diff_col]=-1

    # Get the sub-outages begin (diff = -1) and end (diff = +1) times
    outg_times_beg = ede_df_i_sub[ede_df_i_sub[tmp_diff_col]==-1][valuesinterval_col].tolist()
    outg_times_end = ede_df_i_sub[ede_df_i_sub[tmp_diff_col]==1][valuesinterval_col].tolist()
    assert(len(outg_times_beg)==len(outg_times_end))

    # Combine the begin and end times
    outg_times = list(zip(outg_times_beg, outg_times_end))
    #-------------------------
    # Sanity check on outage times
    for i_outg, outg_time in enumerate(outg_times):
        # Each element should contain a beginning and ending time
        assert(len(outg_time)==2)
        # The beginning time should occur before the ending time
        assert(outg_time[0]<=outg_time[1])
        # This outage should occur after the last outage
        if i_outg>0:
            assert(outg_time[0]>=outg_times[i_outg-1][1])
    #-------------------------
    return outg_times


def estimate_cmi_using_ede_for_meter(
    ede_df_i, 
    broad_out_t_beg, 
    broad_out_t_end, 
    expand_search_time=pd.Timedelta('1 hour'), 
    
    pd_ids=['3.26.0.47', '3.26.136.47'], 
    pu_ids=['3.26.0.216', '3.26.136.216'], 
    
    SN_col='serialnumber', 
    valuesinterval_col='valuesinterval_local', 
    edetypeid_col='enddeviceeventtypeid', 
    return_est_outage_times=False, 
    verbose=False
):
    r"""
    Estimate the outage times using estimate_outage_times_using_ede_for_meter, then calculate the CMI for the customer
    """
    #-------------------------
    outg_times = estimate_outage_times_using_ede_for_meter(
        ede_df_i=ede_df_i, 
        broad_out_t_beg=broad_out_t_beg, 
        broad_out_t_end=broad_out_t_end,
        expand_search_time=expand_search_time, 
        pd_ids=pd_ids, 
        pu_ids=pu_ids, 
        SN_col=SN_col, 
        valuesinterval_col=valuesinterval_col, 
        edetypeid_col=edetypeid_col, 
        verbose=verbose
    )
    #-------------------------
    cmi_i = pd.Timedelta(0)
    for outg_time in outg_times:
        cmi_i += (outg_time[1]-outg_time[0])
    #-------------------------
    if return_est_outage_times:
        return cmi_i, outg_times
    else:
        return cmi_i

In [9]:
def find_outg_rec_nb_in_outg_rec_nbs_in_files_dict(
    outg_rec_nb, 
    outg_rec_nbs_in_files,
    crash_if_not_found=True
):
    #-------------------------
    for file_i, outg_rec_nbs_i in outg_rec_nbs_in_files.items():
        if outg_rec_nb in outg_rec_nbs_i:
            return file_i
    print(f'outg_rec_nb = {outg_rec_nb} not found!')
    if crash_if_not_found:
        assert(0)
    else:
        return None

In [10]:
def invert_file_to_outg_rec_nbs_dict(file_to_outg_rec_nbs_dict):
    r"""
    Input:
        keys   = file paths
        values = outg_rec_nbs contained in given file path
        
    Output:
        keys   = outg_rec_nbs
        values = files containing given outg_rec_nb
    """
    #-------------------------
    outg_rec_nb_to_files_dict = dict()
    for file_i, outg_rec_nbs_i in file_to_outg_rec_nbs_dict.items():
        for outg_rec_nb_ij in outg_rec_nbs_i:
            if outg_rec_nb_ij in outg_rec_nb_to_files_dict.keys():
                outg_rec_nb_to_files_dict[outg_rec_nb_ij].append(file_i)
            else:
                outg_rec_nb_to_files_dict[outg_rec_nb_ij] = [file_i]
    #-------------------------
    return outg_rec_nb_to_files_dict

In [11]:
def check_for_data_in_search_window(
    df_i, 
    t_search_min_max=None, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local'
):
    r"""
    Checks whether or not data are present within the search window
    """
    #-------------------------
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        if t_search_min_max[0] is None:
            t_search_min_max[0] = pd.Timestamp.min
        if t_search_min_max[1] is None:
            t_search_min_max[1] = pd.Timestamp.max
        #-----
        return df_i[
            (df_i[t_int_beg_col]>=t_search_min_max[0]) & 
            (df_i[t_int_end_col]<=t_search_min_max[1])
        ].shape[0]>0
    else:
        return df_i.shape[0]>0

def estimate_outage_times_for_meter(
    df_i, 
    t_search_min_max=None, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
    verbose=True,
    outg_rec_nb_col=None
):
    r"""
    Try to determine when outages occur by looking for periods of time where the meters show values of 0.
    df_i should contain AMI data for a single meter.
    Designed to find multipe groups of zero readings.
    
    Returns a list of dict objects, each having keys: cnsrvtv_t_beg, cnsrvtv_t_end, zero_t_beg, zero_t_end.
    
    IMPORTANT: Due to the 15-minute granularity of the data, the raw output will underestimate the length of the outage
                 between 0-30 minutes (i.e., the outage actually began 0-15 minutes before AMI data received showing meters
                 with 0 values and the outage actually ended 0-15 minutes after last data-point received with meters 
                 showing 0 values).
                ACTUALLY, to be 100% correct, the raw output will underestimate the length of the outage by between 
                  0 and 2*freq of data!!
                  
    REMINDER: The original method, which found zero times and combined them using Utilities.get_fuzzy_overlap, 
                was no good because if there are gaps in the data multiple sub-outages would be split off
              e.g., if a period of zero values lasts for an hour, but there is a 15-minute interval missing data,
                the original method would have split this into two sub-outages
                  
    t_search_min_max:
        Allows the user to set of time subset of the data from which to search for the outages.
        If t_search_min_max is not None, it should be a two-element list/tuple.
        However, one of the elements of the list may be None if, e.g., one wants limits on only the min or max
        
    outg_rec_nb_col:
        Only used for outputting warnings etc. when verbose==True
    """
    #-------------------------
    # If df_i doesn't have any data within search window, return []
    if not check_for_data_in_search_window(
        df_i=df_i, 
        t_search_min_max=t_search_min_max, 
        t_int_beg_col=t_int_beg_col, 
        t_int_end_col=t_int_end_col
    ):
        return []
    #-------------------------
    # As described above, designed to work for DF containing data from a single meter.
    assert(df_i[SN_col].nunique()==1)
    SN_i = df_i[SN_col].unique().tolist()[0] # Used only for error messages
    
    # outg_rec_nb only used for error messages
    if outg_rec_nb_col is not None and outg_rec_nb_col in df_i.columns.tolist():
        outg_rec_nb = df_i[outg_rec_nb_col].unique().tolist()[0]
    else:
        outg_rec_nb = None
    
    #-------------------------
    # For this procedure, only really need the t_int_beg_col, t_int_end_col, value_col (and, SN_col for assertion check)
    df_i = df_i.drop(
        columns=list(set(df_i.columns.tolist()).difference(set([t_int_beg_col, t_int_end_col, value_col, SN_col]))), 
        inplace=False
    )
    
    #-------------------------
    # First, make sure df_i is sorted by t_int_beg_col!
    df_i = df_i.sort_values(by=t_int_beg_col)
    
    #-------------------------
    # It is easier to work with the index in terms of numbers between 0 and df_i.shape[0], therefore
    #   I call reset_index()
    # NOTE: Is storing idx_col really necessary?  I'm thinking it's not...
    assert(df_i.index.nlevels==1)
    if df_i.index.name is not None and df_i.index.name not in df_i.columns.tolist():
        idx_col = df_i.index.name
    else:
        idx_col = Utilities.generate_random_string()
        df_i.index.name = idx_col
    assert(idx_col not in df_i.columns.tolist())
    #-----
    df_i = df_i.reset_index()
    
    
    #-------------------------
    # Find the rows in df_i in which to search for outages
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        #-----
        if t_search_min_max[0] is None:
            idx_search_min = 0
        else:
            idx_search_min = df_i[df_i[t_int_beg_col]>=t_search_min_max[0]].index[0]
        #-----
        if t_search_min_max[1] is None:
            idx_search_max = df_i.shape[0]-1
        else:
            idx_search_max = df_i[df_i[t_int_end_col]<=t_search_min_max[1]].index[-1]
    else:
        idx_search_min = 0
        idx_search_max = df_i.shape[0]-1
        t_search_min_max=[None, None] # Needed only for verbose printing below
    
    #--------------------------------------------------
    # If there is no power (value==0) at the beginning or end of the search window
    #   expand the search window
    #-----
    # NOTE: Below, I want sub_df_i to include idx_search_min and idx_search_max, which is why
    #         the +1 is needed with idx_search_min
    # NOTE: Even if idx_search_max was set to df_i.shape[0]-1, there not no harm
    #         in calling df_i.iloc[i:df_i.shape[0]], whereas calling df_i.iloc[df_i.shape[0]]
    #         would result in an index out-of-bound error!
    #-------------------------
    # If no power at the beginning of the search window
    if df_i.iloc[idx_search_min][value_col]==0:
        # Portion of df_i previous to search window
        df_i_pre = df_i.iloc[:idx_search_min]
        if (df_i_pre[value_col]!=0).any():
            # I want the non-zero value closest to the search window, which, in this case
            #   means the largest index having a non-zero value
            # To achieve this, reverse the order of df_i_pre and use idxmax
            # NOTE: This only works because the indices are integers (and argmax would give the
            #       wrong answer in any case)!  
            idx_search_min = (df_i_pre.iloc[::-1][value_col]!=0).idxmax()
            if verbose:
                print('\n\nExpanding search window to include beginning of first outage')
        else:
            idx_search_min = 0
            if verbose:
                print('\n\nWARNING: could not find beginning of first outage!')
                print('Search window expanded to include minimum time of full df_i')
        #-----
        if verbose:
            print(f'SN = {SN_i}')
            if outg_rec_nb is not None:
                print(f'OUTG_REC_NB = {outg_rec_nb}')
            print(f'\tOrg = {t_search_min_max[0]}\n\tNew = {df_i.iloc[idx_search_min][t_int_beg_col]}')
    #-------------------------
    # If no power at the end of the search window
    if df_i.iloc[idx_search_max][value_col]==0:
        # Portion of df_i following search window
        df_i_post = df_i.iloc[idx_search_max+1:]
        if (df_i_post[value_col]!=0).any():
            # I want the non-zero value closest to the search window, which, in this case
            #   means the smallest index having a non-zero value
            idx_search_max = (df_i_post[value_col]!=0).idxmax()
            if verbose:
                print('\n\nExpanding search window to include ending of last outage')
        else:
            idx_search_max = df_i.shape[0]-1
            if verbose:
                print('\n\nWARNING: could not find ending of last outage')
                print('Search window expanded to include maximum time of full df_i')
        #-----
        if verbose:
            print(f'SN = {SN_i}')
            if outg_rec_nb is not None:
                print(f'OUTG_REC_NB = {outg_rec_nb}')
            print(f'\tOrg = {t_search_min_max[1]}\n\tNew = {df_i.iloc[idx_search_max][t_int_end_col]}')
    #-------------------------
    sub_df_i = df_i.iloc[idx_search_min:idx_search_max+1].copy()
    #--------------------------------------------------
    #-------------------------
    # If no periods with zero values, return empty list
    if not (sub_df_i[value_col]==0).any():
        return []
    
    #-------------------------
    # To be consistent with methods in estimate_outage_times_using_ede_for_meter, where power-up events are assigned
    #   a value of +1 and power-down event assigned a value of 0, I should use !=0 below
    tmp_neq0_col = Utilities.generate_random_string()
    tmp_diff_col = Utilities.generate_random_string()
    #-----
    sub_df_i[tmp_neq0_col] = (sub_df_i[value_col] != 0).astype(int)
    sub_df_i[tmp_diff_col] = sub_df_i[tmp_neq0_col].diff()
    #-------------------------
    # The .diff() operation always leaves the first element as a NaN
    # However, if the first element was the beginning of an outage (i.e., if the value==0)
    #   then the diff should be +1
    if sub_df_i.iloc[0][value_col]==0:
        sub_df_i.loc[sub_df_i.index[0], tmp_diff_col]=-1  
    
    #-------------------------
    # Find the power-down (pd) and power-up (pu) rows
    #-----
    # Outage times are defined by a power loss followed by a power restore.
    # The beginning of a power loss occurs when the value in value_col goes from non-zero to zero
    #   and is located in rows for which sub_df_i[tmp_diff_col]==-1
    # The ending of a power loss occurs when the value in value_col goes from zero to non-zero
    #   and is located in rows for which sub_df_i[tmp_diff_col]==+1
    # NOTE: The use of .reset_index() below ensures that the values returned for rows are numerical index
    #         values from 0 to sub_df_i.shape[0]-1
    # NOTE: Below pd==power-down and pu==power-up.  The pd times are when the first zero values occur (following non-zero
    #         values) and the pu times are when the first non-zero values occur (following zero values)
    #-----
    pd_row_idxs = sub_df_i.reset_index()[sub_df_i.reset_index()[tmp_diff_col]==-1].index.tolist()
    pu_row_idxs = sub_df_i.reset_index()[sub_df_i.reset_index()[tmp_diff_col]==1].index.tolist()
    #-----
    # Sanity checks
    assert((sub_df_i.iloc[pd_row_idxs][tmp_diff_col]==-1).all())
    assert((sub_df_i.iloc[pd_row_idxs][value_col]==0).all())
    #-----
    assert((sub_df_i.iloc[pu_row_idxs][tmp_diff_col]==1).all())
    assert((sub_df_i.iloc[pu_row_idxs][value_col]!=0).all())
    #-------------------------    
    
    #-------------------------
    # As stated above, the pd times are when the first zero values occur (following non-zero
    #   values) and the pu times are when the first non-zero values occur (following zero values)
    #-----
    # pd_row_idxs/zeros_beg_row_idxs/cnsrvtv_beg_row_idxs:
    #   - the idxs denoting when the periods of zeros begin (zeros_beg_row_idxs) is simply pd_row_idxs.
    #     At these times, we know for certain the outage is ongoing, so these are essentially the maximum
    #       times at which the outages could begin
    #   - the conservative estimates for the beginning of the outages is the time period preceding the first zero,
    #       which should be a non-zero value.
    #     At these times, we know for certain (unless the outage is ongoing at the beginning of the data) the outage is not 
    #       ongoing, so these are essentially the minumum times at which the outages could begin
    #     If the first outage is ongoing at the beginning of the data, then zeros_beg_row_idxs[0]==0, in which case the best
    #       we can do is set the conservative row equal to 0 as well
    #-----
    # pu_row_idxs/zeros_end_row_idxs/cnsrvtv_end_row_idxs:
    #   - the idxs denoting when a period of zeros ends with the first non-zero value (cnsrvtv_end_row_idxs) is simply pu_row_idxs.
    #     At these times, we know for certain the outages are no longer ongoin (unless the outage is ongoing at the end of the data).
    #     These are essentially the maximum times at which the outages could end.
    #   - The last zero value (zeros_end_row_idxs) in a group of zeros should be the time period preceding the first non-zero (which 
    #       should be a zero value).
    #     At these times, we know for certain the outage is ongoing, so these are essentially the minimum times at which the outages
    #       could end.
    #----------
    zeros_beg_row_idxs   = pd_row_idxs
    cnsrvtv_beg_row_idxs = [x-1 if x>0 else 0 for x in pd_row_idxs]
    #-----
    cnsrvtv_end_row_idxs = pu_row_idxs
    zeros_end_row_idxs = [x-1 if x>0 else 0 for x in pu_row_idxs]
    #-------------------------
    
    #-------------------------
    # All the conservative estimates should have non-zero values
    # NOTE: Cannot simply include, e.g.,  only values from cnsrvtv_beg_row_idxs which
    #         are greater than zero, as the first pd could be at i=1, making
    #         i=0 a perfectly acceptable value
    #       Basically, one needs to include those in cnsrvtv_beg_row_idxs not 
    #         equal to their counterparts in pd_row_idxs
    assert((sub_df_i.iloc[
        [cnsrvtv_beg_row_idxs[i] for i in range(len(pd_row_idxs)) 
         if cnsrvtv_beg_row_idxs[i]!=pd_row_idxs[i]]
    ][value_col]!=0).all())
    assert((sub_df_i.iloc[cnsrvtv_end_row_idxs][value_col]!=0).all())
    #-----
    # Similarly, the zero times should have zero values!
    assert((sub_df_i.iloc[zeros_beg_row_idxs][value_col]==0).all())
    assert((sub_df_i.iloc[zeros_end_row_idxs][value_col]==0).all())
    
    #-------------------------
    # Sanity check: all conservative beg(end) values should have matching zeros beg(end) value
    assert(len(cnsrvtv_beg_row_idxs)==len(zeros_beg_row_idxs))
    assert(len(zeros_end_row_idxs)==len(cnsrvtv_end_row_idxs))
    #-------------------------
    # In the case where an outage is ongoing at the end of the data, there will be
    #   one more beginning time than ending time.
    # In such a case, the best we can do is set the ending times (zeros and conservative)
    #   equal to the end of the data
    # If verbose, the user will have already been notified of this
    if len(zeros_end_row_idxs)<len(zeros_beg_row_idxs):
        assert(sub_df_i.iloc[-1][value_col]==0)
        zeros_end_row_idxs.append(sub_df_i.shape[0]-1)
        cnsrvtv_end_row_idxs.append(sub_df_i.shape[0]-1)
    assert(len(zeros_end_row_idxs)==len(zeros_beg_row_idxs))
    
    #-------------------------
    # Convert the row indices to times
    cnsrvtv_beg_times = sub_df_i.iloc[cnsrvtv_beg_row_idxs][t_int_beg_col].tolist()
    zeros_beg_times   = sub_df_i.iloc[zeros_beg_row_idxs][t_int_beg_col].tolist()
    #-----
    cnsrvtv_end_times = sub_df_i.iloc[cnsrvtv_end_row_idxs][t_int_end_col].tolist()
    zeros_end_times   = sub_df_i.iloc[zeros_end_row_idxs][t_int_end_col].tolist()
    
    # Assertion not really needed
    assert(
        len(cnsrvtv_beg_times)==
        len(zeros_beg_times)==
        len(cnsrvtv_end_times)==
        len(zeros_end_times)
    )
    #-------------------------
    # Construct return_list of dict objects
    return_list = []
    for i_outg in range(len(cnsrvtv_beg_times)):
        return_list.append(dict(
            cnsrvtv_t_beg = cnsrvtv_beg_times[i_outg], 
            zero_t_beg    = zeros_beg_times[i_outg], 
            zero_t_end    = zeros_end_times[i_outg], 
            cnsrvtv_t_end = cnsrvtv_end_times[i_outg]
        ))
    #-------------------------
    # Sanity check on outage times
    for i_outg, outg_dict_i in enumerate(return_list):
        # Time ordering should be: cnsrvtv_t_beg, zero_t_beg, zero_t_end, cnsrvtv_t_end
        assert(
            outg_dict_i['cnsrvtv_t_beg']<=
            outg_dict_i['zero_t_beg']<=
            outg_dict_i['zero_t_end']<=
            outg_dict_i['cnsrvtv_t_end']
        )
        # This outage should occur after the last outage
        # NOTE: We are using zero_t_beg/_end instead of cnsrvtv_t_beg/_end.
        #       For the case where a meter regains power briefly (briefly here meaning during a single
        #         15T interval), the cnsrvtv_t_beg of the right outage period will actually be BEFORE the
        #         cnsrvtv_t_end of the left outage.
        #       Visually, this situation will result in a triangular structure in the voltage signal data (zeros
        #         to the left, one single peak, followed by zeros to the right).
        #       We use zero_t_beg/_end because the zero_t_beg of the right outage will always be after
        #         the zero_t_end of the left outage period.
        if i_outg>0:
            assert(outg_dict_i['zero_t_beg']>=return_list[i_outg-1]['zero_t_end'])
    #-------------------------
    return return_list

In [12]:
def combine_est_outg_times_with_overlap_cnsrvtv(
    est_outg_times,
    overlap_to_enforce=None
):
    r"""
    The conservative beginning time of an outage can overlap with the convervative ending time of
      the previous outage.
    This will occur, e.g., when, during an outage, a meter regains power for one interval (visually, this
      corresponds to a triangular shape in the voltage data).
    In such instances, this function can be used to combine the two into a single est_outg_time.
    
    overlap_to_enforce:
        If not None:
            Should be a pd.Timedelta.
            It is enforced that overlaps between any two est_outg_times must equal this value
            A typical value is overlap_to_enforce=pd.Timedelta('15T')
        
    """
    #-------------------------
    # We are going to manipulate est_outg_times (e.g., after popping off first element, subsequent changes
    #   to that object will actually change to underlying dict values as well!
    # So make a copy so original is not altered
    est_outg_times = copy.deepcopy(est_outg_times)
    #-------------------------
    if overlap_to_enforce is not None:
        assert(isinstance(overlap_to_enforce, pd.Timedelta))
    #-------------------------
    # First, make sure all elements of est_outg_times are dicts with the expected keys
    expected_keys = ['cnsrvtv_t_beg', 'zero_t_beg', 'zero_t_end', 'cnsrvtv_t_end']
    for est_outg_times_i in est_outg_times:
        assert(isinstance(est_outg_times_i, dict))
        assert(len(set(expected_keys).difference(set(est_outg_times_i.keys())))==0)
    #-----
    # Next, make sure est_outg_times is sorted
    est_outg_times = natsorted(est_outg_times, key=lambda x: x['cnsrvtv_t_beg'])
    #-------------------------
    return_est_outg_times = []
    # Set the first range in return_est_outg_times simply as the first range in the list
    est_outg_times_curr = est_outg_times.pop(0)
    return_est_outg_times.append(est_outg_times_curr)
    #-----
    for est_outg_times_i in est_outg_times:
        if est_outg_times_i['cnsrvtv_t_beg'] > est_outg_times_curr['cnsrvtv_t_end']:
            # cnsrvtv_t_beg_i is after current end, so new interval needed
            return_est_outg_times.append(est_outg_times_i)
            est_outg_times_curr = est_outg_times_i
        else:
            # cnsrvtv_t_beg_i <= current_end, so overlap.
            # The beg of return_est_outg_times[-1] remains the same, 
            #   but the end of return_est_outg_times[-1] should be changed to
            #   the max of current_end and end
            #-----
            if overlap_to_enforce is not None:
                overlap_i = est_outg_times_curr['cnsrvtv_t_end']-est_outg_times_i['cnsrvtv_t_beg']
                assert(overlap_i==overlap_to_enforce)
            #-----
            return_est_outg_times[-1]['zero_t_end'] = max(
                return_est_outg_times[-1]['zero_t_end'], 
                est_outg_times_i['zero_t_end']
            )
            return_est_outg_times[-1]['cnsrvtv_t_end'] = max(
                return_est_outg_times[-1]['cnsrvtv_t_end'], 
                est_outg_times_i['cnsrvtv_t_end']
            )
    return return_est_outg_times

In [13]:
def estimate_outage_times(
    df, 
    t_search_min_max=None, 
    pct_SNs_required_for_outage=0, 
    relax_pct_SNs_required_if_no_outgs_found=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    combine_overlaps=True, 
    verbose=True,
    outg_rec_nb_col=None
):
    r"""
    Try to determine when outages occur by looking for periods of time where some minimum percentage
      of the meters (defined by pct_SNs_required_for_outage) show values of 0.
    Designed to find multipe groups of zero readings.
    Strategy: Basically, reduce df down to one entry per timestamp (instead of multiple SNs per timestamp) whose
                value signifies whether or not the the timestamp can be considered as a zero value or a non-zero value.
              A throw-away SN_col is then added on so that the estimate_outage_times_for_meter method may be used.
              
    !!!!! IMPORTANT !!!!!
        pct_SNs_required_for_outage is the minimum percentage of available data at each timestamp needed to be zero
          for the timestamp to be considered a zero value.
        In many cases, this is equal to the percentage of SNs with a zero value, BUT NOT ALWAYS.
        If, e.g., one meter in an outage has timestamps offset by a minute compared to the others, this will 
          not be the case!
    
    Returns a list of dict objects, each having keys: cnsrvtv_t_beg, cnsrvtv_t_end, zero_t_beg, zero_t_end.
    
    IMPORTANT: Due to the 15-minute granularity of the data, the raw output will underestimate the length of the outage
                 between 0-30 minutes (i.e., the outage actually began 0-15 minutes before AMI data received showing meters
                 with 0 values and the outage actually ended 0-15 minutes after last data-point received with meters 
                 showing 0 values).
                ACTUALLY, to be 100% correct, the raw output will underestimate the length of the outage by between 
                  0 and 2*freq of data!!
                  
    t_search_min_max:
        Allows the user to set of time subset of the data from which to search for the outages.
        If t_search_min_max is not None, it should be a two-element list/tuple.
        However, one of the elements of the list may be None if, e.g., one wants limits on only the min or max
        
    outg_rec_nb_col:
        Only used for outputting warnings etc. when verbose==True
    """
    #-------------------------
    # If df doesn't have any data within search window, return []
    if not check_for_data_in_search_window(
        df_i=df, 
        t_search_min_max=t_search_min_max, 
        t_int_beg_col=t_int_beg_col, 
        t_int_end_col=t_int_end_col
    ):
        return []
    #------------------------- 
    # outg_rec_nb only used for error messages
    if outg_rec_nb_col is not None and outg_rec_nb_col in df.columns.tolist():
        outg_rec_nb = df[outg_rec_nb_col].unique().tolist()[0]
    else:
        outg_rec_nb = None
        
    #------------------------- 
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        if t_search_min_max[0] is None:
            t_search_min_max[0] = pd.Timestamp.min
        if t_search_min_max[1] is None:
            t_search_min_max[1] = pd.Timestamp.max
        #-----
        df = df[
            (df[t_int_beg_col]>=t_search_min_max[0]) & 
            (df[t_int_end_col]<=t_search_min_max[1])
        ]
    
    #-------------------------
    # For this procedure, only really need the t_int_beg_col, t_int_end_col, value_col
    df = df.drop(
        columns=list(set(df.columns.tolist()).difference(set([t_int_beg_col, t_int_end_col, value_col]))), 
        inplace=False
    )
    
    #-------------------------
    # Make sure df is sorted by t_int_beg_col!
    df = df.sort_values(by=t_int_beg_col)
    
    #-------------------------
    # NOTE: Cannot use >= below!
    #       A typical value is to set pct_SNs_required_for_outage=0, and if using >= below,
    #         everything would come back as true, regardless of whether or not any 0 values
    #         were registered.
    zeros_df = df.groupby([t_int_beg_col, t_int_end_col]).apply(
        lambda x: 100*((x[value_col]==0).sum()/x.shape[0])>pct_SNs_required_for_outage
    )

    # To be able to feed into estimate_outage_times_for_meter, we actually want the opposite of zeros_df,
    #   where non-zero values are assigned 1 and zero values are assigned 0
    neq0_df = (~zeros_df).astype(int)

    #-------------------------
    # At this point, neq0_df is a series with indices equal to starttimeperiod_local and values equal to 0 or 1, 
    #   indicating whether or not the time period as a whole can be treated as value 0 (without power) or 1 (with power)
    # Name the series value_col, so after calling reset_index to column will equal value_col, as needed for input into
    #   estimate_outage_times_for_meter
    neq0_df.name = value_col
    neq0_df = neq0_df.reset_index()

    #-------------------------
    # estimate_outage_times_for_meter requires a SN_col with one unique value, so satisfy that
    SN_col='SN'
    if outg_rec_nb is None:
        SN_val = 'AGG_OUTAGE'
    else:
        SN_val = f'AGG_OUTAGE {outg_rec_nb}'
    neq0_df[SN_col] = SN_val

    #-------------------------
    # Now, simply feed into estimate_outage_times_for_meter
    return_list = estimate_outage_times_for_meter(
        df_i=neq0_df, 
        t_search_min_max=t_search_min_max, 
        t_int_beg_col=t_int_beg_col, 
        t_int_end_col=t_int_end_col, 
        value_col=value_col, 
        SN_col=SN_col, 
        verbose=verbose,
        outg_rec_nb_col=outg_rec_nb_col
    )

    #-------------------------
    if len(return_list)==0 and relax_pct_SNs_required_if_no_outgs_found and pct_SNs_required_for_outage>0:
        # NOTE: Important below that input relax_pct_SNs_required_if_no_outgs_found be set to False
        #       OTHERWISE INFINITE LOOP WHEN NO OUTAGE TIMES FOUND!
        print('No outage_times found in estimate_outage_times, so relaxing the pct_SNs_required contraint')
        return_list = estimate_outage_times(
            df=df, 
            t_search_min_max=t_search_min_max, 
            pct_SNs_required_for_outage=0, 
            relax_pct_SNs_required_if_no_outgs_found=False, 
            t_int_beg_col=t_int_beg_col, 
            t_int_end_col=t_int_end_col, 
            value_col=value_col, 
            verbose=verbose,
            outg_rec_nb_col=outg_rec_nb_col
        )
        
    #-------------------------
    if combine_overlaps and len(return_list)>0:
        return_list = combine_est_outg_times_with_overlap_cnsrvtv(
            est_outg_times=return_list,
            overlap_to_enforce=None
        )
    
    #-------------------------
    return return_list

# COMBINE AMI AND EDE

In [14]:
def outg_time_audit(
    outg_est_times_i, 
    dovs_outg_t_beg_end=None, 
    outg_times_ede=None, 
    selection_method='min', 
    return_all_best_ests=False
):
    r"""
    Given the zero times for the (sub)outage, frequency of the data, and DOVS outage data and/or EDE outage times,
    return the best estimate for the outage time.
    
    NOTE: To be certain, this explicitly places preference on the zero times as found by the 15-minute interval.
          The conservative values found using that data are replaced on if dovs and/or ede data supply a value that is
            consistent with the 15-minute interval data (i.e., in the uncertainty intervals, see below for more info).
          Therefore, if dovs or ede complete disagree with 15-minute interval, the latter is always chosen.
    
    The conservative estimates for the outage time are found by subtracting the frequency of the data (freq) from the first
      zero time and adding freq to the last zero time.
    The uncertainty intervals are defined as:
        [first zero - freq, first zero]
        [last zero,         last zero + freq]
    If dovs and/or ede supply times within the uncertainty region, the values used are chosen according to selection_method: 
        selection_method=='min':       those minimizing the outage time are chosen
        selection_method=='max':       those maximizing the outage time are chosen
        selection_method=='dovs pref': dovs times preferred, meaning dovs times are chosen if both dovs and ede available,
                                         but ede times are taken if available when dovs are not
        selection_method=='dovs only':  dovs times are chosen if available.  
                                         If dovs not available but ede is, ede ignored
        selection_method=='ede pref':  ede times preferred, meaning ede times are chosen if both dovs and ede available
                                         but dovs times are taken if available when ede are not
        selection_method=='ede only':  ede times are chosen if available.  
                                         If ede not available but dovs is, dovs ignored
                                         
    outg_times_ede:
        this can be a list of 2-element lists representing multiple outage beg/end pairs (if, e.g., found using
          estimate_outage_times_using_ede_for_meter)
        OR
        this can be a dict with two keys, pd_times and pu_times, the values of which are lists containing the 
          associated times (if, e.g., found using get_pd_pu_times_for_meter)
    
    Given the conservative estimates for the outage time (and frequency of the data), and DOVS outage data and/or EDE outage times,
    return the best estimate for the outage time.
    
    cnsrvtv_beg_end:
        The conservative estimates for the beginning and end of the outage.
        These are typically the period before the first zero and the period after the last zero. 
    
    NOTE: This assumes the user wants a conservative estimate.! 
            The non-conservative estimate essentially counts to times the meter registered a zero count, so no outside audit needed.
    """
    #-------------------------
    assert(isinstance(outg_est_times_i, dict))
    assert(len(set(outg_est_times_i.keys()).difference(set(['cnsrvtv_t_beg', 'cnsrvtv_t_end', 'zero_t_beg', 'zero_t_end'])))==0)
    
    #-------------------------
    # First, find the conservative outage beginning and ending
    cnsrvtv_beg_end = [outg_est_times_i['cnsrvtv_t_beg'], outg_est_times_i['cnsrvtv_t_end']]
    zero_t_beg_end  = [outg_est_times_i['zero_t_beg'], outg_est_times_i['zero_t_end']]
    #-------------------------
    if not dovs_outg_t_beg_end and not outg_times_ede:
        return cnsrvtv_beg_end
    #-------------------------
    assert(selection_method in ['min', 'max', 'dovs pref', 'dovs only', 'ede pref', 'ede only'])
    #----------------------------------------------------------------------------------------------------
    dovs_best_est = [None, None]
    if dovs_outg_t_beg_end:
        # If the outage information from DOVS is between the conservative beginning/end and the
        #   first/last recorded zero, DOVSs may be used in audit
        if dovs_outg_t_beg_end[0]>cnsrvtv_beg_end[0] and dovs_outg_t_beg_end[0]<zero_t_beg_end[0]:
            dovs_best_est[0] = dovs_outg_t_beg_end[0]
        #-----
        if dovs_outg_t_beg_end[1]<cnsrvtv_beg_end[1] and dovs_outg_t_beg_end[1]>zero_t_beg_end[1]:
            dovs_best_est[1] = dovs_outg_t_beg_end[1]
    #----------------------------------------------------------------------------------------------------
    ede_best_est = [None, None]
    if outg_times_ede is not None:
        # Expecting outg_times_ede to be a list of outage times or a dict with pd_times, pu_times keys
        assert(Utilities.is_object_one_of_types(outg_times_ede, [list, dict]))
        #----------
        if isinstance(outg_times_ede, list) and len(outg_times_ede)>0:
            # Expecting outg_times_ede to be a list of outage times, so list of two-element lists
            outg_times_ede_shape = np.array(outg_times_ede).shape
            assert(outg_times_ede_shape[1]==2)
            #-----
            # Unzip outg_times_ede, taking it from a list of outage times (2-element lists) to a list of two lists,
            #   one containing the beginning times and one containing the end times
            # This is essentially a transpose
            outg_times_beg_end_ede = list(zip(*outg_times_ede))
            #-----
            # Expecting outg_times_beg_end_ede to be a list containing two lists
            outg_times_beg_end_ede_shape = np.array(outg_times_beg_end_ede).shape
            assert(outg_times_beg_end_ede_shape[0]==2)

            # Since this is a transpose...
            assert(outg_times_ede_shape[0]==outg_times_beg_end_ede_shape[1])
            #-----
            outg_times_beg_ede = outg_times_beg_end_ede[0]
            outg_times_end_ede = outg_times_beg_end_ede[1]
        else:
            outg_times_beg_ede=[]
            outg_times_end_ede=[]
        #----------
        if isinstance(outg_times_ede, dict):
            assert('pd_times' in outg_times_ede.keys() and 'pu_times' in outg_times_ede.keys())
            outg_times_beg_ede = outg_times_ede['pd_times']
            outg_times_end_ede = outg_times_ede['pu_times']

        #--------------------------------------------------
        # If multiple suitable beg/ends are found, those which maximize the outage time are chosen
        #   i.e., min of ede_best_ests_beg and max of ede_best_ests_end
        # This keeps in line with our strategy of being conservative at each step.
        #-----
        # To be certain, if outg_times_ede is a list, meaning it is a collection of (outg_beg,outg_end pairs) 
        #   for different sub-outages, the method below allows one sub-outage to be used to resolve the ambiguity
        #   in the start time and another to resolve that of the end time.
        #-------------------------
        ede_best_ests_beg = []
        for beg_t_i in outg_times_beg_ede:
            if beg_t_i>cnsrvtv_beg_end[0] and beg_t_i<zero_t_beg_end[0]:
                ede_best_ests_beg.append(beg_t_i)
        #-----
        if ede_best_ests_beg:
            ede_best_est_beg = np.min(ede_best_ests_beg)
        else:
            ede_best_est_beg = None #excplicitly make None for later
        #-------------------------
        ede_best_ests_end = []
        for end_t_i in outg_times_end_ede:
            if end_t_i<cnsrvtv_beg_end[1] and end_t_i>zero_t_beg_end[1]:
                ede_best_ests_end.append(end_t_i)
        #-----
        if ede_best_ests_end:
            ede_best_est_end = np.max(ede_best_ests_end)
        else:
            ede_best_est_end = None #excplicitly make None for later
        #--------------------------------------------------
        ede_best_est = [ede_best_est_beg, ede_best_est_end]
    #----------------------------------------------------------------------------------------------------
    # NOTE: dovs always 0th element, ede 1st element
    best_ests_beg = [dovs_best_est[0], ede_best_est[0]]
    best_ests_end = [dovs_best_est[1], ede_best_est[1]]
    #--------------------------------------------------
    if np.all([x is None for x in best_ests_beg]):
        best_est_beg = None
    else:
        # NOTE: because of if statement above, we can be certain here there is at least one
        #       non-null value, so taking min/max should be safe (since not finding min/max of empty list)
        if selection_method=='min':
            best_est_beg = np.max([x for x in best_ests_beg if x])
        elif selection_method=='max':
            best_est_beg = np.min([x for x in best_ests_beg if x])
        elif selection_method=='dovs pref' or selection_method=='dovs only':
            if best_ests_beg[0]:
                best_est_beg = best_ests_beg[0]
            else:
                if selection_method=='dovs pref':
                    assert(best_ests_beg[1])
                    best_est_beg = best_ests_beg[1]
                else:
                    #selection_method=='dovs only' case
                    best_est_beg = None
        elif selection_method=='ede pref' or selection_method=='ede only':
            if best_ests_beg[1]:
                best_est_beg = best_ests_beg[1]
            else:
                if selection_method=='ede pref':
                    assert(best_ests_beg[0])
                    best_est_beg = best_ests_beg[0]
                else:
                    #selection_method=='ede only' case
                    best_est_beg = None
        else:
            assert(0)
    #-------------------------
    if np.all([x is None for x in best_ests_end]):
        best_est_end = None
    else:
        # NOTE: because of if statement above, we can be certain here there is at least one
        #       non-null value, so taking min/max should be safe (since not finding min/max of empty list)
        if selection_method=='min':
            best_est_end = np.min([x for x in best_ests_end if x])
        elif selection_method=='max':
            best_est_end = np.max([x for x in best_ests_end if x])
        elif selection_method=='dovs pref' or selection_method=='dovs only':
            if best_ests_end[0]:
                best_est_end = best_ests_end[0]
            else:
                if selection_method=='dovs pref':
                    assert(best_ests_end[1])
                    best_est_end = best_ests_end[1]
                else:
                    #selection_method=='dovs only' case
                    best_est_end = None
        elif selection_method=='ede pref' or selection_method=='ede only':
            if best_ests_end[1]:
                best_est_end = best_ests_end[1]
            else:
                if selection_method=='ede pref':
                    assert(best_ests_end[0])
                    best_est_end = best_ests_end[0]
                else:
                    #selection_method=='ede only' case
                    best_est_end = None
        else:
            assert(0)
    #----------------------------------------------------------------------------------------------------
    # By definition, if any best estimates are found, they are in the uncertainty intervals, and therefore
    #  reduce the outage time in comparison to the conservative estimates
    if best_est_beg:
        assert(best_est_beg > cnsrvtv_beg_end[0])
        final_out_t_beg = best_est_beg
    else:
        final_out_t_beg = cnsrvtv_beg_end[0]
    #-----
    if best_est_end:
        assert(best_est_end < cnsrvtv_beg_end[1])
        final_out_t_end = best_est_end
    else:
        final_out_t_end = cnsrvtv_beg_end[1]
    #-------------------------
    if return_all_best_ests:
        all_best_est_dict = dict(
            ede=ede_best_est, 
            dovs=dovs_best_est,
            conservative=cnsrvtv_beg_end, 
            zero_times=zero_t_beg_end,
            winner = [final_out_t_beg, final_out_t_end]
        )
        return final_out_t_beg, final_out_t_end, all_best_est_dict
    else:
        return final_out_t_beg, final_out_t_end

In [15]:
# Default arg for ede_df_i will be None, and if None, old functionality without EDE in place
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# TODO : Don't need freq anymore@@!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def calculate_mi_for_meter_ami_w_ede_help(
    df_i, 
    ede_df_i, 
    t_search_min_max=None, 
    conservative_estimate=True, 
    dovs_outg_t_beg_end=None, 
    freq=None, 
    est_ede_kwargs=None, 
    audit_selection_method='min', 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
    return_all_best_ests=False
):
    r"""
    Find the times for which df_i registers a value of 0 and use that information to calculate the Minutes Interrupted (MI)
      for the meter.
    df_i should contain AMI data for a single meter.
      
    t_search_min_max:
        Allows the user to set of time subset of the data from which to search for the outages/calculate mi.
        If t_search_min_max is not None, it should be a two-element list/tuple.
        However, one of the elements of the list may be None if, e.g., one wants limits on only the min or max
        
    conservative_estimate:
        It appears that AMI meters only report values of 0 if all readings in the 15-minute period are zero.
        I believe that if at least one value is non-zero, the AMI will report a non-zero value.
            e.g., if power out from 12:00 to 12:14, but turns on from 12:14 to 12:15, the 12:15 reading will be non-zero
        Thus, simply calculating the period of time when the measurements are 0 inherently underestimates the actual outage time.
        The more conservative approach, which typically overestimates the MI, would be to add 2*freq to MI, where freq is the 
          frequency of the data (typically 15 minutes).
            To elaborate a bit more: the outage actually begins in the interval before the first 0 value is regiesterd and ends
              in the interval after the last 0 is registered.
        If conservative_estimate==True, 2*freq is added to the MI UNLESS dovs_outg_t_beg_end is supplied (see description below)
        
    dovs_outg_t_beg_end:
        Only has an affect if conservative_estimate==True
        If the outage beginning and ending times are supplied from DOVS, they can sometimes be used in place of the conservative
          starting and ending times to help resolve the uncertainty.
        Essentially, if a DOVS time falls within the uncertainty interval, it can be used in place of the conservative time (where
          the uncertainty interval is the time before/after the first/last recorded zero value and the preceding/following interval).
            i.e., on the front end, use DOVS if time between 0-freq before last zero value
                  on the back end, use DOVS if time between 0-freq after last zero value
        If a DOVS time is well outside of the uncertainty interval (e.g., if the DOVS start time is well before or well after the first
          recorded 0 value), it cannot be used.
          
    freq:
        The user can supply the frequency of the data if desired or needed.
        If freq is None, it will be inferred from df_i.
        In general, probably safest to just let the algorithm infer the frequency.
        I suppose this could be useful to save some time in large data runs over many outages and serial numbers (although I doubt
          the savings will be significant).
        It could also be useful if for whatever reason there is an issue inferring the frequency from the data (e.g., maybe a measurement
          falls slightly outside of the normal 15-minute intervals, like 12:44:59 or similar)
          
    est_ede_kwargs:
        Keyword arguments to input in estimate_outage_times_using_ede_for_meter.
        NOTE: ede_df_i, broad_out_t_beg, broad_out_t_end, and expand_search_time SHOULD NOT be included
                in est_ede_kwargs, as the function will set these        
    """
    #-------------------------
    # As described above, designed to work for DF containing data from a single meter.
    assert(df_i[SN_col].nunique()==1)
    if ede_df_i is not None:
        assert(ede_df_i[SN_col].nunique()==1)
        assert(df_i[SN_col].unique().tolist()[0]==ede_df_i[SN_col].unique().tolist()[0])
    
    #-------------------------
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        if t_search_min_max[0] is None:
            t_search_min_max[0] = pd.Timestamp.min
        if t_search_min_max[1] is None:
            t_search_min_max[1] = pd.Timestamp.max
    #-------------------------
    # Make sure values sorted
    df_i = df_i.sort_values(by=t_int_beg_col)
    #-------------------------
    # Need frequency of the data for conservative estimates
    if freq is None:
        freq = Utilities_df.determine_freq_in_df_col(
            df=df_i, 
            groupby_SN=False, 
            return_val_if_not_found='error', 
            assert_single_freq=True, 
            assert_expected_freq_found=False, 
            expected_freq=pd.Timedelta('15 minutes'), 
            time_col=t_int_beg_col, 
            SN_col=SN_col
        )
    #--------------------------------------------------
    # Find the time periods where all values are zero
    outg_est_times = estimate_outage_times_for_meter(
        df_i=df_i, 
        t_search_min_max=t_search_min_max, 
        t_int_beg_col=t_int_beg_col, 
        t_int_end_col=t_int_end_col, 
        value_col=value_col, 
        SN_col=SN_col
    )
    #--------------------------------------------------
    # Grab the outage times from ede
    if ede_df_i is not None:
        if not est_ede_kwargs:
            est_ede_kwargs = {}
        #broad_out_t_beg = t_search_min_max[0]
        #broad_out_t_end = t_search_min_max[1]
        # Since estimate_outage_times_for_meter is able to expand the search window, instead of
        #   using broad_out_t_beg_end = t_search_min_max, I should set these values equal to the
        #   min/max of the conservative estimates from outg_est_times
        if len(outg_est_times)>0:
            broad_out_t_beg = np.min([x['cnsrvtv_t_beg'] for x in outg_est_times])
            broad_out_t_end = np.max([x['cnsrvtv_t_end'] for x in outg_est_times])
        else:
            broad_out_t_beg = t_search_min_max[0]
            broad_out_t_end = t_search_min_max[1]
        dflt_est_ede_kwargs = dict(
            ede_df_i=ede_df_i, 
            broad_out_t_beg=broad_out_t_beg, 
            broad_out_t_end=broad_out_t_end,
            expand_search_time=pd.Timedelta(0), 
            use_full_ede_outgs=False, 
            pd_ids=['3.26.0.47', '3.26.136.47'], 
            pu_ids=['3.26.0.216', '3.26.136.216'], 
            SN_col='serialnumber', 
            valuesinterval_col='valuesinterval_local', 
            edetypeid_col='enddeviceeventtypeid', 
            verbose=False
        )
        #-----
        # Make sure none of the kwargs set by function are included in est_ede_kwargs
        supplied_est_ede_kwargs = ['ede_df_i', 'broad_out_t_beg', 'broad_out_t_end', 'expand_search_time']
        assert(len(set(est_ede_kwargs.keys()).intersection(set(supplied_est_ede_kwargs)))==0)
        #-----
        # Want est_ede_kwargs to be kept over dflt_est_ede_kwargs, so use to_supplmnt_dict=est_ede_kwargs in the following
        est_ede_kwargs = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=est_ede_kwargs, 
            default_values_dict=dflt_est_ede_kwargs, 
            extend_any_lists=False, 
            inplace=False
        )
        use_full_ede_outgs = est_ede_kwargs.pop('use_full_ede_outgs')
        #-----
        if use_full_ede_outgs:
            outg_times_ede = estimate_outage_times_using_ede_for_meter(**est_ede_kwargs)
        else:
            outg_times_ede = get_pd_pu_times_for_meter(
                ede_df_i=ede_df_i, 
                t_search_min_max=[est_ede_kwargs['broad_out_t_beg'], est_ede_kwargs['broad_out_t_end']], 
                pd_ids=est_ede_kwargs['pd_ids'], 
                pu_ids=est_ede_kwargs['pu_ids'], 
                SN_col=est_ede_kwargs['SN_col'], 
                valuesinterval_col=est_ede_kwargs['valuesinterval_col'], 
                edetypeid_col=est_ede_kwargs['edetypeid_col']
            )
    else:
        outg_times_ede = None
    #--------------------------------------------------
    # Iterate through outg_est_times, calculate mi_j for each, and add to total mi
    mi=pd.Timedelta(0)
    all_best_ests = []
    for outg_est_times_j in outg_est_times:
        #-------------------------
        #TODO: Is there really any reason to find times from df_i?
        #      Can't I just directly use outg_est_times_j?
        #      Only point seems to be to enforce assertion, but probably 
        #        the definition of estimate_outage_times_for_meter requires that outcome?
        # First, grab the subset of the data corresponding to the zero time
        df_i_zr_j = df_i[
            (df_i[t_int_beg_col]>=outg_est_times_j['zero_t_beg']) & 
            (df_i[t_int_end_col]<=outg_est_times_j['zero_t_end'])
        ]
        #-------------------------
        # Since zero_t_end/zero_t_beg keys used above, all values should be zero
        assert((df_i_zr_j[value_col]==0).all())

        #-------------------------
        # Non-conservative MI, simply the difference in time between the last zero value and the first
        # As mentioned elsehwere, this inherently underestimates the MI
        mi_j = df_i_zr_j.iloc[-1][t_int_end_col]-df_i_zr_j.iloc[0][t_int_beg_col]

        if conservative_estimate:
            outg_j_beg_end = outg_time_audit(
                outg_est_times_i=outg_est_times_j, 
                dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
                outg_times_ede=outg_times_ede, 
                selection_method=audit_selection_method, 
                return_all_best_ests=return_all_best_ests
            )
            outg_j_beg, outg_j_end = outg_j_beg_end[0], outg_j_beg_end[1]
            mi_j = outg_j_end-outg_j_beg
            #-----
            if return_all_best_ests:
                assert(len(outg_j_beg_end)==3)
                all_best_ests_j = outg_j_beg_end[2]
                #-----
                all_best_ests.append(all_best_ests_j)
        #-------------------------
        mi += mi_j
    #-------------------------
    mi = mi.total_seconds()/60
    if return_all_best_ests:
        return mi, all_best_ests
    else:
        return mi

In [16]:
def get_nonoverlapping_search_windows(
    est_outg_times, 
    expand_outg_search_time
):
    r"""
    Given a list of estimated outage times and expand search times, return a list of non-overlapping t_search_min_max values.
    Returns a list of two-element lists (0th element is t_search_min, 1st is t_search_max).
    The length of the return list will equal that of est_outg_times
    
    NOTE: If I ever decide to move to asymmetric expand_outg_search_time, I believe this method should still work.
          However, one should carefully verify instead of assuming.
    """
    #-------------------------
    assert(
        Utilities.is_object_one_of_types(est_outg_times, [list, tuple]) and
        Utilities.are_all_list_elements_one_of_types(est_outg_times, [list, tuple]) and
        Utilities.are_list_elements_lengths_homogeneous(est_outg_times, length=2)
    )

    #--------------------------------------------------
    # First, go through and naively set t_search values
    # To more easily find overlapping regions, define a left, center, and right search window for each outage, where:
    #   left (left)   = search period preceding outage, ending at est. outage begin
    #   center (cntr) = est. outage begin to est. outage end
    #   right (rght)  = search period following outage, beginning at est. outage end
    # The left and right windows will be of length expand_outg_search_time UNLESS they overlap with another outage, in which
    #   case the endpoint of the window will be defined by the overlapping outage.
    #     left - check if overlaps with est. outage end of preceding outage.
    #            If overlaps, minimum of left set to est. outage end of preceding outage.
    #     rght - check if overlaps with est. outage beginning of following outage.
    #            If overlaps, maximum of right set to est. outage beginning of following outage.
    # NOTE: Ensuring the search windows themselves do not overlap occurs in the next for loop, not this one!
    #       This is just a rough chopping
    # NOTE: After completing the code, no real need to keep 'cntr' windows, but no harm in keeping (might be useful later?)
    #-----
    search_windows_coll = []
    est_outg_times = natsorted(est_outg_times)
    for i_outg in range(len(est_outg_times)):
        est_outg_times_i = est_outg_times[i_outg]
        #-----
        # Simple sanity check: ensure outage end occurs after outage beginning!
        assert(len(est_outg_times_i)==2)
        assert(est_outg_times_i[1] > est_outg_times_i[0])
        # Sanity check: the given estimate outage times should not overlap!
        # Note: Sufficient to check that beginning of current is greater than ending of preceding
        if i_outg>0:
            assert(est_outg_times_i[0] >= est_outg_times[i_outg-1][1])
        #-------------------------
        # No adjustment needed on center window
        search_windows_i = dict(cntr=est_outg_times_i)
        #-------------------------
        # Left window
        if i_outg==0:
            # First element, so no preceding outage to compare
            search_windows_i['left']=[
                est_outg_times_i[0]-expand_outg_search_time, 
                est_outg_times_i[0]
            ]
        else:
            # Desired left window = est_outg_times_i[0]-expand_outg_search_time to est_outg_times_i[0]
            # Check if minimum value of desired left window overlaps with est. outage end of preceding outage
            # NOTE: im1 == i minus 1
            left_i = [est_outg_times_i[0]-expand_outg_search_time, est_outg_times_i[0]]
            est_outg_times_im1 = est_outg_times[i_outg-1]
            if left_i[0]<est_outg_times_im1[1]:
                left_i[0] = est_outg_times_im1[1]
            #-----
            search_windows_i['left']=left_i
        #-------------------------
        # Right window
        if i_outg==len(est_outg_times)-1:
            # Last element, so no following outage to compare
            search_windows_i['rght']=[
                est_outg_times_i[1], 
                est_outg_times_i[1]+expand_outg_search_time
            ]
        else:
            # Desired right window = est_outg_times_i[1] to est_outg_times_i[1]+expand_outg_search_time
            # Check if maximum value of desired right window overlaps with est. outage beginning of following outage
            # NOTE: ip1 == i plus 1
            rght_i = [est_outg_times_i[1], est_outg_times_i[1]+expand_outg_search_time]
            est_outg_times_ip1 = est_outg_times[i_outg+1]
            if rght_i[1]>est_outg_times_ip1[0]:
                rght_i[1] = est_outg_times_ip1[0]
            #-----
            search_windows_i['rght']=rght_i
        #-------------------------
        search_windows_coll.append(search_windows_i)
    #--------------------------------------------------
    # Now, go back through and resolve any overlaps.
    # If overlaps exist, set the boundary equal to the midpoint of the overlap.
    # It should be sufficient to go through and compare the left search window of the
    #   current element to the right of the previous
    assert(len(search_windows_coll)==len(est_outg_times))
    for i_outg in range(len(search_windows_coll)):
        if i_outg==0:
            continue
        search_windows_i   = search_windows_coll[i_outg]
        search_windows_im1 = search_windows_coll[i_outg-1]
        #-----
        left_i   = search_windows_i['left']
        rght_im1 = search_windows_im1['rght']
        #-----
        # Sanity check: Due to rough clipping done in first iteration:
        #   max of left_i (outg. beg. i) should be greater than (or eq. to) max of rght_im1
        #   min of rght_im1 (outg. end i-1) should be less than (or eq. to) min of left_i
        assert(left_i[1] >= rght_im1[1])
        assert(rght_im1[0] <= left_i[0])
        #-------------------------
        # If there is no overlap, continue
        if left_i[0] >= rght_im1[1]:
            continue
        #-------------------------
        # Overlap exists, so resolve by setting endpoints equal to midpoint of overlap
        #-----
        # The overlap min is the maximum of the mins
        # The overlap max is the minimum of the maxs
        ovrlp_min = np.max([left_i[0], rght_im1[0]])
        ovrlp_max = np.min([left_i[1], rght_im1[1]])
        #-----
        ovrlp_mid = Utilities_dt.calc_dt_mean([ovrlp_min, ovrlp_max])
        #-----
        # Update values in search_windows_coll (update min of left_i and max of right_im1 to equal ovrlp_mid)
        search_windows_coll[i_outg]['left'][0]   = ovrlp_mid
        search_windows_coll[i_outg-1]['rght'][1] = ovrlp_mid
        
    #--------------------------------------------------
    # Finally, get the final search windows as the min of left and max of right
    t_search_min_max_coll = []
    for search_windows_i in search_windows_coll:
        left_i = search_windows_i['left']
        rght_i = search_windows_i['rght']
        t_search_min_max_coll.append([left_i[0], rght_i[1]])
        
    #--------------------------------------------------
    # Final sanity check!
    assert(len(est_outg_times)==len(t_search_min_max_coll))
    for i_outg in range(len(t_search_min_max_coll)):
        assert(t_search_min_max_coll[i_outg][1]>t_search_min_max_coll[i_outg][0])
        if i_outg>0:
            assert(t_search_min_max_coll[i_outg][0]>=t_search_min_max_coll[i_outg-1][1])
    #--------------------------------------------------
    return t_search_min_max_coll

In [17]:
def convert_all_best_ests_dict_to_df(all_best_ests_dict):
    r"""
    Convert all_best_ests_dict to pd.DataFrame object.
    all_best_ests_dict is returned by, e.g., calculate_ci_cmi_w_ami_w_ede_help.
    This is a pretty specific function, so it should probably only be used within calculate_ci_cmi_w_ami_w_ede_help
      or directly on an object returned by it.
    NOTE: SNs for which no suboutages were found are EXCLUDED from df!
    -----
    all_best_ests_dict:
        It should be a dictionary whose keys equal serial numbers in an outage.
        Each key is a list of dict objects.
            Each dict object represents on suboutage time, and contains keys for the various estimates.
            The length of the list equals the number of suboutages found (can be zero if none found)
            Typical keys are: 'zero_times', 'conservative', 'winner', 'ede', and 'dovs'
            For each key, there is a two-element list representing the beg/end times.
    EX:
        {'645792023': [],
         '645779036': 
           [
             {
              'ede': [Timestamp('2023-01-01 01:22:37'), None],
              'dovs': [Timestamp('2023-01-01 01:22:00'), None],
              'conservative': [Timestamp('2023-01-01 01:15:00'), Timestamp('2023-01-01 02:30:00')],
              'winner': [Timestamp('2023-01-01 01:22:37'), Timestamp('2023-01-01 02:30:00')],
              'zero_times': [Timestamp('2023-01-01 01:30:00'), Timestamp('2023-01-01 02:15:00')]
              },
             {
              'ede': [None, None],
              'dovs': [None, Timestamp('2023-01-01 04:13:00')],
              'conservative': [Timestamp('2023-01-01 03:00:00'), Timestamp('2023-01-01 04:15:00')],
              'winner': [Timestamp('2023-01-01 03:00:00'), Timestamp('2023-01-01 04:13:00')],
              'zero_times': [Timestamp('2023-01-01 03:15:00'), Timestamp('2023-01-01 04:00:00')]
             }
          ],
        ...
    """
    #-------------------------
    return_df = pd.DataFrame()
    #-----
    for (SN_i, PN_i), best_ests_list_i in all_best_ests_dict.items():
        if len(best_ests_list_i)==0:
            continue
        for outg_j, best_est_dict_ij in enumerate(best_ests_list_i):
            best_est_dict_ij_split=dict()
            best_est_dict_ij_split['SN'] = SN_i
            best_est_dict_ij_split['PN'] = PN_i
            best_est_dict_ij_split['i_outg'] = outg_j
            for est_key, beg_end_vals in best_est_dict_ij.items():
                best_est_dict_ij_split[f"{est_key}_min"] = beg_end_vals[0]
                best_est_dict_ij_split[f"{est_key}_max"] = beg_end_vals[1]
            return_df = pd.concat([return_df, pd.DataFrame(best_est_dict_ij_split, index=[return_df.shape[0]])])
    #-------------------------
    return return_df

In [18]:
#TODO: I don't think I need n_SNs_not_in_ede anymore
def calculate_ci_cmi_w_ami_w_ede_help(
    df, 
    ede_df, 
    dovs_outg_t_beg_end, 
    expand_outg_search_time=pd.Timedelta('1 hour'), 
    conservative_estimate=True, 
    est_ede_kwargs=None, 
    audit_selection_method='min', 
    return_CI_SNs=False, 
    use_est_outg_times=False, 
    pct_SNs_required_for_outage_est=0, 
    expand_outg_est_search_time=pd.Timedelta('1 hour'), 
    use_only_overall_endpoints_of_est_outg_times=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
    PN_col='aep_premise_nb', 
    return_all_best_ests=False, 
    return_all_best_ests_type='dict'
):
    r"""
    Given a DF containing AMI data together with outage begin and end times from DOVS, calculate the CI (customers interrupted) 
      and CMI (customer minutes interrupted).
      
    NOTE: The times from DOVS are used simply as time markers around which the algorithm searches for outages events.
          The DOVS times aren't really necessary aside from setting this time search region (defined by both dovs_outg_t_beg_end
            and expand_outg_search_time)
          So, by no means is this a simple calculation using only DOVS.
          Moreover, any beg/end time together with expand_outg_search_time could be used.
          
    NOTE: if ede_df is supplied, it should be for the same outage as that of df.
      
    Return value is tuple containing (CI, CMI)
      UNLESS return_CI_SNs==True, in which case return (CI, CMI, outg_SNs)
    
    NOTE:  If one wants to calculate CI/CMI using PNs (premise numbers) instead of SNs (serial numbers), one must be careful
           to first drop duplicate values (after dropping SNs col), or else the same answer will be found for PNs as SNs
           
    dovs_outg_t_beg_end:
        Should be a list/tuple of length two containing the outage beginning and ending times from DOVS.
        This, together with expand_outg_search_time sets the region around which the algorithm searches for outages.
    
    expand_outg_search_time:
        Sets the region around each outage (either found using estimate_outage_times if use_est_outg_times==True, or taken
          directly from dovs_outg_t_beg_end) to search for individual meter outages (using calculate_mi_for_meter).
        If use_est_outg_times==True, probably want expand_outg_search_time tighter and expand_outg_est_search_time looser.
        If use_est_outg_times==False, probably want expand_outg_search_time looser
          
    conservative_estimate:
        Should likely have this set to True, otherwise one will certainly be underestimating the CI/CMI
        If False, CI/CMI are calculated only using the periods with values of 0.
          As documented elsewhere, it appears that AMI meters only report values of 0 if all readings in the 15-minute period are zero.
          I believe that if at least one value is non-zero, the AMI will report a non-zero value.
              e.g., if power out from 12:00 to 12:14, but turns on from 12:14 to 12:15, the 12:15 reading will be non-zero
          Thus, simply calculating the period of time when the measurements are 0 inherently underestimates the actual outage time.
        If True, a more conservative approach is used, in which the CI/CMI include the period preceding the first 0 value and that
          following the last 0 value.
          If applicable, the endpoint will be taken from DOVS (via dovs_outg_t_beg_end) instead of the full period preceding/following
            as described above.
          Find more information regarding this functionality in the dovs_outg_t_beg_end section of the calculate_mi_for_meter documentation.
        
    return_CI_SNs:
        If True, also return outg_SNs.
        This feature is mainly included for functionality where out_t_beg_end contains multiple outage times.
        
    use_est_outg_times:
        If True, estimate the outage times by looking at the collection of all SNs in df
          i.e., try to determine when outages occur by looking for periods of time where the meters show values of 0.
        If a single outage is actually comprised of smaller suboutages, the suboutage times will be returned and used.
        NOTE: If no estimate outages are found, dovs_outg_t_beg_end will be used (assuming it is not None)
        -----
        pct_SNs_required_for_outage_est:
            The percentage of serial numbers which must show a reading of 0 for a time period to be considered an outage.
              e.g., if pct_SNs_required_for_outage==0, all serial numbers must have a reading of 0
              e.g., if pct_SNs_required_for_outage==25, 25% of the serial numbers must have a reading of zero.
            NOTE: pct_SNs_required_for_outage_est=0 essentially means at least one SN must register a value of 0
        
        expand_outg_est_search_time=pd.Timedelta('1 hour'):
            The interval around dovs_outg_t_beg_end to use when trying to estimate the outage times.
            This should typically be looser than expand_outg_search_time.
            
        use_only_overall_endpoints_of_est_outg_times:
            As stated above, if a single outage is actually comprised of smaller suboutages, the suboutage times 
              will be returned.
            If use_only_overall_endpoints_of_est_outg_times==True, the beginning of the first suboutage and ending
              of the last suboutage will only be used.
            If use_only_overall_endpoints_of_est_outg_times==False, all suboutages will be used fully.
    """
    #-------------------------
    # Probably easier/faster to supply calculate_mi_for_meter with freq, so infer here
    freq = Utilities_df.determine_freq_in_df_col(
        df=df, 
        groupby_SN=True, 
        return_val_if_not_found='error', 
        assert_single_freq=True, 
        assert_expected_freq_found=False, 
        expected_freq=pd.Timedelta('15 minutes'), 
        time_col=t_int_beg_col, 
        SN_col=SN_col
    )
    #-------------------------
    if use_est_outg_times:
        # Find an estimate of the outage time using the AMI data
        # NOTE: Below, conservative_estimate should always be True (not necessarily equal to input parameter)
        est_outg_times = estimate_outage_times(
            df = df, 
            t_search_min_max=[
                dovs_outg_t_beg_end[0]-expand_outg_est_search_time, 
                dovs_outg_t_beg_end[1]+expand_outg_est_search_time
            ], 
            pct_SNs_required_for_outage=pct_SNs_required_for_outage_est, 
            relax_pct_SNs_required_if_no_outgs_found=True, 
            t_int_beg_col=t_int_beg_col, 
            t_int_end_col=t_int_end_col, 
            value_col=value_col, 
            verbose=False,
            outg_rec_nb_col=None
        )
        if len(est_outg_times)==0:
            if dovs_outg_t_beg_end is not None:
                outg_times = [dovs_outg_t_beg_end]
            else:
                outg_times = []
        else:
            if use_only_overall_endpoints_of_est_outg_times:
                outg_times = [[est_outg_times[0]['cnsrvtv_t_beg'], est_outg_times[-1]['cnsrvtv_t_end']]]
            else:
                outg_times = [[x['cnsrvtv_t_beg'], x['cnsrvtv_t_end']] for x in est_outg_times]
    else:
        assert(dovs_outg_t_beg_end is not None)
        outg_times = [dovs_outg_t_beg_end]
    #-------------------------
    # Get the search min/max times from the est. outage times and expand search time
    t_search_min_max_coll = get_nonoverlapping_search_windows(
        est_outg_times=outg_times, 
        expand_outg_search_time=expand_outg_search_time
    )
    assert(len(t_search_min_max_coll)==len(outg_times))
    #-----
    CMI=0
    outg_SNs=[]
    all_best_ests_dict = dict()
    for t_search_min_max_i in t_search_min_max_coll:
        df_outg_i=df 
        # Get the SNs and PNs for outg_i
        SNs_PNs_i = df_outg_i[[SN_col, PN_col]].value_counts().index.tolist()
        #-------------------------
        for SN_ij, PN_ij in SNs_PNs_i:
            df_outg_i_SN_j = df_outg_i[df_outg_i[SN_col]==SN_ij]
            assert(df_outg_i_SN_j[PN_col].nunique()==1)
            #-----
            if ede_df is not None:
                if SN_ij in ede_df[SN_col].tolist():
                    ede_df_outg_i_SN_j = ede_df[ede_df[SN_col]==SN_ij]
                else:
                    ede_df_outg_i_SN_j=None
            else:
                ede_df_outg_i_SN_j=None
            #-----
            mi_ij = calculate_mi_for_meter_ami_w_ede_help(
                df_i=df_outg_i_SN_j, 
                ede_df_i=ede_df_outg_i_SN_j, 
                t_search_min_max=t_search_min_max_i, 
                conservative_estimate=conservative_estimate, 
                dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
                freq=freq, 
                est_ede_kwargs=est_ede_kwargs, 
                audit_selection_method=audit_selection_method, 
                t_int_beg_col=t_int_beg_col, 
                t_int_end_col=t_int_end_col, 
                value_col=value_col, 
                SN_col=SN_col, 
                return_all_best_ests=return_all_best_ests
            )
            #-------------------------
            if return_all_best_ests:
                assert(isinstance(mi_ij, tuple) and len(mi_ij)==2)
                # NOTE: mi_ij must be grabbed from tuple second!
                all_best_ests_ij = mi_ij[1]
                mi_ij = mi_ij[0]
                #-----
                if (SN_ij, PN_ij) in all_best_ests_dict.keys():
                    all_best_ests_dict[(SN_ij, PN_ij)].extend(all_best_ests_ij)
                else:
                    all_best_ests_dict[(SN_ij, PN_ij)] = all_best_ests_ij
            #-------------------------                
            CMI += mi_ij
            if mi_ij>0:
                outg_SNs.append(SN_ij) 
    #-------------------------
    # If there are multiple sub-outage periods, likely repeat values in outg_SNs
    # So need to perform set operation
    outg_SNs = list(set(outg_SNs))
    CI = len(outg_SNs)
    #-------------------------
    if not return_CI_SNs and not return_all_best_ests:
        return CI, CMI
    else:
        return_dict = dict(
            CI=CI, 
            CMI=CMI
        )
        if return_CI_SNs:
            return_dict['CI_SNs'] = outg_SNs
        if return_all_best_ests:
            assert(return_all_best_ests_type in ['dict', 'pd.DataFrame'])
            if return_all_best_ests_type=='dict':
                return_dict['all_best_ests'] = all_best_ests_dict
            else:
                return_dict['all_best_ests'] = convert_all_best_ests_dict_to_df(all_best_ests_dict)
        #-----
        return return_dict

In [19]:
def add_est_outg_times_to_axis(
    ax, 
    est_outg_times, 
    t_beg_kwargs=None,
    t_end_kwargs=None,
    include='both', 
    include_outage_limits_text=True
):
    r"""
    At vertical lines showing estimates outage times.
    
    Note: out_t_beg and out_t_end aren't really necessary, they simply direct which side of
          the estimated lines to print the text
    """
    #-------------------------
    assert(include in ['both', 'beg', 'end'])
    #-------------------------
    if t_beg_kwargs is None:
        t_beg_kwargs={}
    if t_end_kwargs is None:
        t_end_kwargs={}
    #-----
    dflt_t_beg_kwargs=dict(color='red', linestyle='--')
    dflt_t_end_kwargs=dict(color='green', linestyle='--')
    #-----
    t_beg_kwargs = Utilities.supplement_dict_with_default_values(to_supplmnt_dict=t_beg_kwargs, default_values_dict=dflt_t_beg_kwargs, inplace=False)
    t_end_kwargs = Utilities.supplement_dict_with_default_values(to_supplmnt_dict=t_end_kwargs, default_values_dict=dflt_t_end_kwargs, inplace=False)
    #-------------------------
    for dt_off_i, dt_on_i in est_outg_times:
        if include=='both' or include=='beg':
            ax.axvline(dt_off_i, **t_beg_kwargs)
        if include=='both' or include=='end':
            ax.axvline(dt_on_i, **t_end_kwargs)
        #-----
        if include_outage_limits_text:
            ax.text(dt_off_i, ax.get_ylim()[0], 'Est. outg. beg', 
                    rotation=90, verticalalignment='bottom', 
                    horizontalalignment='right' if dt_off_i<out_t_beg else 'left')
            ax.text(dt_on_i, ax.get_ylim()[0], 'Est. outg. end', 
                    rotation=90, verticalalignment='bottom', 
                    horizontalalignment='left' if dt_on_i>out_t_end else 'right')
    #-------------------------

In [20]:
def add_best_est_to_axis(
    ax, 
    est_val_beg,
    est_val_end,
    line_kwargs, 
    expand_ax_to_accommodate=True
):
    r"""
    Mainly a helper function for add_all_best_ests_to_axis, but can certainly be used on its own
    
    NOTE:
    """
    #-------------------------
    # Set color_beg equal to line_kwargs['color_beg'] if exists (and remove), else set equal to line_kwargs['color']
    # Similar for color_end
    color_beg = line_kwargs.pop('color_beg', line_kwargs.get('color', 'black'))
    color_end = line_kwargs.pop('color_end', line_kwargs.get('color', 'black'))
    #-----
    # Same idea for linestyles as used above for colors
    linestyle_beg = line_kwargs.pop('linestyle_beg', line_kwargs.get('linestyle', '-'))
    linestyle_end = line_kwargs.pop('linestyle_end', line_kwargs.get('linestyle', '-'))

    # Need to strip off 'color' since explicitly setting via beg(end)_color
    line_kwargs.pop('color', None)
    line_kwargs.pop('linestyle', None)
    # Note: Below, pd.isna() catches values of None, NaN, and NaT
    if not pd.isna(est_val_beg):
        if (
            not expand_ax_to_accommodate and 
            (mpl.dates.date2num(est_val_beg)<ax.get_xlim()[0] or mpl.dates.date2num(est_val_beg)>ax.get_xlim()[1])
        ):
            pass
        else:
            ax.axvline(est_val_beg, color=color_beg, linestyle=linestyle_beg, **line_kwargs)
    if not pd.isna(est_val_end):
        if (
            not expand_ax_to_accommodate and 
            (mpl.dates.date2num(est_val_end)<ax.get_xlim()[0] or mpl.dates.date2num(est_val_end)>ax.get_xlim()[1])
        ):
            pass
        else:
            ax.axvline(est_val_end, color=color_end, linestyle=linestyle_end, **line_kwargs)

def add_all_best_ests_to_axis(
    ax, 
    all_best_ests, 
    line_kwargs_by_est_key=None, 
    keys_to_include=['winner', 'conservative', 'zero_times'], 
    expand_ax_to_accommodate=True
):
    r"""
    At vertical lines showing estimates outage times.
    NOTE: This will also work with means_df from get_mean_times_w_dbscan
    -----
    all_best_ests:
        This should either be a list or a pd.DataFrame object containing estimated values.
        If a list, each element should be a dictionary with the keys_to_include/expctd_keys (defined in code below).
        If a pd.DataFrame, the columns should contain f'{x}_min', f'{x}_max' for x in keys_to_include/expctd_keys
        -----
        At the time of writing, one could use the results from calculate_ci_cmi_w_ami_w_ede_help in the following ways:
        If return_all_best_ests_type=='dict':
            all_best_ests = list(itertools.chain.from_iterable(list(results['all_best_ests'].values())))
        If return_all_best_ests_type=='pd.DataFrame':
            all_best_ests = results['all_best_ests']
    
    keys_to_include:
        See expctd_keys in code below for list of acceptable keys
        Set equal to 'all' to include all keys.
        Otherwise, set equal to a single key or list of keys
    """
    #-------------------------
    if all_best_ests is None:
        return
    #-------------------------
    assert(Utilities.is_object_one_of_types(all_best_ests, [list, tuple, pd.DataFrame]))
    #-------------------------
    expctd_keys = ['conservative', 'zero_times', 'winner', 'ede', 'dovs']
    #-------------------------
    dflt_line_kwargs = dict(
        conservative=dict(color_beg='red', color_end='green', linestyle=':'),
        zero_times=dict(color_beg='red', color_end='green', linestyle=':'),
#         winner=dict(color='goldenrod', linestyle='--'),
        winner=dict(color_beg='red', color_end='green', linestyle='--'),
        ede=dict(color_beg='red', color_end='green', linestyle='-.'),
        dovs=dict(color_beg='red', color_end='green', linestyle='-'),

    )
    #-------------------------
    if keys_to_include=='all':
        keys_to_include=expctd_keys
    assert(Utilities.is_object_one_of_types(keys_to_include, [str, list, tuple]))
    if isinstance(keys_to_include, str):
        keys_to_include = [keys_to_include]
    assert(len(set(keys_to_include).difference(set(expctd_keys)))==0)
    #-------------------------
    if line_kwargs_by_est_key is None:
        line_kwargs_by_est_key=dflt_line_kwargs
    else:
        line_kwargs_by_est_key = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=line_kwargs_by_est_key, 
            default_values_dict=dflt_line_kwargs, 
            extend_any_lists=True, 
            inplace=False
        )
    #-------------------------
    if Utilities.is_object_one_of_types(all_best_ests, [list, tuple]):
        for all_best_ests_i in all_best_ests:
            for k,v in all_best_ests_i.items():
                assert(k in expctd_keys)
                if k not in keys_to_include:
                    continue
                beg_ik, end_ik = v[0], v[1]
                line_kwargs_k = copy.deepcopy(line_kwargs_by_est_key[k])
                #-----
                add_best_est_to_axis(
                    ax=ax, 
                    est_val_beg=beg_ik,
                    est_val_end=end_ik,
                    line_kwargs=line_kwargs_k, 
                    expand_ax_to_accommodate=expand_ax_to_accommodate
                )
    else:
        assert(isinstance(all_best_ests, pd.DataFrame))
        #-----
        cols_to_include = [[f'{x}_min', f'{x}_max'] for x in keys_to_include]
        cols_to_include = list(itertools.chain.from_iterable(cols_to_include))
        assert(len(set(cols_to_include).difference(all_best_ests.columns.tolist()))==0)
        #-----
        for idx_i, row_i in all_best_ests.iterrows():
            for k in keys_to_include:
                beg_ik, end_ik = row_i[[f"{k}_min", f"{k}_max"]]
                line_kwargs_k = copy.deepcopy(line_kwargs_by_est_key[k])
                #-----
                add_best_est_to_axis(
                    ax=ax, 
                    est_val_beg=beg_ik,
                    est_val_end=end_ik,
                    line_kwargs=line_kwargs_k, 
                    expand_ax_to_accommodate=expand_ax_to_accommodate
                )
                
                
# SHOULD NOT BE NEEDED ANYMORE BECAUSE add_all_best_ests_to_axis can be used instead!                
# def add_mean_best_ests_to_axis(
#     ax, 
#     means_df, 
#     line_kwargs_by_est_key=None, 
#     keys_to_include=['winner', 'conservative', 'zero_times'], 
#     expand_ax_to_accommodate=True
# ):
#     r"""
#     At vertical lines showing estimates outage times.
#     Note: all_best_ests_list will contain more than one element if multiple suboutages
#     Each element in all_best_ests_list should be a dictionary with the expctd_keys
    
#     keys_to_include:
#         See expctd_keys in code below for list of acceptable keys
#         Set equal to 'all' to include all keys.
#         Otherwise, set equal to a single key or list of keys
#     """
#     #-------------------------
#     expctd_keys = ['conservative', 'zero_times', 'winner', 'ede', 'dovs']
#     #-------------------------
#     dflt_line_kwargs = dict(
#         conservative=dict(color_beg='red', color_end='green', linestyle=':'),
#         zero_times=dict(color_beg='red', color_end='green', linestyle=':'),
# #         winner=dict(color='goldenrod', linestyle='--'),
#         winner=dict(color_beg='red', color_end='green', linestyle='--'),
#         ede=dict(color_beg='red', color_end='green', linestyle='-.'),
#         dovs=dict(color_beg='red', color_end='green', linestyle='-'),

#     )
#     #-------------------------
#     if keys_to_include=='all':
#         keys_to_include=expctd_keys
#     assert(Utilities.is_object_one_of_types(keys_to_include, [str, list, tuple]))
#     if isinstance(keys_to_include, str):
#         keys_to_include = [keys_to_include]
#     assert(len(set(keys_to_include).difference(set(expctd_keys)))==0)
#     #-------------------------
#     if line_kwargs_by_est_key is None:
#         line_kwargs_by_est_key=dflt_line_kwargs
#     else:
#         line_kwargs_by_est_key = Utilities.supplement_dict_with_default_values(
#             to_supplmnt_dict=line_kwargs_by_est_key, 
#             default_values_dict=dflt_line_kwargs, 
#             extend_any_lists=True, 
#             inplace=False
#         )
#     #-------------------------
#     for key in keys_to_include:
#         assert(f"{key}_min" in means_df.columns.tolist())
#         assert(f"{key}_max" in means_df.columns.tolist())    
#     #-------------------------
#     for idx, row in means_df.iterrows():
#     #     if str(idx).startswith('Unclustered'):
#     #         # Unclustered specific code:
#         for key in keys_to_include:
#             beg_ik, end_ik = row[[f"{key}_min", f"{key}_max"]]
#             line_kwargs_k = copy.deepcopy(line_kwargs_by_est_key[key])
#             #-----
#             add_best_est_to_axis(
#                 ax=ax, 
#                 est_val_beg=beg_ik,
#                 est_val_end=end_ik,
#                 line_kwargs=line_kwargs_k, 
#                 expand_ax_to_accommodate=expand_ax_to_accommodate
#             ) 

In [21]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics

def get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=False
):
    r"""
    """
    #-------------------------
    # For this procedure to work, they cannot be and NaNs in ests_to_include_in_clustering.
    # Therefore, it may be best to use winner, as there will always be winner values
    if best_ests_df.dropna(subset=ests_to_include_in_clustering).shape[0]==0:
        print("All rows contain at least one NaN in ests_to_include_in_clustering!")
        print("Resetting ests_to_include_in_clustering to only includer winners")
        ests_to_include_in_clustering=['winner_min', 'winner_max']
    #-----
    best_ests_df = best_ests_df.dropna(subset=ests_to_include_in_clustering)
    #-------------------------
    # Although it is apparently possible to feed dbscan the raw Datetime objects, it appears that they are essentially
    #   converted to timestamps, meaning they are converted to numbers on the order of 1.6e9, which makes them somewhat
    #   difficult to deal with (e.g., in setting the eps parameter for dbscan)
    # Therefore, for my purposes it is preferable to convert these to smaller numbers, such as the difference between the row's
    #   datetime and the approximate beginning of the outage (NOTE, the comparison value doesn't matter too much, we just want
    #   something in the ballpark so the differences are not too big).
    # I will express these differences in terms of minutes, so the eps value should be set in terms of minutes
    comp_val = best_ests_df[ests_to_include_in_clustering[0]].min()
    #-----
    db_cols = []
    for clust_col in ests_to_include_in_clustering:
        best_ests_df[f'{clust_col}_db'] = (best_ests_df[clust_col]-comp_val).dt.total_seconds()/60
        db_cols.append(f'{clust_col}_db')
    #-------------------------    
    db = DBSCAN(eps=eps_min, min_samples=min_samples).fit(best_ests_df[db_cols])
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    #-------------------------
    # Set the label values in best_ests_df
    best_ests_df['db_label'] = labels
    #-------------------------
    means_df = best_ests_df.groupby(['db_label'])[ests_to_include_in_output].mean()
    if n_noise_>0:
        # Unclustered data will be handled separately, and not aggregated.  So remove from means_df
        means_df=means_df.drop(index=[-1])
        #-----
        unclstrd_df = best_ests_df[best_ests_df['db_label']==-1][ests_to_include_in_output].copy()
        unclstrd_df['db_label'] = [f"Unclustered {i}" for i in range(unclstrd_df.shape[0])]
        unclstrd_df = unclstrd_df.set_index('db_label')
        #-----
        # Want to set the correct db_label value for these in best_ests_df in case return_labelled_best_ests_df
        best_ests_df.loc[best_ests_df['db_label']==-1, 'db_label'] = [f"Unclustered {i}" for i in range(best_ests_df[best_ests_df['db_label']==-1].shape[0])]
        #-----
        means_df = pd.concat([means_df, unclstrd_df])
    #-------------------------
    if return_labelled_best_ests_df:
        return means_df, best_ests_df.drop(columns=db_cols)
    else:
        return means_df

In [22]:
def add_nPNs_to_means_df(
    means_df, 
    best_ests_df_w_db_lbl,
    db_label_col='db_label', 
    PN_col='PN', 
    n_PNs_col = 'n_PNs'
):
    r"""
    Add the number of premise numbers per db_label group to means_df
    NOTE: The index of means_df should be the db_labels (db_label_col is to locate the labels
            within best_ests_df_w_db_lbl)
    """
    #-------------------------
    n_PNs_dict = {}
    for db_label_i, row_i in means_df.iterrows():
        n_PNs_i = best_ests_df_w_db_lbl[best_ests_df_w_db_lbl[db_label_col]==db_label_i][PN_col].nunique()
        n_PNs_dict[db_label_i] = n_PNs_i
    #-----
    n_PNs_srs = pd.Series(n_PNs_dict)
    n_PNs_srs.name = n_PNs_col
    #-------------------------
    means_df = pd.merge(means_df, n_PNs_srs, left_index=True, right_index=True, how='inner')
    return means_df

In [23]:
def combine_PNs_in_best_ests_df_i(
    best_ests_df_for_PN_i, 
    min_cols, 
    max_cols, 
    return_col_order, 
    likeness_thresh = pd.Timedelta('1 minutes'), 
    SN_col = 'SN', 
    PN_col = 'PN', 
    i_outg_col = 'i_outg' 
):
    r"""
    !!!!! THIS IS A HELPER FUNCTION FOR combine_PNs_in_best_ests_df. !!!!!
    !!!!! IT IS NOT RECOMMENDED TO USE THIS FUNCTION OUTSIDE OF combine_PNs_in_best_ests_df !!!!!
    """
    #-------------------------
    if best_ests_df_for_PN_i.shape[0]==1:
        return best_ests_df_for_PN_i[return_col_order].squeeze()
    #-------------------------
    # best_ests_df_for_PN_i should come from best_ests_df being grouped by PN_col and i_outg_col
    # Therefore, there should be a single unique value for these!
    #   NOTE: Could also do nunique for each column sparately instead of using value_counts
    assert(best_ests_df_for_PN_i[[PN_col, i_outg_col]].value_counts().shape[0]==1)
    #-------------------------
    ranges = best_ests_df_for_PN_i[min_cols+max_cols].apply(lambda x: x.dropna().max()-x.dropna().min())
    #-----
    # NOTE: NaN values are possible if all values are None for given column
    if (ranges>likeness_thresh).any():
        print('Outage estimates for serial numbers in given premise are not similar!')
        print('CRASH IMMINENT')
        print(f"PN = {best_ests_df_for_PN_i[PN_col].unique()[0]}")
        print(f"threshold = {likeness_thresh}")
        print(f"Violators:\n{ranges[ranges>likeness_thresh]}")
    assert(((ranges.isna()) | (ranges<=likeness_thresh)).all())
    #-------------------------
    return_srs = best_ests_df_for_PN_i[min_cols+max_cols].agg({x:'min' for x in min_cols}|{x:'max' for x in max_cols})
    return_srs = return_srs[return_col_order]
    #-------------------------
    return return_srs


def combine_PNs_in_best_ests_df(
    best_ests_df, 
    likeness_thresh = pd.Timedelta('1 minutes'), 
    SN_col = 'SN', 
    PN_col = 'PN', 
    i_outg_col = 'i_outg'     
):
    r"""
    Combine all serial numbers (SNs) for each premise number (PN) in best_ests_df.
    The purpose is so that CI/CMI may be calculated at the premise level, instead of meter level.
    When there are multiple meters for a given premise:
        1. If there are multiple sub-outages, it is checked to ensure each sub-outage contains the same
             set of serial numbers, see EXPLANATION OF STACKED GROUPBY OPERATIONS below for more info.
        2. For each sub-outage, it is ensured that the time estimates from all meters is roughly equal,
             as defined by the likeness_thresh parameter.
           e.g., winner_min should be roughly the same for all meters on a premise, as should winner_max, etc.
        3. For each premise and sub-outage:
               for all minimum time estimates, the minimum value amongst the meters on a premise is kept.
               for all maximum time estimates, the maximum value amongst the meters on a premise is kept.
           This maintains our strategy of reporting the most conservative value for the outage time, although,
             due to the tightness of likeness_thresh, this doesn't make a huge difference.
    
    best_ests_df:
        Should contain SN_col, PN_col, i_outg_col and various min/max time estimate columns.
        In a typical situation, best_ests_df will have the following columns:
            ['SN', 'PN', 'i_outg',
             'ede_min', 'ede_max',
             'dovs_min', 'dovs_max',
             'conservative_min', 'conservative_max',
             'zero_times_min', 'zero_times_max',
             'winner_min', 'winner_max']
        NOTE: minimum columns must end with '_min' and maxima with '_max'
              Also, best_ests_df should only contain 'SN', 'PN', 'i_outg' and min/max columns
    
    likeness_thresh:
        Sets the maximum allowed difference between time estimates for meters of a given premise.
        All SNs on a given PN should lose and regain power at the same time, although slight differences exist (typically
          only a few seconds difference, and likely due to delays in meters registering events)
          
    --------------------------------------------------
    EXPLANATION OF STACKED GROUPBY OPERATIONS
    --------------------------------------------------
    The purpose of the multiple groupby operations is to ensure that, when multiple sub-outages exist for a premise (i.e., 
      when i_outg=0, 1, ...), all sub-outages share the same set of meters.
    One would not expect, e.g., the first sub-outage to only affect one meter on a given premise and the second sub-outage
      to affect multiple meters on the premise.
    The full chain of groupby commands I am referencing is:
    
        (best_ests_df
         .groupby([PN_col, i_outg_col])[SN_col].apply(list)
         .groupby([PN_col]).apply(list)
         .apply(lambda x: Utilities.are_all_lists_eq(x))
        )  
        
    Suppose best_ests_df has the following form:
    
              SN    PN  i_outg
        0  SN_11  PN_1       0
        1  SN_11  PN_1       1
        2  SN_12  PN_1       0
        3  SN_12  PN_1       1
        4  SN_21  PN_2       0
        5  SN_21  PN_2       1
        6  SN_22  PN_2       0
        7  SN_22  PN_2       1
        
    In this example, there are two premises (PN_1 and PN_2), each having two meters (SN_11, SN_12 associated with PN_1 
      and SN_21, SN_22 associated with PN_2), and each having an outage split into two sub-outages (i_outg 0 and 1).
    The output for the steps of the chain of groupby commands is as follows:
    -------------------------
    1. Get a list of the meters (SNs) associated with each premise and sub-outage
        ----------
        (best_ests_df
         .groupby([PN_col, i_outg_col])[SN_col].apply(list)
        )
        ----------
        PN    i_outg
        PN_1  0         [SN_11, SN_12]
              1         [SN_11, SN_12]
        PN_2  0         [SN_21, SN_22]
              1         [SN_21, SN_22]        
    -------------------------
    2. Combine these into a list of lists for each premise
        ----------
        (best_ests_df
         .groupby([PN_col, i_outg_col])[SN_col].apply(list)
         .groupby([PN_col]).apply(list)
        )
        ----------
        PN
        PN_1    [[SN_11, SN_12], [SN_11, SN_12]]
        PN_2    [[SN_21, SN_22], [SN_21, SN_22]]        
    -------------------------
    3. Use Utilities.are_all_lists_eq function to ensure the collection of meters for each premise is consistent
         for all sub-outages
        ----------
        (best_ests_df
         .groupby([PN_col, i_outg_col])[SN_col].apply(list)
         .groupby([PN_col]).apply(list)
         .apply(lambda x: Utilities.are_all_lists_eq(x))
        )
        ----------
        PN
        PN_1    True
        PN_2    True
    """
    #-------------------------
    # Make sure SN_col, PN_col, and i_outg_col all in best_ests_df
    assert(len(set([SN_col, PN_col, i_outg_col]).difference(best_ests_df.columns.tolist()))==0)
    
    # Determine which cols are mins and which are maxs
    # NOTE: The method used for finding minmax_cols together with the assertion below assures that 
    #       best_ests_df contains only SN_col, PN_col, i_outg_col and min/max cols
    minmax_cols = [x for x in best_ests_df.columns.tolist() if x not in [SN_col, PN_col, i_outg_col]]
    min_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_min')]
    max_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_max')]
    assert(len(set(minmax_cols).symmetric_difference(set(min_cols+max_cols)))==0)
    assert(len(min_cols)==len(max_cols))
    
    # Maintain the original order of columns in best_ests_df in the returned df
    return_col_order = [x for x in best_ests_df.columns.tolist() if x in min_cols+max_cols]
    
    #-------------------------
    # All columns in minmax_cols must be datetime type
    # Below ensures all min/max columns are datetime
    for col_i in min_cols+max_cols:
        dtype_i = best_ests_df[col_i].dtype
        if not (dtype_i is datetime.datetime or is_datetime64_dtype(dtype_i)):
            best_ests_df = Utilities_df.convert_col_type_w_pd_to_datetime(
                df=best_ests_df, 
                column=col_i, 
                inplace=True
            )
    
    #-------------------------
    # Really, the reduction operations below only need to be performed on PNs with more than one SN
    # To save resources and time, split apart those needing reduced and those not
    SNs_per_PN = best_ests_df[[SN_col, PN_col]].drop_duplicates()[PN_col].value_counts()
    PNs_w_mult_SNs = SNs_per_PN[SNs_per_PN>1].index.tolist()
    
    # If there are no PNs with multiple SNs, return best_ests_df (without SN_col, as the purpose of this
    #   is to eliminate SN_col by grouping by premise)
    if len(PNs_w_mult_SNs)==0:
        return best_ests_df.drop(columns=[SN_col]).sort_values(by=[PN_col, i_outg_col], ignore_index=True)
    
    best_ests_df_w_mult  = best_ests_df[best_ests_df[PN_col].isin(PNs_w_mult_SNs)]
    best_ests_df_wo_mult = best_ests_df[~best_ests_df[PN_col].isin(PNs_w_mult_SNs)]
    assert(best_ests_df_w_mult.shape[0]+best_ests_df_wo_mult.shape[0]==best_ests_df.shape[0])
    
    #-------------------------
    # Ensure that, when multiple sub-outages exist for a premise (i.e., when i_outg=0, 1, ...), 
    #   all sub-outages share the same set of meters.
    # See function documentation above for explanation of operations
    assert(
        (best_ests_df_w_mult
         .groupby([PN_col, i_outg_col])[SN_col].apply(list)
         .groupby([PN_col]).apply(list)
         .apply(lambda x: Utilities.are_all_lists_eq(x))
        ).all()
    )
    
    #-------------------------
    # Perform reduction
    #   For all minimum time estimates, the minimum value amongst the meters on a premise is kept.
    #   For all maximum time estimates, the maximum value amongst the meters on a premise is kept.
    return_df = best_ests_df_w_mult.groupby([PN_col, i_outg_col], as_index=False, group_keys=False).apply(
        lambda x: combine_PNs_in_best_ests_df_i(
            best_ests_df_for_PN_i=x, 
            min_cols=min_cols, 
            max_cols=max_cols, 
            return_col_order=return_col_order, 
            likeness_thresh=likeness_thresh, 
            SN_col=SN_col, 
            PN_col=PN_col, 
            i_outg_col=i_outg_col
        )
    )
    
    #-------------------------
    # Combine together with collection of PNs having a single SN
    # NOTE: Since we are grouping by PN_col, i_outg_col and aggregating the min/max cols,
    #       the SN_col should not be present in the final output
    best_ests_df_wo_mult = best_ests_df_wo_mult.drop(columns=[SN_col])
    assert(return_df.columns.tolist()==best_ests_df_wo_mult.columns.tolist())
    return_df = pd.concat([return_df, best_ests_df_wo_mult])
    
    #-------------------------
    # Sometimes it seems the operation can change the type of data in min/max cols
    #   e.g., I have seen epoch times returned, instead of datetime object
    #     (e.g., 1673095676000000000, which is actually 2023-01-07 12:47:56)
    # This was occurred in the older version, prior to splitting best_ests_df_w_mult and 
    #   best_ests_df_wo_mult, but there's no harm in keeping this check in place
    # Below ensures all min/max columns are datetime
    for col_i in min_cols+max_cols:
        dtype_i = return_df[col_i].dtype
        if not (dtype_i is datetime.datetime or is_datetime64_dtype(dtype_i)):
            return_df = Utilities_df.convert_col_type_w_pd_to_datetime(
                df=return_df, 
                column=col_i, 
                inplace=True
            )
    
    #-------------------------
    return_df = return_df.sort_values(by=[PN_col, i_outg_col], ignore_index=True)
    
    #-------------------------
    return return_df

In [24]:
def alter_best_ests_df_using_dovs_outg_t_beg(
    best_ests_df,
    dovs_df, 
    outg_rec_nb, 
    outg_rec_nb_idfr='index', 
    dt_off_ts_full_col='DT_OFF_TS_FULL', 
    winner_min_col='winner_min', 
    winner_max_col='winner_max', 
    i_outg_col='i_outg'
):
    r"""
    Alter best_ests_df to use the outage beginning time as given by DOVS and the ending times as estimated using AMI.
    To summarize:
        Take dovs_outg_t_beg to be the outage starting time as given by DOVS.
        Any sub-outages which end before dovs_outg_t_beg will be completely removed.
        Any sub-outages which begin after dovs_outg_t_beg will be left alone.
        Sub-outages with beginning times before dovs_outg_t_beg and ending times after dovs_outg_t_beg will
          have their beginning times altered to equal dovs_outg_t_beg
    """
    #-------------------------
    # First, get the needed info from DOVS
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df=dovs_df, 
        outg_rec_nb=outg_rec_nb, 
        outg_rec_nb_idfr=outg_rec_nb_idfr, 
        assert_outg_rec_nb_found=True
    )
    assert(dovs_df_i.shape[0]==1)
    # Get the outage time from DOVS
    dovs_outg_t_beg = dovs_df_i.iloc[0][dt_off_ts_full_col]
    #-------------------------
    # Since we are using the DOVS beginning time, we only consider sub-outages that end after this time
    #   Put differently, get rid of any sub-outages which end before the posted DOVS beginning time
    return_df = best_ests_df[best_ests_df[winner_max_col]>dovs_outg_t_beg].copy()
    #-------------------------
    # Make sure return_df is sorted properly, and reset i_outg
    return_df=return_df.sort_values(by=[winner_min_col, winner_max_col], ignore_index=True)
    return_df[i_outg_col]=return_df.index
    #-------------------------
    # Any sub-outages which begin after dovs_outg_t_beg should be left alone
    #   Only those with starting times before dovs_outg_t_beg should be altered
    return_df.loc[return_df[winner_min_col]<dovs_outg_t_beg, winner_min_col] = dovs_outg_t_beg
    #-------------------------
    return return_df

In [25]:
# TODO: Do I need to update this function to use t_int_beg_col, t_int_end_col instead of time_col?
def group_ami_df_by_PN(
    ami_df, 
    SN_col='serialnumber', 
    PN_col='aep_premise_nb', 
    time_col='starttimeperiod_local', 
    value_col='value', 
    include_index_in_shared_cols=True, 
    gpby_dropna=True
):
    r"""
    Relatively simple-minded method to group ami_df by premise number (i.e., instead of the possibility of
      multiple entries for a premise at a given datetime corresponding the multiple meters on the premise, the
      DF will be collapsed down to a single value for the premise).
    The DF will be grouped by PN_col and time_col.
    All columns outside of SN_col, PN_col, time_col, and value_col must have a single value per group (i.e., per
      unique combination of PN_col and time_col).
    The value_col will be averaged.
    The SN_col will be dropped
    
    include_index_in_shared_cols:
        As various elements will be grouped/rows collapsed/however you want to think about it, the index will
          necessarily be lost, 
        In many cases (e.g., when the indices are time stamps) the indices for those being combined should be 
          shared (but, pandas doesn't know that).
        If this is the case, set include_index_in_shared_cols==True
        NOTE: For the foreseeable applications, this only really makes sense if the indices are datetime objects,
              so this will be enforced.
    """
    #-------------------------
    # Make sure all of the needed columns are present
    assert(len(set([SN_col, PN_col, time_col, value_col]).difference(set(ami_df.columns.tolist())))==0)
    
    #-------------------------
    grp_by_cols = [PN_col, time_col]
    cols_shared_by_groups = [x for x in ami_df.columns.tolist() 
                             if x not in grp_by_cols+[SN_col, value_col]]
    return_col_order = [x for x in ami_df.columns.tolist() if x != SN_col]
    
    #-------------------------
    # Really, the aggregation operation only needs to be performed on PNs with more than one SN
    # To save resources and time, split apart those needing reduced and those not
    #-----
    # To be most correct, for this application one should include time_col, so this is slightly different to what is 
    #   done in combine_PNs_in_best_ests_df
    SNs_per_PN_and_time = ami_df[[SN_col, PN_col, time_col]].drop_duplicates()[[PN_col, time_col]].value_counts()
    PNs_w_mult_SNs = SNs_per_PN_and_time[SNs_per_PN_and_time>1].index.get_level_values(0).unique().tolist()
    
    # If there are no PNs with multiple SNs, return ami_df (without SN_col, as the purpose of this
    #   is to eliminate SN_col by grouping by premise)
    if len(PNs_w_mult_SNs)==0:
        return ami_df.drop(columns=[SN_col]).sort_values(by=[time_col, PN_col])

    ami_df_w_mult  = ami_df[ami_df[PN_col].isin(PNs_w_mult_SNs)]
    ami_df_wo_mult = ami_df[~ami_df[PN_col].isin(PNs_w_mult_SNs)]
    assert(ami_df_w_mult.shape[0]+ami_df_wo_mult.shape[0]==ami_df.shape[0])
    
    #-------------------------
    if include_index_in_shared_cols:
        # As various elements will be grouped/rows collapsed/however you want to think about it, the index will
        #   necessarily be lost, even though the indices for those being combined should be shared (but, pandas
        #   doesn't know that).
        # In order to retain the index, stored it in a temporary column and re-set it later
        # NOTE: This assumes there is a single index level.  If MultiIndex, method will need re-worked
        assert(ami_df.index.nlevels==1)
        assert(is_datetime64_dtype(ami_df.index.dtype))
        if ami_df.index.name is not None and ami_df.index.name not in ami_df.columns.tolist():
            tmp_idx_col = ami_df.index.name
        else:
            tmp_idx_col = Utilities.generate_random_string()
        ami_df_w_mult = ami_df_w_mult.reset_index(drop=False, names=tmp_idx_col)
        cols_shared_by_groups.append(tmp_idx_col)
    
    #-------------------------
    # Make sure all columns in cols_shared_by_groups have a single value per group
    assert((ami_df_w_mult.groupby(grp_by_cols, dropna=gpby_dropna)[cols_shared_by_groups].nunique()<=1).all().all())
    
    #-------------------------
    # Perform aggregation
    return_df = ami_df_w_mult.groupby(
        grp_by_cols, dropna=gpby_dropna, as_index=False, group_keys=False
    ).agg(
        {x:'first' for x in cols_shared_by_groups}|
        {value_col:'mean'}
    )
    
    #-------------------------
    if include_index_in_shared_cols:
        # Set index back to original values
        return_df = return_df.set_index(tmp_idx_col, drop=True)
    
    #-------------------------
    # Make sure columns are as expected
    assert(len(set(return_df.columns.tolist()).symmetric_difference(set(return_col_order)))==0)
    return_df = return_df[return_col_order]
    
    #-------------------------
    # Combine together with collection of PNs having a single SN
    # NOTE: The SN_col should not be present in the final output
    ami_df_wo_mult = ami_df_wo_mult.drop(columns=[SN_col])
    assert(return_df.columns.tolist()==ami_df_wo_mult.columns.tolist())
    return_df = pd.concat([return_df, ami_df_wo_mult])
    
    #-------------------------
    return_df = return_df.sort_values(by=[time_col, PN_col])
    return return_df

In [26]:
def check_found_ami_for_SN(
    df_i, 
    ts_req, 
    time_col='starttimeperiod_local', 
    SN_col='serialnumber', 
):
    r"""
    Intended for use in check_found_ami_for_all_SNs_in_outage.
    df_i must be for a single SN
    Just checks that all required timestamps (ts_req) are found in df_i[time_col]
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    if len(set(ts_req).difference(set(df_i[time_col].tolist())))==0:
        return True
    else:
        return False

In [27]:
def check_found_ami_for_all_SNs_in_outage(
    df, 
    t_search_min_max, 
    requirement='all', 
    time_col='starttimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
):
    r"""
    """
    #-------------------------
    assert(requirement in ['all', 'endpoints'])
    #-------------------------
    # Need frequency for rounding and generating required timestamps (if requirement=='all')
    freq = Utilities_df.determine_freq_in_df_col(
        df=df, 
        groupby_SN=True, 
        return_val_if_not_found='error', 
        assert_single_freq=True, 
        assert_expected_freq_found=False, 
        expected_freq=pd.Timedelta('15 minutes'), 
        time_col=time_col, 
        SN_col=SN_col
    )
    #-------------------------
    assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]) and len(t_search_min_max)==2)
    # t_search_min_max must be datetime object
    if not isinstance(t_search_min_max[0], datetime.datetime) or not isinstance(t_search_min_max[1], datetime.datetime):
        t_search_min_max = [pd.to_datetime(t_search_min_max[0]), pd.to_datetime(t_search_min_max[1])]
    # Need to round t_search_min down to nearest freq and t_search_max up to nearest freq
    t_search_min_max = [t_search_min_max[0].floor(freq=freq), t_search_min_max[1].ceil(freq=freq)]
    #-------------------------
    # Generate the needed timestamps according to requirement argument
    if requirement=='all':
        ts_req = pd.date_range(start=t_search_min_max[0], end=t_search_min_max[1], freq=freq)
    elif requirement=='endpoints':
        ts_req = t_search_min_max
    else:
        assert(0)
    #-------------------------
    # Generate the series containing all pass values (index is SN, value is boolean representing pass/fail)
    pass_srs = df.groupby(SN_col).apply(
        lambda x: check_found_ami_for_SN(
            df_i=x, 
            ts_req=ts_req, 
            time_col=time_col, 
            SN_col=SN_col
        )
    )

    #-------------------------
    # In order to pass, all SNs must have the required AMI
    if all(pass_srs):
        return True
    else:
        return False

In [28]:
def check_found_ami_for_all_SNs_in_outage_2(
    df, 
    t_search_min_max, 
    requirement='all', 
    time_col='starttimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
):
    r"""
    """
    #-------------------------
    assert(requirement in ['all', 'endpoints'])
    #-------------------------
    # Need frequency for rounding and generating required timestamps (if requirement=='all')
    freq = Utilities_df.determine_freq_in_df_col(
        df=df, 
        groupby_SN=True, 
        return_val_if_not_found='error', 
        assert_single_freq=True, 
        assert_expected_freq_found=False, 
        expected_freq=pd.Timedelta('15 minutes'), 
        time_col=time_col, 
        SN_col=SN_col
    )
    #-------------------------
    assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]) and len(t_search_min_max)==2)
    # t_search_min_max must be datetime object
    if not isinstance(t_search_min_max[0], datetime.datetime) or not isinstance(t_search_min_max[1], datetime.datetime):
        t_search_min_max = [pd.to_datetime(t_search_min_max[0]), pd.to_datetime(t_search_min_max[1])]
    # Need to round t_search_min down to nearest freq and t_search_max up to nearest freq
    t_search_min_max = [t_search_min_max[0].floor(freq=freq), t_search_min_max[1].ceil(freq=freq)]
    #-------------------------
    # Generate the needed timestamps according to requirement argument
    if requirement=='all':
        ts_req = pd.date_range(start=t_search_min_max[0], end=t_search_min_max[1], freq=freq)
    elif requirement=='endpoints':
        ts_req = t_search_min_max
    else:
        assert(0)
    #-------------------------
    # Generate a list containing the pass values
    pass_vals = []
    for SN_i in df[SN_col].unique().tolist():
        df_i = df[df[SN_col]==SN_i]
        pass_i = check_found_ami_for_SN(
            df_i=df_i, 
            ts_req=ts_req, 
            time_col=time_col, 
            SN_col=SN_col, 
        )
        pass_vals.append(pass_i)
    #-------------------------
    # In order to pass, all SNs must have the required AMI
    if all(pass_vals):
        return True
    else:
        return False

In [30]:
# Moved to DOVSAudit
def assess_outage_inclusion_requirements(
    ami_df_i, 
    outg_rec_nb, 
    dovs_df, 
    max_pct_PNs_missing_allowed=0, 
    ami_df_i_info_dict=None, 
    dovs_df_info_dict=None, 
    check_found_ami_for_all_SNs_kwargs=None
):
    r"""
    Check whether or not outage should be included in analysis.
    Returns boolean; True if the outage should be included, False if not
    This check includes:
        - ensure minimum percentage of PNs listed in DOVS are found in AMI
            - By default, all PNs must be included for an outage to be deemed suitable
            - Value set by max_pct_PNs_missing_allowed parameter, which should be between 0 and 100
        - ensure usable AMI data are found for each meter
            - Enforced using the function check_found_ami_for_all_SNs_in_outage, which has a
                requirement parameter which (at the time of writing) may be set to 'all' or 'endpoints'
        
    ami_df_i:
        AMI DataFrame for a single outage.
        This should be reduced down to only the data of interest, e.g., only data with 'aep_derived_uom'=='VOLT'
          and 'aep_srvc_qlty_idntfr'=='AVG' or instantaneous voltage data after reduction is run (using, e.g.,
          the function reduce_INSTV_ABC_1_vals_in_df)
          
    outg_rec_nb:
        The outage record number, used to identify the outage information in dovs_df
        
    dovs_df:
        DOVS DataFrame containing information about the outage.
        The preferred form for this DF is consolidated form, meaning one row per outage with the elements
          of the premise column being list objects
          
    max_pct_PNs_missing_allowed:
        The maximum percent of PNs found in DOVS missing from AMI allowed for the outage to be included (to return True)
        default: 0 (meaning all PNs must be found)
        
          
    ami_df_i_info_dict:
        Gives necessary information regarding ami_df_i.
        See default values in dflt_ami_df_i_info_dict below
        PN_col:
            default: 'aep_premise_nb'
        time_col:
            default: 'starttimeperiod_local', 
        value_col:
            default: 'value', 
        SN_col: 
            default: 'serialnumber', 
        outg_rec_nb_col:
            default: 'OUTG_REC_NB_GPD_FOR_SQL'
            Not necessary in ami_df_i, but if present, the value will be compared to outg_rec_nb to ensure consistency
        outg_t_beg_col:
            default: 'DT_OFF_TS_FULL'
            Not necessary in ami_df_i, but if present, the value will be compared to the value extracted from dovs_df
        outg_t_end_col:
            default: 'DT_ON_TS'
            Not necessary in ami_df_i, but if present, the value will be compared to the value extracted from dovs_df
          
    dovs_df_info_dict:
        Gives necessary information regarding dovs_df, such as whether or not it is consolidated and the
          names of needed columns (e.g., PN_col)
        Default values given in code below in dflt_dovs_df_info_dict variable.
            is_consolidated:
                default: True
            PN_col:
                default: 'premise_nbs'
            outg_rec_nb_idfr:
                default: 'index'
                This directs where the outg_rec_nbs are stored in dovs_df, which can be a column or the index.
                This should be a string, list, or tuple.
                If the outg_rec_nbs are located in a column, idfr should simply be the column
                    - Single index columns --> simple string
                    - MultiIndex columns   --> appropriate tuple to identify column
                If the outg_rec_nbs are located in the index:
                    - Single level index --> simple string 'index' or 'index_0'
                    - MultiIndex index:  --> 
                        - string f'index_{level}', where level is the index level containing the outg_rec_nbs
                        - tuple of length 2, with 0th element ='index' and 1st element = idx_level_name where
                            idx_level_name is the name of the index level containing the outg_rec_nbs
            outg_t_beg_col:
                default: 'DT_OFF_TS_FULL'
            outg_t_end_col:
                default: 'DT_ON_TS'
                
    check_found_ami_for_all_SNs_kwargs:
        Arguments to be fed into check_found_ami_for_all_SNs_in_outage function.
        See default values in dflt_check_found_ami_for_all_SNs_kwargs in code below.
        User should really only set t_search_min_max and requirement
            df:
                Does not need to be input by user, as it will be set to ami_df_i
            t_search_min_max:
                If not set by user, this will be set to [out_t_beg-pd.Timedelta('1hour'), out_t_end+pd.Timedelta('1hour')]
            requirement:
                default: 'all'
            time_col:
                Does not need to be input by user
            value_col:
                Does not need to be input by user
            SN_col:
                Does not need to be input by user
    """
    #-------------------------
    if ami_df_i.shape[0]==0:
        return False
    
    #--------------------------------------------------
    # Grab the relevant info from dovs_df
    #-------------------------
    dflt_dovs_df_info_dict = dict(
        is_consolidated=True,
        PN_col='premise_nbs', 
        outg_rec_nb_idfr='index', 
        outg_t_beg_col='DT_OFF_TS_FULL', 
        outg_t_end_col='DT_ON_TS'
    )
    dovs_df_info_dict = Utilities.supplement_dict_with_default_values(dovs_df_info_dict, dflt_dovs_df_info_dict, inplace=False)
    
    #-------------------------
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df=dovs_df, 
        outg_rec_nb=outg_rec_nb, 
        outg_rec_nb_idfr=dovs_df_info_dict['outg_rec_nb_idfr'], 
        assert_outg_rec_nb_found=True
    )
    
    #-------------------------
    # Make sure the necessary columns are in dovs_df_i
    necessary_dovs_cols = [
        dovs_df_info_dict['PN_col'], 
        dovs_df_info_dict['outg_t_beg_col'], 
        dovs_df_info_dict['outg_t_end_col']
    ]
    assert(len(set(necessary_dovs_cols).difference(set(dovs_df_i.columns.tolist())))==0)
    
    #-------------------------
    # Grab the needed info from dovs_df_i
    if dovs_df_info_dict['is_consolidated']:
        assert(dovs_df_i.shape[0]==1)
        out_t_beg = dovs_df_i.iloc[0][dovs_df_info_dict['outg_t_beg_col']]
        out_t_end = dovs_df_i.iloc[0][dovs_df_info_dict['outg_t_end_col']]
        PNs_dovs  = dovs_df_i.iloc[0][dovs_df_info_dict['PN_col']]
        assert(Utilities.is_object_one_of_types(PNs_dovs, [list, tuple, set]))
    else:
        assert(dovs_df_i[dovs_df_info_dict['outg_t_beg_col']].nunique()==1)
        assert(dovs_df_i[dovs_df_info_dict['outg_t_end_col']].nunique()==1)
        out_t_beg = dovs_df_i[dovs_df_info_dict['outg_t_beg_col']].unique().tolist()[0]
        out_t_end = dovs_df_i[dovs_df_info_dict['outg_t_end_col']].unique().tolist()[0]
        PNs_dovs  = dovs_df_i[dovs_df_info_dict['PN_col']].unique().tolist()
        
    #--------------------------------------------------
    # Determine the PNs found in ami_df and compare to those from DOVS
    #-------------------------
    dflt_ami_df_i_info_dict = dict(
        PN_col='aep_premise_nb', 
        time_col='starttimeperiod_local', 
        value_col='value', 
        SN_col='serialnumber', 
        outg_rec_nb_col='OUTG_REC_NB_GPD_FOR_SQL', 
        outg_t_beg_col='DT_OFF_TS_FULL', 
        outg_t_end_col='DT_ON_TS'        
    )
    ami_df_i_info_dict = Utilities.supplement_dict_with_default_values(ami_df_i_info_dict, dflt_ami_df_i_info_dict, inplace=False)
    
    #-------------------------
    # Make sure the necessary columns are present in ami_df_i
    necessary_ami_cols = [
        ami_df_i_info_dict['PN_col'], 
        ami_df_i_info_dict['time_col'], 
        ami_df_i_info_dict['value_col'], 
        ami_df_i_info_dict['SN_col']
    ]
    assert(len(set(necessary_ami_cols).difference(set(ami_df_i.columns.tolist())))==0)
    
    #-------------------------
    # If outage information also contained in ami_df, compare against expected values
    if ami_df_i_info_dict['outg_rec_nb_col'] in ami_df_i.columns.tolist():
        assert(ami_df_i[ami_df_i_info_dict['outg_rec_nb_col']].nunique()==1)
        assert(ami_df_i[ami_df_i_info_dict['outg_rec_nb_col']].unique().tolist()[0]==outg_rec_nb)
    if ami_df_i_info_dict['outg_t_beg_col'] in ami_df_i.columns.tolist():
        assert(ami_df_i[ami_df_i_info_dict['outg_t_beg_col']].nunique()==1)
        assert(ami_df_i[ami_df_i_info_dict['outg_t_beg_col']].unique().tolist()[0]==out_t_beg)
    if ami_df_i_info_dict['outg_t_end_col'] in ami_df_i.columns.tolist():
        assert(ami_df_i[ami_df_i_info_dict['outg_t_end_col']].nunique()==1)
        assert(ami_df_i[ami_df_i_info_dict['outg_t_end_col']].unique().tolist()[0]==out_t_end)
    
    #-------------------------
    # Grab PNs_ami
    PNs_ami = ami_df_i[ami_df_i_info_dict['PN_col']].unique().tolist()
    
    #-----*****-----*****-----*****-----*****-----
    # Find difference in PNs between DOVS and AMI
    dff_pns = list(set(PNs_dovs).difference(set(PNs_ami)))
    
    # NOTE: For the current analysis, where data are queried according to premise numbers, there should
    #         never be a situation where PNs found in AMI not present in DOVS.
    #       For now, assert this to be true.
    #       In the future, if I instead query at the transformer level to try to pick up PNs possible missed
    #         by DOVS, this may not be the case.
    assert(len(set(PNs_ami).difference(set(PNs_dovs)))==0)
    
    # Check if difference within allowable range, as set by max_pct_PNs_missing_allowed
    pct_pns_missing = 100.*len(dff_pns)/len(PNs_dovs)
    if pct_pns_missing > max_pct_PNs_missing_allowed:
        return False
    
    #--------------------------------------------------
    # Check that usable AMI data are found for each meter
    #   At this point, the minimum percentage of PNs listed in DOVS found in AMI criterion passed
    #-------------------------
    dflt_check_found_ami_for_all_SNs_kwargs = dict(
        t_search_min_max=[out_t_beg-pd.Timedelta('1hour'), out_t_end+pd.Timedelta('1hour')], 
        requirement='all', 
        time_col=ami_df_i_info_dict['time_col'], 
        value_col=ami_df_i_info_dict['value_col'], 
        SN_col=ami_df_i_info_dict['SN_col'], 
    )
    check_found_ami_for_all_SNs_kwargs = Utilities.supplement_dict_with_default_values(
        check_found_ami_for_all_SNs_kwargs, 
        dflt_check_found_ami_for_all_SNs_kwargs, 
        inplace=False
    )
    # Make sure the user didn't accidentally set time_col, value_col, or SN_col incorrectly
    assert(check_found_ami_for_all_SNs_kwargs['time_col']==ami_df_i_info_dict['time_col'])
    assert(check_found_ami_for_all_SNs_kwargs['value_col']==ami_df_i_info_dict['value_col'])
    assert(check_found_ami_for_all_SNs_kwargs['SN_col']==ami_df_i_info_dict['SN_col'])
    
    check_found_ami_for_all_SNs_kwargs['df'] = ami_df_i
    
    #-------------------------
    # Run ami_df_i through check_found_ami_for_all_SNs_in_outage
    all_found_ami_i = check_found_ami_for_all_SNs_in_outage(**check_found_ami_for_all_SNs_kwargs)
    
    # Since minimum percentage of PNs already passed, all_found_ami_i can simply be returned here
    return all_found_ami_i

In [31]:
# MOVED TO DOVSAudit
def choose_best_slicer_and_perform_slicing(
    df, 
    slicers, 
    groupby_SN=False, 
    t_search_min_max=None, 
    time_col='starttimeperiod_local', 
    value_col=None, 
    SN_col='serialnumber', 
    return_sorted=True
):
    r"""
    From slicers, choose the best option, slice the full df, and return.
    This can be done for the entire df as a whole, or by serial number, depending on the value of groupby_SN
    If t_search_min_max is set, the best slicer is chosen from the subset of df, df_choose_slcr, with
      time values within the constraints of t_search_min_max
    Although the best slicer is determined from the subset, the slicing is performed on the entirety of df.
    How is the best slicer chosen?
        If value_col is None, the best slicer is that with the most rows after slicing df_choose_slcr.
        If value_col is set, the best slicer is that with the most not-NA value_col entries after slicing df_choose_slcr.
        NOTE: If multiple best slicers exists, the first is chosen
        
    return_sorted:
        If true, the returned df is sorted according to time_col, SN_col
    """
    #----------------------------------------------------------------------------------------------------
    # NOTE: Need to exercise groupby_SN==True option here (instead of, e.g., after df_choose_slcr is found),
    #         so that slicing is done on entirety of df, as described in documentation
    if groupby_SN:
        # NOTE: groupby_SN MUST BE SET TO FALSE BELOW, otherwise infinite loop!
        return_df = df.groupby([SN_col], as_index=False, group_keys=False).apply(
            lambda x: choose_best_slicer_and_perform_slicing(
                df=x, 
                slicers=slicers, 
                groupby_SN=False, 
                t_search_min_max=t_search_min_max, 
                time_col=time_col, 
                value_col=value_col, 
                SN_col=SN_col, 
                return_sorted=return_sorted
            )
        )
        #-----
        # Sort, if return_sorted==True
        if return_sorted:
            return_df = return_df.sort_values(by=[time_col, SN_col])
        #-----
        return return_df
    #----------------------------------------------------------------------------------------------------
    assert(Utilities.is_object_one_of_types(slicers, [list, tuple]))
    #-------------------------
    if t_search_min_max is not None:
        assert(Utilities.is_object_one_of_types(t_search_min_max, [list, tuple]))
        assert(len(t_search_min_max)==2)
        #-----
        # At least one of t_search_min_max should not be None
        assert(t_search_min_max[0] is not None or t_search_min_max[1] is not None)
        if t_search_min_max[0] is None:
            t_search_min_max[0] = pd.Timestamp.min
        if t_search_min_max[1] is None:
            t_search_min_max[1] = pd.Timestamp.max
        #-----
        df_choose_slcr = df[
            (df[time_col]>=t_search_min_max[0]) & 
            (df[time_col]<=t_search_min_max[1])
        ]
    else:
        df_choose_slcr = df
    #-------------------------
    # Construct slcrs_w_counts to choose best slicer
    slcrs_w_counts = []
    for slcr_i in slicers:
        if value_col is None:
            counts_i = slcr_i.perform_slicing(df_choose_slcr).shape[0]
        else:
            counts_i = slcr_i.perform_slicing(df_choose_slcr)[value_col].notna().sum()
        slcrs_w_counts.append((slcr_i, counts_i))
    #-------------------------
    # Find the winner from slcrs_w_counts
    slicer = max(slcrs_w_counts, key=lambda x: x[1])[0]
    
    #-------------------------
    # Perform the slicing
    return_df = slicer.perform_slicing(df).copy()
    
    #-------------------------
    # Sort, if return_sorted==True
    if return_sorted:
        return_df = return_df.sort_values(by=[time_col, SN_col])
    
    #-------------------------
    return return_df

In [32]:
# MOVED TO DOVSAudit
def reduce_INSTV_ABC_1_vals_in_df(
    df, 
    value_col='value', 
    aep_derived_uom_col='aep_derived_uom', 
    aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
    output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1', 
    include_index_in_shared_cols=True, 
    gpby_dropna=True
):
    r"""
    Function to reduce down multiple INSTVA1/INSTVB1/INSTVC1 for each timestamp into a single average value.
    -----
    In general, it is desired that each serial number have a single value per time stamp.
    For this function, df can have a mix of INSTV(ABC)1 and AVG voltage readings, HOWEVER, it is expected that for
      each SN/timestamp combination, there should only be INSTV(ABC)1 values or a single AVG value.
    This is built only for aep_derived_uom==VOLT, and aep_srvc_qlty_idntfr in [INSTVA1, INSTVB1, INSTVC1, AVG]
    
    include_index_in_shared_cols:
        As various elements will be grouped/rows collapsed/however you want to think about it, the index will
          necessarily be lost, 
        In many cases (e.g., when the indices are time stamps) the indices for those being combined should be 
          shared (but, pandas doesn't know that).
        If this is the case, set include_index_in_shared_cols==True
        NOTE: For the foreseeable applications, this only really makes sense if the indices are datetime objects,
              so this will be enforced.
    """
    #-------------------------
    assert(df[aep_derived_uom_col].nunique()==1)
    assert(df[aep_derived_uom_col].unique()[0]=='VOLT')
    #-----
    assert(len(set(df[aep_srvc_qlty_idntfr_col].unique().tolist()).difference(
        set(['INSTVA1', 'INSTVB1', 'INSTVC1', 'AVG'])
    ))==0)
    #-------------------------
    # Grab og_cols to maintain column ordering in output
    og_cols = df.columns.tolist()
    
    #-------------------------
    grp_by_cols = [x for x in df.columns.tolist() 
                   if x not in [value_col, aep_derived_uom_col, aep_srvc_qlty_idntfr_col]]
    cols_shared_by_groups = [aep_derived_uom_col]
    agg_dict = {
        value_col:               'mean', 
        aep_derived_uom_col:     'first', 
        aep_srvc_qlty_idntfr_col: lambda x: ' '.join(natsorted(set(x)))
    }
    if include_index_in_shared_cols:
        # As various elements will be grouped/rows collapsed/however you want to think about it, the index will
        #   necessarily be lost, even though the indices for those being combined should be shared (but, pandas
        #   doesn't know that).
        # In order to retain the index, stored it in a temporary column and re-set it later
        # NOTE: This assumes there is a single index level.  If MultiIndex, method will need re-worked
        assert(df.index.nlevels==1)
        assert(is_datetime64_dtype(df.index.dtype))
        if df.index.name is not None and df.index.name not in df.columns.tolist():
            tmp_idx_col = df.index.name
        else:
            tmp_idx_col = Utilities.generate_random_string()
        df = df.reset_index(drop=False, names=tmp_idx_col)
        cols_shared_by_groups.append(tmp_idx_col)
        agg_dict[tmp_idx_col] = 'first'
    #----------
    # Make sure all columns in cols_shared_by_groups have a single value per group
    assert((df.groupby(grp_by_cols, dropna=gpby_dropna)[cols_shared_by_groups].nunique()<=1).all().all())    
    
    #----------
    # Do aggregation
    return_df = df.groupby(grp_by_cols, dropna=gpby_dropna, as_index=False, group_keys=False).agg(agg_dict)
    
    #-------------------------
    # Due to join operation about with natsorted and set, the only possible values for aep_srvc_qlty_idntfr_col
    #   in return_df are the sorted combinations of ['INSTVA1', 'INSTVB1', 'INSTVC1']
    #   e.g., 'INSTVA1', 'INSTVB1', 'INSTVC1', 'INSTVA1 INSTVB1', ... 'INSTVA1 INSTVB1 INSTVC1'
    # Assert that this is true
    accptbl_fnl_srvc_qlty_idntfrs = ['INSTVA1', 'INSTVB1', 'INSTVC1']
    #-----
    combs = []
    for i in range(1, len(accptbl_fnl_srvc_qlty_idntfrs)+1):
        els = [list(x) for x in itertools.combinations(accptbl_fnl_srvc_qlty_idntfrs, i)]
        combs.extend(els)
    #-----
    accptbl_fnl_srvc_qlty_idntfrs = [' '.join(x) for x in combs]
    accptbl_fnl_srvc_qlty_idntfrs.append('AVG')
    #-----
    assert(len(set(df[aep_srvc_qlty_idntfr_col].unique().tolist()).difference(set(accptbl_fnl_srvc_qlty_idntfrs)))==0)
    #-------------------------
    # Set rows with aep_srvc_qlty_idntfr!='VOLT' equal to output_aep_srvc_qlty_idntfr
    return_df.loc[return_df[aep_srvc_qlty_idntfr_col]!='AVG', aep_srvc_qlty_idntfr_col] = output_aep_srvc_qlty_idntfr
    #-------------------------
    if include_index_in_shared_cols:
        # Set index back to original values
        return_df = return_df.set_index(tmp_idx_col, drop=True)
    #-------------------------
    assert(len(set(og_cols).symmetric_difference(set(return_df.columns.tolist())))==0)
    return_df = return_df[og_cols]
    #-------------------------
    return return_df

In [33]:
def identify_outg_rec_nbs_to_remove(
    paths, 
    slicers, 
    verbose=False
):
    r"""
    """
    #-------------------------
    # First, need to iterate through paths to retrieve list of all outg_rec_nbs
    # This is necessary as outages can be split across multiple files, so one cannot
    #   simply iterate through the files
    paths=natsorted(paths)
    outg_rec_nbs_in_files = dict()
    for path in paths:
        assert(path not in outg_rec_nbs_in_files.keys())
        df = GenAn.read_df_from_csv(path)
        outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
    outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
    all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())
    
    #-------------------------
    # Build dovs_df
    dovs = DOVSOutages(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args=None, 
        init_df_in_constructor=True,
        build_sql_function=DOVSOutages_SQL.build_sql_outage, 
        build_sql_function_kwargs=dict(
            outg_rec_nbs=all_outg_rec_nbs, 
            field_to_split='outg_rec_nbs', 
            include_premise=True
        ), 
        build_consolidated=True
    )
    dovs_df = dovs.df.copy()
    
    #-------------------------
    # Now, iterate through all outages
    outg_rec_nbs_to_remove = []
    for i_outg, outg_rec_nb in enumerate(all_outg_rec_nbs):
        if verbose:
            print(f'\n\ti_outg: {i_outg+1}/{len(all_outg_rec_nbs)}')
            print(f'\toutg_rec_nb = {outg_rec_nb}')
        ami_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_dict[outg_rec_nb])

        #--------------------------------------------------
        ami_df_i = ami_df[ami_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()     
        
        # Although I cannot yet call choose_best_slicer_and_perform_slicing and reduce_INSTV_ABC_1_vals_in_df, 
        #   as the standard cleaning and conversions must be done first, I am able to cut down the size of
        #   ami_df_i by joining the slicers with 'or' statements.
        # Thus, ami_df_i will be reduced to only the subset of data which will be considered in 
        #   choose_best_slicer_and_perform_slicing
        # As mentioned, this will cut down the size of ami_df_i and will also save time and resources by not having
        #   to run entire DF through cleaning and conversions procedures.
        ami_df_i = DFSlicer.combine_slicers_and_perform_slicing(
            df=ami_df_i, 
            slicers=slicers, 
            join_slicers='or'
        )
        if ami_df_i.shape[0]==0:
            outg_rec_nbs_to_remove.append(outg_rec_nb)
            continue        
        
        #--------------------------------------------------
        ami_df_i = AMINonVee.perform_std_initiation_and_cleaning(ami_df_i)
        #-----
        # Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
        ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=ami_df_i, 
            time_col='starttimeperiod', 
            placement_col='starttimeperiod_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=ami_df_i, 
            time_col='endtimeperiod', 
            placement_col='endtimeperiod_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        #--------------------------------------------------
        ami_df_i = choose_best_slicer_and_perform_slicing(
            df=ami_df_i, 
            slicers=slicers, 
            groupby_SN=True, 
            t_search_min_max=None, 
            time_col='starttimeperiod_local', 
            value_col=None, 
            SN_col='serialnumber', 
            return_sorted=True
        )

        ami_df_i = reduce_INSTV_ABC_1_vals_in_df(
            df=ami_df_i, 
            value_col='value', 
            aep_derived_uom_col='aep_derived_uom', 
            aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
            output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1'
        )
        
        if ami_df_i.shape[0]==0:
            outg_rec_nbs_to_remove.append(outg_rec_nb)
            continue
            
        to_include_i = assess_outage_inclusion_requirements(
            ami_df_i=ami_df_i, 
            outg_rec_nb=outg_rec_nb, 
            dovs_df=dovs_df, 
            max_pct_PNs_missing_allowed=0

        )
        if not to_include_i:
            outg_rec_nbs_to_remove.append(outg_rec_nb)
    #--------------------------------------------------
    return outg_rec_nbs_to_remove

In [34]:
def build_detailed_summary_df(
    means_df, 
    best_ests_df_w_db_lbl,
    CI_tot, 
    CMI_tot, 
    n_PNs_ami, 
    outg_rec_nb, 
    dovs_df_i, 
    db_label_col='db_label', 
    winner_min_col='winner_min', 
    winner_max_col='winner_max', 
    PN_col='PN',
    i_outg_col='i_outg'
):
    r"""
    Build a summmary_df for outage.
    This summary is really just a single-row DataFrame (i.e., a series).
    The intent is for this to be used when running over a large number of outages.
    
    PN_col:
        In the most typical case, where CI/CMI is set by premise number, this should be set to the
          premise number column.
        However, if one wants to calculate CI/CMI by serial number, this should be set to the serial number column.
    """
    #-------------------------
    # Don't want to alter means_df, best_ests_df_w_db_lbl outside of function
    means_df=means_df.copy()
    best_ests_df_w_db_lbl=best_ests_df_w_db_lbl.copy()
    
    #--------------------------------------------------
    # Get DOVS info
    assert(
        isinstance(dovs_df_i, pd.Series) or 
        (isinstance(dovs_df_i, pd.DataFrame) and dovs_df_i.shape[0]==1)
    )
    #-------------------------
    # Make sure OUTG_REC_NB in dovs_df_i agrees with input value safecheck
    if isinstance(dovs_df_i, pd.DataFrame):
        if dovs_df_i.index.name=='OUTG_REC_NB':
            assert(dovs_df_i.index[0]==outg_rec_nb)
        else:
            assert('OUTG_REC_NB' in dovs_df_i.columns)
            assert(dovs_df_i.iloc[0]['OUTG_REC_NB']==outg_rec_nb)
    else:
        if 'OUTG_REC_NB' in dovs_df_i.index:
            assert(dovs_df_i['OUTG_REC_NB']==outg_rec_nb)
        else:
            assert(dovs_df_i.name==outg_rec_nb)

    #-------------------------
    # Make dovs_df_i a pd.Series object (if pd.DataFrame, this will collapse to pd.Series, if
    #   pd.Series, this will have no effect)
    # This isn't necessary, it just eliminate the need to use .loc[0] all over the place
    dovs_df_i = dovs_df_i.squeeze()

    #-------------------------
    # Grab needed entries from DOVS data
    dovs_outg_t_beg = dovs_df_i['DT_OFF_TS_FULL']
    dovs_outg_t_end = dovs_df_i['DT_ON_TS']
    ci_dovs         = dovs_df_i['CI_NB']
    cmi_dovs        = dovs_df_i['CMI_NB']
    n_PNs_dovs      = len(set(dovs_df_i['premise_nbs']))
    outage_nb       = dovs_df_i['OUTAGE_NB']
    mjr_cause_cd    = dovs_df_i['MJR_CAUSE_CD']
    mnr_cause_cd    = dovs_df_i['MNR_CAUSE_CD']
    dvc_typ_nb      = dovs_df_i['DVC_TYP_NM']   
    opco_id         = dovs_df_i['OPERATING_UNIT_ID']
    
    #--------------------------------------------------
    # If only a single sub-outage, only return 'Full Outage' entry
    if means_df.shape[0]==1:
        return_df = pd.DataFrame(
            data=dict(
                outg_i_beg=means_df.iloc[0][winner_min_col], 
                outg_i_end=means_df.iloc[0][winner_max_col], 
                CI_i=CI_tot,
                CMI_i=CMI_tot
            ), 
            index=pd.MultiIndex.from_tuples(
                [(outage_nb, outg_rec_nb, 'Full Outage')], 
                names=['OUTAGE_NB','OUTG_REC_NB', 'Outage Subset']
            )
        )
        # Remove fractional seconds
        return_df['outg_i_beg']=return_df['outg_i_beg'].dt.strftime('%Y-%m-%d %H:%M:%S')
        return_df['outg_i_end']=return_df['outg_i_end'].dt.strftime('%Y-%m-%d %H:%M:%S')

    #--------------------------------------------------
    else:  
        #-------------------------
        # The following protects against the case where one inputs means_df/best_ests_df_w_db_lbl which were built
        #   at the serial number level, but tries to run this function at the premise number level (by, e.g., setting
        #   PN_col='PN')
        # In such a case, the CI would still come out correct (due to how it is calculated below), but the CMI would be 
        #   larger than in reality, as premises with multiple SNs would get counted multiple times.
        if (best_ests_df_w_db_lbl.groupby([PN_col, i_outg_col]).size()>1).any():
            print('Each combination of PN_col, i_outg_col in best_ests_df_w_db_lbl should have a single entry')
            print('The input violates this!  CRASH IMMINENT!!!!!')
            print('Likely, one is trying to calculate CI/CMI by premise, but forgot to run combine_PNs_in_best_ests_df')
        assert((best_ests_df_w_db_lbl.groupby([PN_col, i_outg_col]).size()==1).all())

        #-------------------------
        # Add nPNs column to means_df, as this will be used for CI of the sub-outages
        n_PNs_col = f'n_{PN_col}s'
        #-----
        means_df = add_nPNs_to_means_df(
            means_df=means_df, 
            best_ests_df_w_db_lbl=best_ests_df_w_db_lbl,
            db_label_col=db_label_col, 
            PN_col=PN_col, 
            n_PNs_col=n_PNs_col
        )

        #-------------------------
        # Sort means_df
        means_df.sort_values(by=[winner_min_col, winner_max_col])

        #-------------------------
        # Create winner_max-winner_min column (winner_delta_col), used to calculate CMI
        winner_delta_col = Utilities.generate_random_string()
        best_ests_df_w_db_lbl[winner_delta_col] = best_ests_df_w_db_lbl[winner_max_col]-best_ests_df_w_db_lbl[winner_min_col]

        #-------------------------
        # Build a series containing the CMI values by db_label
        cmi_by_label = best_ests_df_w_db_lbl.groupby([db_label_col])[winner_delta_col].apply(lambda x: x.sum().total_seconds()/60)
        cmi_by_label.name = 'CMI_i'
        assert(len(set(cmi_by_label.index).symmetric_difference(means_df.index))==0)

        # The CMI_tot input should equal cmi_by_label.sum() (likely they won't be EXACTLY the same, hence
        #   the use of Utilities.are_approx_equal)
        # NOTE: The same cannot be said for CI, as individual meters can suffer multiple sub-outages
        assert(Utilities.are_approx_equal(CMI_tot, cmi_by_label.sum(), precision=0.00001))

        #-------------------------
        # Construct return_df
        return_df = means_df[[winner_min_col, winner_max_col, n_PNs_col]].copy()
        return_df = return_df.merge(cmi_by_label, left_index=True, right_index=True, how='left')

        # Remove fractional seconds
        return_df[winner_min_col]=return_df[winner_min_col].dt.strftime('%Y-%m-%d %H:%M:%S')
        return_df[winner_max_col]=return_df[winner_max_col].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Adjust column names
        return_df=return_df.rename(columns={
            winner_min_col:'outg_i_beg', 
            winner_max_col:'outg_i_end', 
            n_PNs_col:'CI_i'
        })

        # Change index to sub-outage numbers
        return_df.index = [f'Sub-outage {i}' for i in range(1, return_df.shape[0]+1)]

        #-------------------------
        # Build a row containing the net outage information
        net_outg_row = pd.DataFrame(
            data=dict(
                outg_i_beg=return_df['outg_i_beg'].min(), 
                outg_i_end=return_df['outg_i_end'].max(), 
                CI_i=CI_tot,
                CMI_i=CMI_tot
            ), 
            index=['Full Outage']
        )

        # Add net_outg_row to beginning of return_df
        return_df = pd.concat([net_outg_row, return_df])
        return_df.index.name = 'Outage Subset'

        #-------------------------
        # Include the OUTG_REC_NB and OUTAGE_NB
        return_df = Utilities_df.prepend_level_to_MultiIndex(return_df, level_val=outg_rec_nb, level_name='OUTG_REC_NB', axis=0)
        return_df = Utilities_df.prepend_level_to_MultiIndex(return_df, level_val=outage_nb, level_name='OUTAGE_NB', axis=0)
        
    #--------------------------------------------------
    # Add n_PNs_ami, n_PNs_dovs, and %
    return_df['n_PNs_ami']     = n_PNs_ami
    return_df['n_PNs_DOVS']    = n_PNs_dovs
    return_df['pct_PNs_found'] = 100*return_df['n_PNs_ami']/return_df['n_PNs_DOVS']
    #-------------------------
    # Add DOVS
    return_df['DT_OFF_TS_FULL']    = dovs_outg_t_beg
    return_df['DT_ON_TS']          = dovs_outg_t_end
    return_df['CI_NB']             = ci_dovs
    return_df['CMI_NB']            = cmi_dovs
    return_df['MJR_CAUSE_CD']      = mjr_cause_cd
    return_df['MNR_CAUSE_CD']      = mnr_cause_cd
    return_df['DVC_TYP_NM']        = dvc_typ_nb
    return_df['OPERATING_UNIT_ID'] = opco_id
    
    #--------------------------------------------------
    # For new format, I still want 'OUTAGE_NB' and 'OUTG_REC_NB' as indices, but I'm going to move 'Outage Subset'
    #   into the columns
    # I also want to list all of the DOVS attributes first
    return_df=return_df.reset_index(level='Outage Subset')
    #-----
    return_df = Utilities_df.move_cols_to_front(
        return_df, 
        [
            'DT_OFF_TS_FULL', 
            'DT_ON_TS', 
            'CI_NB', 
            'CMI_NB', 
            'MJR_CAUSE_CD', 
            'MNR_CAUSE_CD', 
            'DVC_TYP_NM', 
            'OPERATING_UNIT_ID', 
            'n_PNs_DOVS'
        ]
    )
    #-------------------------
    return return_df

In [35]:
def drop_multiindex_duplicate_idxs_for_csv_output(df):
    r"""
    Eliminate duplicate values from MultiIndex index before writing to CSV to make reading by eye easier.
    IMPORTANT!!!!! The CSV generated here isn't useful for futher manipulation using code, only for inspection by eye
                   Thus, it's intended more for when I deliver results to others, not for use myself
    -----
    Essentially, instead of the output looking like:
        idx0     idx1     col1   col2   col3
        a        1        ...    ...    ...     
        a        2        ...    ...    ...
        b        2        ...    ...    ...
        b        3        ...    ...    ...
    
    It will instead look like:
        idx0     idx1     col1   col2   col3
        a        1        ...    ...    ... 
                 2        ...    ...    ...
        b        2        ...    ...    ...
                 3        ...    ...    ...
    """
    #-------------------------
    # Make sure df has MultiIndex index, and each level has a name
    assert(df.index.nlevels>1)
    assert((len(df.index.names)==df.index.nlevels))
    assert(all([True if x else False for x in df.index.names]))
    idx_names = list(df.index.names)
    #-------------------------
    df = df.reset_index()
    #-------------------------
    dupl_srs_list = []
    for i_idx_lvl in range(len(idx_names)):
        dupl_srs_i = df.duplicated(subset=idx_names[:i_idx_lvl+1])
        dupl_srs_list.append(dupl_srs_i)
    assert(len(dupl_srs_list)==len(idx_names))
    #-------------------------
    for i_idx_lvl in range(len(idx_names)):
        df[idx_names[i_idx_lvl]] = df[idx_names[i_idx_lvl]].where(~dupl_srs_list[i_idx_lvl], '')
    #-------------------------
    return df

In [36]:
# NOTE:  THERE EXISTS A NEW VERSSION OF THE BELOW FUNCTION
def plot_all_out_not(
    fig_num, 
    ami_df_i, 
    ami_df_i_out, 
    ami_df_i_not_out, 
    dovs_outg_t_beg, 
    dovs_outg_t_end, 
    cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end, 
    means_df, 
    outg_rec_nb, 
    outage_nb, 
    n_PNs_dovs, 
    n_PNs, 
    n_SNs, 
    ci_dovs, 
    cmi_dovs, 
    ci_ami, 
    cmi_ami, 
    ci_ami_dovs_beg, 
    cmi_ami_dovs_beg, 
    expand_time=pd.Timedelta('1 hour'), 
    mean_keys_to_include=['winner', 'conservative', 'zero_times']
):
    #-------------------------
    fig, axs = Plot_General.default_subplots(
        n_x=1,
        n_y=3,
        fig_num=fig_num,
        sharex=False,
        sharey=False,
        unit_figsize_width=14,
        unit_figsize_height=6, 
        return_flattened_axes=True,
        row_major=True
    )
    Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

    palette = Plot_General.get_standard_colors_dict(
        keys=ami_df_i['serialnumber'].unique().tolist(), 
        palette='colorblind'
    )

    #-------------------------
    i_subplot=0
    fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
        fig=fig, 
        ax=axs[i_subplot], 
        data=ami_df_i, 
        x='starttimeperiod_local', 
        y='value', 
        hue='serialnumber', 
        out_t_beg=dovs_outg_t_beg, 
        out_t_end=dovs_outg_t_end, 
        expand_time=expand_time, 
        plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
        data_label='', 
        title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
        ax_args=None, 
        xlabel_args=None, 
        ylabel_args=None, 
        df_mean=None, 
        df_mean_col=None, 
        mean_args=None, 
        draw_outage_limits=True, 
        draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text=dict(
            out_t_beg_text='DOVS Beg.', 
            out_t_beg_ypos=(0.12, 'ax_coord'), 
            out_t_beg_va='bottom', 
            out_t_beg_ha='center', 
            out_t_beg_color='red', 
            #-----
            out_t_end_text='DOVS End', 
            out_t_end_ypos=(0.12, 'ax_coord'), 
            out_t_end_va='bottom', 
            out_t_end_ha='center', 
            out_t_end_color='green', 
        ), 
        draw_without_hue_also=False, 
        seg_line_freq=None, 
        palette=palette
    )
    axs[i_subplot].legend().set_visible(False)
    Plot_General.set_general_plotting_args(
        ax=axs[i_subplot], 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
        ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
    )


    #-------------------------
    i_subplot=1
    if ami_df_i_out.shape[0]>0:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i_out, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=expand_time, 
            plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
            data_label='', 
            title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ), 
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_subplot].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_subplot], 
            means_df, 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=mean_keys_to_include
        )
        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(
            label=f'Out', 
            fontdict=dict(fontsize=24)
        )

    #-------------------------
    i_subplot=2
    if ami_df_i_not_out.shape[0]>0:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i_not_out, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=expand_time, 
            plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
            data_label='', 
            title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ), 
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_subplot].legend().set_visible(False)
        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(label='Not Out', fontdict=dict(fontsize=24))


    #--------------------------------------------------
    # Add legend to first plot
    patch_dovs_beg = Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS Beg.'
    )
    patch_dovs_end = Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS End'
    )
    #-----
    patch_ui_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='Beg. Uncertainty Interval'
    )
    patch_ui_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='End Uncertainty Interval'
    )
    #-----
    patch_best_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. Beg.'
    )
    patch_best_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. End'
    )
    #-------------------------
    if len(set(['conservative', 'zero_times']).difference(mean_keys_to_include))==0:
        handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end]
    else:
        handles=[patch_dovs_beg, patch_dovs_end, patch_best_beg, patch_best_end]
    #-------------------------
    leg_1 = axs[0].legend(
        title=None, 
        handles=handles, 
        bbox_to_anchor=(1, 1.025), 
        loc='upper left', 
        fontsize=15
    )

    #--------------------------------------------------
    ci_info_fontsize = 20
    left_text_x=0.95
    shift_text_down = 0

    fig.text(left_text_x, 0.745-shift_text_down, f'OUTG_REC_NB: {outg_rec_nb}', fontsize=ci_info_fontsize+4)
    fig.text(left_text_x, 0.715-shift_text_down, f"OUTAGE_NB:     {outage_nb}", fontsize=ci_info_fontsize+4)

    fig.text(left_text_x, 0.675-shift_text_down, f"#PNs from DOVS = {n_PNs_dovs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.640-shift_text_down, "----- Found in AMI -----", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.615-shift_text_down, f"#PNs = {n_PNs}", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.590-shift_text_down, f"#SNs = {n_SNs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.525-shift_text_down, '-----'*5+'\nDOVS\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.500-shift_text_down, f'CI    = {ci_dovs}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.475-shift_text_down, f'CMI = {np.round(cmi_dovs, decimals=2)}', fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.400-shift_text_down, '-----'*5+'\nUsing AMI\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.375-shift_text_down, f'CI    = {ci_ami}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.350-shift_text_down, f'CMI = {np.round(cmi_ami, decimals=2)}', fontsize=ci_info_fontsize)
    #-----
    fig.text(
        left_text_x, 0.325-shift_text_down, 
        f'$\Delta$CI    = {ci_dovs-ci_ami} ({np.round(100*(ci_dovs-ci_ami)/ci_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    fig.text(
        left_text_x, 0.300-shift_text_down, 
        f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami)/cmi_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )

    fig.text(left_text_x, 0.225-shift_text_down, '-----'*5+'\nAMI w/ DOVS t_beg\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.200-shift_text_down, f'CI    = {ci_ami_dovs_beg}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.175-shift_text_down, f'CMI = {np.round(cmi_ami_dovs_beg, decimals=2)}', fontsize=ci_info_fontsize)
    #-----
    fig.text(
        left_text_x, 0.150-shift_text_down, 
        f'$\Delta$CI    = {ci_dovs-ci_ami_dovs_beg} ({np.round(100*(ci_dovs-ci_ami_dovs_beg)/ci_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    fig.text(
        left_text_x, 0.125-shift_text_down, 
        f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami_dovs_beg, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami_dovs_beg)/cmi_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    
    #--------------------------------------------------
    return fig, axs

In [37]:
def plot_all_out_not_NEW(
    fig_num, 
    ami_df_i, 
    ami_df_i_out, 
    ami_df_i_not_out, 
    dovs_outg_t_beg, 
    dovs_outg_t_end, 
    cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end, 
    means_df, 
    outg_rec_nb, 
    outage_nb, 
    n_PNs_dovs, 
    ci_dovs, 
    cmi_dovs, 
    ci_ami, 
    cmi_ami, 
    only_connect_continuous=True, 
    data_freq=pd.Timedelta('15T'),  
    name='AMI', 
    results_2_dict=None, 
    expand_time=pd.Timedelta('1 hour'), 
    mean_keys_to_include=['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=None, 
    **kwargs
):
    #-------------------------
    if removed_due_to_overlap_col is not None:
        assert(removed_due_to_overlap_col in ami_df_i.columns.tolist())
        assert(removed_due_to_overlap_col in ami_df_i_out.columns.tolist())
        assert(removed_due_to_overlap_col in ami_df_i_not_out.columns.tolist())
    #-------------------------
    dflt_default_subplots_args = dict(
        n_x=1,
        n_y=3,
        fig_num=fig_num,
        sharex=False,
        sharey=False,
        unit_figsize_width=14,
        unit_figsize_height=6, 
        return_flattened_axes=True,
        row_major=True
    )
    if default_subplots_args is None:
        default_subplots_args = dflt_default_subplots_args
    else:
        assert(isinstance(default_subplots_args, dict))
        default_subplots_args = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=default_subplots_args, 
            default_values_dict=dflt_default_subplots_args, 
            extend_any_lists=False, 
            inplace=False
        )
    fig, axs = Plot_General.default_subplots(**default_subplots_args)
    Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

    palette = Plot_General.get_standard_colors_dict(
        keys=ami_df_i['serialnumber'].unique().tolist(), 
        palette='colorblind'
    )

    #-------------------------
    # The subplots share many common arguments.  
    # Let's collect them all first to avoid repeating code
    shared_plot_kwargs = dict(
        x                          = 'starttimeperiod_local', 
        y                          = 'value', 
        hue                        = 'serialnumber', 
        out_t_beg                  = dovs_outg_t_beg, 
        out_t_end                  = dovs_outg_t_end, 
        expand_time                = expand_time, 
        plot_time_beg_end          = [cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
        only_connect_continuous    = only_connect_continuous, 
        data_freq                  = data_freq,     
        data_label                 = '', 
        ax_args                    = None, 
        xlabel_args                = None, 
        ylabel_args                = None, 
        df_mean                    = None, 
        df_mean_col                = None, 
        mean_args                  = None, 
        draw_outage_limits         = True, 
        draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text = dict(
            out_t_beg_text  = 'DOVS Beg.', 
            out_t_beg_ypos  = (0.12, 'ax_coord'), 
            out_t_beg_va    = 'bottom', 
            out_t_beg_ha    = 'center', 
            out_t_beg_color = 'red', 
            #-----
            out_t_end_text  = 'DOVS End', 
            out_t_end_ypos  = (0.12, 'ax_coord'), 
            out_t_end_va    = 'bottom', 
            out_t_end_ha    = 'center', 
            out_t_end_color = 'green', 
        ), 
        draw_without_hue_also      = False, 
        seg_line_freq              = None, 
        palette                    = palette
    )
    
    #-------------------------
    i_subplot=0
    if removed_due_to_overlap_col is None:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i, 
            title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            **shared_plot_kwargs
        )
    else:
        fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_subplot], 
            data=ami_df_i[ami_df_i[removed_due_to_overlap_col]==False], 
            title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
            **shared_plot_kwargs
        )
        #-----
        if ami_df_i[ami_df_i[removed_due_to_overlap_col]==True].shape[0]>0:
            fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                fig=fig, 
                ax=axs[i_subplot], 
                data=ami_df_i[ami_df_i[removed_due_to_overlap_col]==True], 
                title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                lineplot_kwargs=dict(linestyle='dotted', alpha=0.5), 
                **shared_plot_kwargs
            )
    axs[i_subplot].legend().set_visible(False)
    Plot_General.set_general_plotting_args(
        ax=axs[i_subplot], 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
        ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
    )


    #-------------------------
    i_subplot=1
    if ami_df_i_out.shape[0]>0:
        if removed_due_to_overlap_col is None:
            fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                fig=fig, 
                ax=axs[i_subplot], 
                data=ami_df_i_out, 
                title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                **shared_plot_kwargs
            )
        else:
            fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                fig=fig, 
                ax=axs[i_subplot], 
                data=ami_df_i_out[ami_df_i_out[removed_due_to_overlap_col]==False], 
                title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                **shared_plot_kwargs
            )
            #-----
            if ami_df_i_out[ami_df_i_out[removed_due_to_overlap_col]==True].shape[0]>0:
                fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                    fig=fig, 
                    ax=axs[i_subplot], 
                    data=ami_df_i_out[ami_df_i_out[removed_due_to_overlap_col]==True], 
                    title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                    lineplot_kwargs=dict(linestyle='dashdot', alpha=0.5), 
                    **shared_plot_kwargs
                )
        axs[i_subplot].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_subplot], 
            means_df, 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=mean_keys_to_include
        )
        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(
            label=f'Out', 
            fontdict=dict(fontsize=24)
        )

    #-------------------------
    i_subplot=2
    if ami_df_i_not_out.shape[0]>0:
        if removed_due_to_overlap_col is None:
            fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                fig=fig, 
                ax=axs[i_subplot], 
                data=ami_df_i_not_out, 
                title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                **shared_plot_kwargs
            )
        else:
            fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                fig=fig, 
                ax=axs[i_subplot], 
                data=ami_df_i_not_out[ami_df_i_not_out[removed_due_to_overlap_col]==False], 
                title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                **shared_plot_kwargs
            )
            #-----
            if ami_df_i_not_out[ami_df_i_not_out[removed_due_to_overlap_col]==True].shape[0]>0:
                fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
                    fig=fig, 
                    ax=axs[i_subplot], 
                    data=ami_df_i_not_out[ami_df_i_not_out[removed_due_to_overlap_col]==True], 
                    title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
                    lineplot_kwargs=dict(linestyle='dashdot'), 
                    **shared_plot_kwargs
                )
        axs[i_subplot].legend().set_visible(False)
        Plot_General.set_general_plotting_args(
            ax=axs[i_subplot], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
                dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
            ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
        )
    else:
        axs[i_subplot].set_title(label='Not Out', fontdict=dict(fontsize=24))


    #--------------------------------------------------
    # Add legend to first plot
    patch_dovs_beg = Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS Beg.'
    )
    patch_dovs_end = Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS End'
    )
    #-----
    patch_ui_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='Beg. Uncertainty Interval'
    )
    patch_ui_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='End Uncertainty Interval'
    )
    #-----
    patch_best_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. Beg.'
    )
    patch_best_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. End'
    )
    #-------------------------
    if len(set(['conservative', 'zero_times']).difference(mean_keys_to_include))==0:
        handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end]
    else:
        handles=[patch_dovs_beg, patch_dovs_end, patch_best_beg, patch_best_end]
    #-------------------------
    leg_i_plot   = kwargs.get('leg_i_plot', 0)
    leg_kwargs   =  kwargs.get('leg_kwargs', None)
    dflt_leg_kwargs = dict(
        title=None, 
        handles=handles, 
        bbox_to_anchor=(1, 1.025), 
        loc='upper left', 
        fontsize=15
    )
    if leg_kwargs is None:
        leg_kwargs = dflt_leg_kwargs
    else:
        assert(isinstance(leg_kwargs, dict))
        leg_kwargs = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=leg_kwargs, 
            default_values_dict=dflt_leg_kwargs, 
            extend_any_lists=False, 
            inplace=False
        )
    leg_1 = axs[leg_i_plot].legend(**leg_kwargs)

    #--------------------------------------------------
    #--------------------------------------------------
    n_PNs = ami_df_i['aep_premise_nb'].nunique()
    n_SNs = ami_df_i['serialnumber'].nunique()
    #-------------------------
    ci_info_fontsize = kwargs.get('ci_info_fontsize', 18)
    left_text_x      = kwargs.get('left_text_x', 0.95)
    shift_text_down  = kwargs.get('shift_text_down', 0)

    fig.text(left_text_x, 0.745-shift_text_down, f'OUTG_REC_NB: {outg_rec_nb}', fontsize=ci_info_fontsize+4)
    fig.text(left_text_x, 0.720-shift_text_down, f"OUTAGE_NB:     {outage_nb}", fontsize=ci_info_fontsize+4)

    fig.text(left_text_x, 0.685-shift_text_down, f"#PNs from DOVS = {n_PNs_dovs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.660-shift_text_down, "----- Found in AMI -----", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.635-shift_text_down, f"#PNs = {n_PNs}", fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.610-shift_text_down, f"#SNs = {n_SNs}", fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.550-shift_text_down, '-----'*5+'\nDOVS\n'+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.530-shift_text_down, f'CI    = {ci_dovs}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.510-shift_text_down, f'CMI = {np.round(cmi_dovs, decimals=2)}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.490-shift_text_down, f'Beg. = {dovs_outg_t_beg.strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.470-shift_text_down, f'End  = {dovs_outg_t_end.strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)

    fig.text(left_text_x, 0.410-shift_text_down, '-----'*5+'\n{}\n'.format(name)+'-----'*5, fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.390-shift_text_down, f'CI    = {ci_ami}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.370-shift_text_down, f'CMI = {np.round(cmi_ami, decimals=2)}', fontsize=ci_info_fontsize)
    #-----
    fig.text(
        left_text_x, 0.350-shift_text_down, 
        f'$\Delta$CI    = {ci_dovs-ci_ami} ({np.round(100*(ci_dovs-ci_ami)/ci_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    fig.text(
        left_text_x, 0.330-shift_text_down, 
        f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami)/cmi_dovs, decimals=2)}%)', 
        fontsize=ci_info_fontsize
    )
    fig.text(left_text_x, 0.310-shift_text_down, f'min(Beg.) = {means_df["winner_min"].min().strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)
    fig.text(left_text_x, 0.290-shift_text_down, f'max(End) = {means_df["winner_max"].max().strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)

    if results_2_dict is not None:
        assert(isinstance(results_2_dict, dict))
        assert(set(results_2_dict.keys()).difference(set(['ci_ami', 'cmi_ami', 'means_df', 'name']))==set())
        #-----
        fig.text(left_text_x, 0.230-shift_text_down, '-----'*5+'\n{}\n'.format(results_2_dict['name'])+'-----'*5, fontsize=ci_info_fontsize)
        fig.text(left_text_x, 0.210-shift_text_down, f'CI    = {results_2_dict["ci_ami"]}', fontsize=ci_info_fontsize)
        fig.text(left_text_x, 0.190-shift_text_down, f'CMI = {np.round(results_2_dict["cmi_ami"], decimals=2)}', fontsize=ci_info_fontsize)
        #-----
        fig.text(
            left_text_x, 0.170-shift_text_down, 
            f'$\Delta$CI    = {ci_dovs-results_2_dict["ci_ami"]} ({np.round(100*(ci_dovs-results_2_dict["ci_ami"])/ci_dovs, decimals=2)}%)', 
            fontsize=ci_info_fontsize
        )
        fig.text(
            left_text_x, 0.150-shift_text_down, 
            f'$\Delta$CMI = {np.round(cmi_dovs-results_2_dict["cmi_ami"], decimals=2)} ({np.round(100*(cmi_dovs-results_2_dict["cmi_ami"])/cmi_dovs, decimals=2)}%)', 
            fontsize=ci_info_fontsize
        )
        if results_2_dict["means_df"] is not None:
            fig.text(left_text_x, 0.130-shift_text_down, f'min(Beg.) = {results_2_dict["means_df"]["winner_min"].min().strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)
            fig.text(left_text_x, 0.110-shift_text_down, f'max(End) = {results_2_dict["means_df"]["winner_max"].max().strftime("%m/%d %H:%M:%S")}', fontsize=ci_info_fontsize)
    
    #--------------------------------------------------
    return fig, axs

In [38]:
def plot_suboutg_endpts(
    fig_num, 
    ami_df_i, 
    means_df, 
    best_ests_df_w_db_lbl, 
    dovs_outg_t_beg, 
    dovs_outg_t_end, 
    outg_rec_nb, 
    expand_time=pd.Timedelta('15 minutes'), 
    mean_keys_to_include=['winner', 'conservative', 'zero_times']
):
    #-------------------------
    means_df = means_df.sort_values(by=['winner_min', 'winner_max'])
    fig, axs = Plot_General.default_subplots(
        n_x=2, 
        n_y=means_df.shape[0], 
        fig_num=fig_num
    )
    if means_df.shape[0]==1:
        axs = [axs]
    Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

    palette = Plot_General.get_standard_colors_dict(
        keys=ami_df_i['serialnumber'].unique().tolist(), 
        palette='colorblind'
    )

    #-------------------------
    for i_row in range(means_df.shape[0]):
        db_label = means_df.iloc[i_row].name
        ami_df_i_subset = ami_df_i[ami_df_i['aep_premise_nb'].isin(
            best_ests_df_w_db_lbl[best_ests_df_w_db_lbl['db_label']==db_label]['PN'].tolist()
        )]
        n_SNs = ami_df_i_subset['serialnumber'].nunique()
        #****************************************
        fig, axs[i_row][0] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_row][0], 
            data=ami_df_i_subset, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('15 minutes'), 
            plot_time_beg_end=[means_df.iloc[i_row]['conservative_min'], means_df.iloc[i_row]['zero_times_min']], 
            data_label='', 
            title_args=None, 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ),
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_row][0].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_row][0], 
            means_df.iloc[[i_row]], 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=mean_keys_to_include, 
            expand_ax_to_accommodate=False
        )
        axs[i_row][0].text(0.85, 0.9, f'#SNs = {n_SNs}', ha='center', va='center', transform=axs[i_row][0].transAxes, fontsize='xx-large')
        Plot_General.set_general_plotting_args(
            ax=axs[i_row][0], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize='large', direction='out'), 
                dict(axis='y', labelrotation=0, labelsize='large', direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_row][0].get_xlabel(), fontsize='xx-large'), 
            ylabel_args=dict(ylabel=axs[i_row][0].get_ylabel(), fontsize='xx-large')
        )
        #****************************************
        fig, axs[i_row][1] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_row][1], 
            data=ami_df_i_subset, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('15 minutes'), 
            plot_time_beg_end=[means_df.iloc[i_row]['zero_times_max'], means_df.iloc[i_row]['conservative_max']], 
            data_label='', 
            title_args=None, 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ),
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_row][1].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_row][1], 
            means_df.iloc[[i_row]], 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=mean_keys_to_include, 
            expand_ax_to_accommodate=False
        )
        axs[i_row][1].text(0.15, 0.9, f'#SNs = {n_SNs}', ha='center', va='center', transform=axs[i_row][1].transAxes, fontsize='xx-large')
        Plot_General.set_general_plotting_args(
            ax=axs[i_row][1], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize='large', direction='out'), 
                dict(axis='y', labelrotation=0, labelsize='large', direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_row][1].get_xlabel(), fontsize='xx-large'), 
            ylabel_args=dict(ylabel=axs[i_row][1].get_ylabel(), fontsize='xx-large')
        )
    #-------------------------
    #--------------------------------------------------
    # Add legend to first row
    patch_dovs_beg = Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS Beg.'
    )
    patch_dovs_end = Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS End'
    )
    #-----
    patch_ui_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='Beg. Uncertainty Interval'
    )
    patch_ui_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='End Uncertainty Interval'
    )
    #-----
    patch_best_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. Beg.'
    )
    patch_best_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. End'
    )
    #-------------------------
    if len(set(['conservative', 'zero_times']).difference(mean_keys_to_include))==0:
        handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end]
    else:
        handles=[patch_dovs_beg, patch_dovs_end, patch_best_beg, patch_best_end]
    #-------------------------
    leg_1 = axs[0][1].legend(
        title=None, 
        handles=handles, 
        bbox_to_anchor=(1, 1.025), 
        loc='upper left', 
        fontsize=15
    )                
    #-------------------------
    fig.suptitle(f"OUTG_REC_NB: {outg_rec_nb}", y=0.95, fontsize='xx-large')
    #-------------------------
    return fig, axs

In [39]:
def set_i_outg_helper(
    best_ests_df_i, 
    sort_cols=['winner_min', 'winner_max'], 
    i_outg_col='i_outg'
):
    r"""
    Intended only for use within set_i_outg_in_best_ests_df.
    """
    #-------------------------
    return_df=best_ests_df_i.sort_values(by=sort_cols)
    return_df[i_outg_col]=list(range(return_df.shape[0]))
    #-------------------------
    return return_df

def set_i_outg_in_best_ests_df(
    best_ests_df, 
    groupby_cols=['PN'], 
    sort_cols=['winner_min', 'winner_max'], 
    i_outg_col='i_outg'
):
    r"""
    """
    #-------------------------
    # Make sure all the needed columns are contained in best_ests_df
    assert(set(groupby_cols+sort_cols).difference(set(best_ests_df.columns.tolist()))==set())
    #-----
    assert(isinstance(groupby_cols, list))
    assert(isinstance(sort_cols, list))
    #-------------------------
    if i_outg_col not in best_ests_df.columns.tolist():
        best_ests_df[i_outg_col] = None
    #-------------------------
    return_df = best_ests_df.groupby(groupby_cols, as_index=False, group_keys=False).apply(
        lambda x: set_i_outg_helper(
            best_ests_df_i = x, 
            sort_cols      = sort_cols, 
            i_outg_col     = i_outg_col
        )
    )
    #-------------------------
    # Make sure all values set
    assert(return_df[i_outg_col].isna().sum()==0)
    #-------------------------
    return return_df

In [40]:
def get_potential_overlapping_dovs(
    PNs, 
    outg_t_beg, 
    outg_t_end, 
    dovs_sql_fcn=DOVSOutages_SQL.build_sql_std_outage, 
    addtnl_dovs_sql_kwargs=None
):
    r"""
    Find any DOVS events which potentially overlap with the premise numbers in PNs during
    the time period defined by outg_t_beg and outg_t_end.
    
    NOTE: outg_t_beg/_end can come from a DOVS events, or, e.g., from best_ests_df[t_min_col].min() and
          best_ests_df[t_max_col].max()
    -------------------------      
    To find potential DOVS outages overlapping with our findings, for the given PNs, collect all DOVS events whose 
      ending (dt_on_ts) is greater than (or equal to) our earliest estimate 
      and beginning (dt_off_ts_full) is less than (or equal to) our latest estimate
    It may be easier to think about the logic from the standpoint of which DOVS events to EXCLUDE:
      Any DOVS event with ending (dt_on_ts) less than our earliest estimate clearly finished after our window
        and therefore does not overlap
      Any DOVS event with beginning (dt_off_ts_full) greater than our latest estimate clearly began after our
       window and therefore does not overlap.
    -----
    ==>
        A DOVS event is considered overlapping if both of the following are True:
            - it begins before the present outage ends
                - i.e., outg_t_beg_i <= outg_t_end (or, dt_off_ts_full_i <= outg_t_end)
            - it ends after the present outage ends
                - i.e., out_t_end_i >= out_t_beg   (or, dt_on_ts >= out_t_beg)
    -------------------------            
    addtnl_dovs_sql_kwargs:
        Any additional arguments to use in build_sql_function_kwargs when building dovs_df
        NOTE: premise_nbs, field_to_split, dt_on_ts, and dt_off_ts_full are handled by the function and 
                should therefore NOT be included in addtnl_dovs_sql_kwargs
              These will be set as:
                - premise_nbs (from the PNs argument)
                - field_to_split (set to 'premise_nbs')
                - dt_on_ts (using the outg_t_beg argument)
                  -- See above if confused why dt_on_ts paired with outg_t_beg
                - dt_off_ts_full (using outg_t_end argument)
                  -- See above if confused why dt_off_ts_full paired with outg_t_end
    """
    #-------------------------
    assert(isinstance(PNs, list))
    #-------------------------
    dflt_dovs_sql_kwargs = dict(
        premise_nbs=PNs, 
        field_to_split='premise_nbs', 
        dt_on_ts=dict(
            value=outg_t_beg, 
            comparison_operator='>='
        ), 
        dt_off_ts_full=dict(
            value=outg_t_end, 
            comparison_operator='<='
        )        
    )
    #-----
    if addtnl_dovs_sql_kwargs is None:
        dovs_sql_kwargs = dflt_dovs_sql_kwargs
    else:
        assert(isinstance(addtnl_dovs_sql_kwargs, dict))
        #-----
        # Make sure none of the dovs_sql_kwargs handled by this function are included in addtnl_dovs_sql_kwargs
        assert(set(dflt_dovs_sql_kwargs.keys()).intersection(set(addtnl_dovs_sql_kwargs.keys()))==set())
        #-----
        dovs_sql_kwargs = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict    = dflt_dovs_sql_kwargs, 
            default_values_dict = addtnl_dovs_sql_kwargs, 
            extend_any_lists    = True, 
            inplace             = False
        )
    #-------------------------
    dovs = DOVSOutages(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args=None, 
        init_df_in_constructor=True,
        build_sql_function=dovs_sql_fcn, 
        build_sql_function_kwargs=dovs_sql_kwargs
    )
    dovs_df = dovs.df.copy()
    #-------------------------
    return dovs_df

In [41]:
def get_outgs_in_dovs_df_overlapping_interval(
    t_min, 
    t_max, 
    dovs_df, 
    outg_rec_nb_i, 
    outg_rec_nb_col='OUTG_REC_NB', 
    t_min_col='DT_OFF_TS_FULL', 
    t_max_col='DT_ON_TS'
):
    r"""
    Given t_min, t_max, and dovs_df, find any entries in dovs_df which overlap with the interval [t_min, t_max].
    
    t_min,t_max:
        Define the interval during which to look for overlapping DOVS events
    
    dovs_df:
        pd.DataFrame containing DOVS outages
        There MUST ONLY BE one single row per outg_rec_nb
        
    outg_rec_nb_i:
        If supplied, outg_rec_nb_i is excluded from any found overlapping DOVS events.
        In this case, it is assumed t_min,t_max and outg_rec_nb_i are related, and therefore one does not
          want to include outg_rec_nb_i in the final results, because, of course, it trivially overlaps with
          itself!
    """
    #-------------------------
    # Make sure needed columns are contained in dovs_df
    assert(set([outg_rec_nb_col, t_min_col, t_max_col]).difference(set(dovs_df.columns.tolist()))==set())
    #-----
    # There should only be a single row for each outg_rec_nb
    assert(dovs_df[outg_rec_nb_col].nunique()==dovs_df.shape[0])
    #-------------------------
    # Grab the interval
    interval_1 = pd.Interval(t_min, t_max)
    #-------------------------
    # For each row (i.e., for each outage), check whether overlaps with interval_1
    # Store that information in tmp_overlap_col
    tmp_overlap_col = Utilities.generate_random_string()
    dovs_df[tmp_overlap_col] = dovs_df.apply(
        lambda x: pd.Interval(x[t_min_col], x[t_max_col]).overlaps(interval_1), 
        axis=1
    )
    #-------------------------
    # Collect the outg_rec_nbs which overlap with interval_1
    overlap_outg_rec_nbs = dovs_df[dovs_df[tmp_overlap_col]==True][outg_rec_nb_col].unique().tolist()
    # If outg_rec_nb_i was supplied, then ensure it is excluded from overlap_outg_rec_nbs
    if outg_rec_nb_i is not None:
        overlap_outg_rec_nbs = [x for x in overlap_outg_rec_nbs if x!=outg_rec_nb_i]
    #-------------------------
    # Drop tmp_overlap_col
    dovs_df = dovs_df.drop(columns=[tmp_overlap_col])    
    #-------------------------
    return overlap_outg_rec_nbs


def get_outgs_in_dovs_df_overlapping_outg_rec_nb_i(
    outg_rec_nb_i, 
    dovs_df, 
    outg_rec_nb_col='OUTG_REC_NB', 
    t_min_col='DT_OFF_TS_FULL', 
    t_max_col='DT_ON_TS'
):
    r"""
    Given outg_rec_nb_i and dovs_df, find any entries in dovs_df which overlap with outg_rec_nb_i.
    If outg_rec_nb_i is contained in dovs_df, simply extract on/off times
    If outg_rec_nb_i is NOT contained in dovs_df, run SQL query to retrieve info
    
    dovs_df:
        pd.DataFrame containing DOVS outages
        There MUST ONLY BE one single row per outg_rec_nb
    """
    #-------------------------
    # Make sure needed columns are contained in dovs_df
    assert(set([outg_rec_nb_col, t_min_col, t_max_col]).difference(set(dovs_df.columns.tolist()))==set())
    #-----
    # There should only be a single row for each outg_rec_nb
    assert(dovs_df[outg_rec_nb_col].nunique()==dovs_df.shape[0])
    #-------------------------
    if outg_rec_nb_i in dovs_df[outg_rec_nb_col].unique().tolist():
        dovs_df_i = dovs_df[dovs_df[outg_rec_nb_col]==outg_rec_nb_i].copy()
    else:
        # Build dovs_df_i
        dovs_i = DOVSOutages(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args=None, 
            init_df_in_constructor=True,
            build_sql_function=DOVSOutages_SQL.build_sql_outage, 
            build_sql_function_kwargs=dict(
                outg_rec_nbs=[outg_rec_nb_i], 
            )
        )
        dovs_df_i = dovs_i.df.copy()
        #-------------------------
        if outg_rec_nb_col not in dovs_df_i.columns.tolist():
            assert('OUTG_REC_NB' in dovs_df_i.columns.tolist())
            dovs_df_i = dovs_df_i.rename(columns={'OUTG_REC_NB':outg_rec_nb_col})
        #-----
        if t_min_col not in dovs_df_i.columns.tolist():
            assert('DT_OFF_TS_FULL' in dovs_df_i.columns.tolist())
            dovs_df_i = dovs_df_i.rename(columns={'DT_OFF_TS_FULL':t_min_col})
        #-----
        if t_max_col not in dovs_df_i.columns.tolist():
            assert('DT_ON_TS' in dovs_df_i.columns.tolist())
            dovs_df_i = dovs_df_i.rename(columns={'DT_ON_TS':t_max_col})
        #-------------------------
    assert(dovs_df_i.shape[0]==1)
    #-------------------------
    overlap_outg_rec_nbs = get_outgs_in_dovs_df_overlapping_interval(
        t_min = dovs_df_i.iloc[0][t_min_col], 
        t_max = dovs_df_i.iloc[0][t_max_col], 
        dovs_df = dovs_df, 
        outg_rec_nb_i = outg_rec_nb_i, 
        outg_rec_nb_col = outg_rec_nb_col, 
        t_min_col = t_min_col, 
        t_max_col = t_max_col
    )   
    #-------------------------
    return overlap_outg_rec_nbs

In [42]:
def build_overlap_outgs_for_PNs_df_from_srs(
    overlap_outgs_for_PNs, 
    outg_rec_nb_i=None, 
    best_ests_df=None, 
    PN_col_best_ests='PN', 
    return_outg_rec_nb_in_idx=True
):
    r"""
    Both functions get_outgs_in_dovs_df_overlapping_interval_by_PN and get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN 
      use these methods, so makes sense to put in function
    """
    #-------------------------
    assert(isinstance(overlap_outgs_for_PNs, pd.Series))
    overlap_outgs_for_PNs_df = overlap_outgs_for_PNs.to_frame(name='overlap_outg_rec_nbs')
    overlap_outgs_for_PNs_df['n_overlap'] = overlap_outgs_for_PNs_df['overlap_outg_rec_nbs'].apply(len)
    #-------------------------
    if best_ests_df is not None:
        assert(
            isinstance(best_ests_df, pd.DataFrame) and 
            PN_col_best_ests in best_ests_df.columns.tolist()
        )
        #-----
        overlap_outgs_for_PNs_df['lost_power'] = False
        overlap_outgs_for_PNs_df.loc[
            overlap_outgs_for_PNs_df.index.isin(best_ests_df[PN_col_best_ests].unique().tolist()), 
            'lost_power'
        ] = True
        #-----
        # Re-order columns
        overlap_outgs_for_PNs_df=overlap_outgs_for_PNs_df[['lost_power', 'overlap_outg_rec_nbs', 'n_overlap']]
    #-------------------------
    if return_outg_rec_nb_in_idx and outg_rec_nb_i is not None:
        overlap_outgs_for_PNs_df = Utilities_df.prepend_level_to_MultiIndex(
            df=overlap_outgs_for_PNs_df, 
            level_val=outg_rec_nb_i, 
            level_name='OUTG_REC_NB', 
            axis=0
        )
    #-------------------------
    return overlap_outgs_for_PNs_df    

def get_outgs_in_dovs_df_overlapping_interval_by_PN(
    t_min, 
    t_max, 
    dovs_df,
    outg_rec_nb_i=None, 
    best_ests_df=None, 
    outg_rec_nb_col='OUTG_REC_NB', 
    PN_col='PREMISE_NB', 
    t_min_col='DT_OFF_TS_FULL', 
    t_max_col='DT_ON_TS', 
    PN_col_best_ests='PN', 
    return_outg_rec_nb_in_idx=True
):
    r"""
    Given t_min, t_max, and dovs_df, find any entries in dovs_df which overlap with the interval [t_min, t_max].
    
    t_min,t_max:
        Define the interval during which to look for overlapping DOVS events
    
    dovs_df:
        pd.DataFrame containing DOVS outages
        There MUST ONLY BE one single row per outg_rec_nb
        
    outg_rec_nb_i:
        If supplied, outg_rec_nb_i is excluded from any found overlapping DOVS events.
        In this case, it is assumed t_min,t_max and outg_rec_nb_i are related, and therefore one does not
          want to include outg_rec_nb_i in the final results, because, of course, it trivially overlaps with
          itself!
          
    best_ests_df:
        Premises which actually suffer from an outage are found in best_ests_df.
        Thus, if the user supplies best_ests_df, then best_ests_df[PN_col_best_ests] will be used to 
          fill the 'lost_power' column, which is boolean and denotes whether or not the premise lost power.
        If best_ests_df is not supplied, then the 'lost_power' field is not included in the output
    """
    #-------------------------
    #-------------------------
    overlap_outgs_for_PNs = dovs_df.groupby(PN_col, as_index=True).apply(
        lambda x: get_outgs_in_dovs_df_overlapping_interval(
            t_min=t_min, 
            t_max=t_max, 
            dovs_df=x, 
            outg_rec_nb_i=outg_rec_nb_i, 
            outg_rec_nb_col=outg_rec_nb_col, 
            t_min_col=t_min_col, 
            t_max_col=t_max_col
        )
    )
    #-------------------------
    overlap_outgs_for_PNs_df = build_overlap_outgs_for_PNs_df_from_srs(
        overlap_outgs_for_PNs=overlap_outgs_for_PNs, 
        outg_rec_nb_i=outg_rec_nb_i, 
        best_ests_df=best_ests_df, 
        PN_col_best_ests=PN_col_best_ests, 
        return_outg_rec_nb_in_idx=return_outg_rec_nb_in_idx
    )
    #-------------------------
    return overlap_outgs_for_PNs_df

def get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN(
    outg_rec_nb_i, 
    dovs_df, 
    best_ests_df=None, 
    outg_rec_nb_col='OUTG_REC_NB', 
    PN_col='PREMISE_NB', 
    t_min_col='DT_OFF_TS_FULL', 
    t_max_col='DT_ON_TS', 
    PN_col_best_ests='PN', 
    return_outg_rec_nb_in_idx=True
):
    r"""
    Given outg_rec_nb_i and dovs_df, find any entries in dovs_df which overlap with outg_rec_nb_i.
    
    best_ests_df:
        Premises which actually suffer from an outage are found in best_ests_df.
        Thus, if the user supplies best_ests_df, then best_ests_df[PN_col_best_ests] will be used to 
          fill the 'lost_power' column, which is boolean and denotes whether or not the premise lost power.
        If best_ests_df is not supplied, then the 'lost_power' field is not included in the output
    -----
    If outg_rec_nb_i is contained in dovs_df, simply extract on/off times
    If outg_rec_nb_i is NOT contained in dovs_df, run SQL query to retrieve info
    
    dovs_df:
        pd.DataFrame containing DOVS outages
        There MUST ONLY BE one single row per outg_rec_nb
    """
    #-------------------------
    overlap_outgs_for_PNs = dovs_df.groupby(PN_col, as_index=True).apply(
        lambda x: get_outgs_in_dovs_df_overlapping_outg_rec_nb_i(
            outg_rec_nb_i=outg_rec_nb_i, 
            dovs_df=x, 
            outg_rec_nb_col=outg_rec_nb_col, 
            t_min_col=t_min_col, 
            t_max_col=t_max_col
        )
    )
    #-------------------------
    overlap_outgs_for_PNs_df = build_overlap_outgs_for_PNs_df_from_srs(
        overlap_outgs_for_PNs=overlap_outgs_for_PNs, 
        outg_rec_nb_i=outg_rec_nb_i, 
        best_ests_df=best_ests_df, 
        PN_col_best_ests=PN_col_best_ests, 
        return_outg_rec_nb_in_idx=return_outg_rec_nb_in_idx
    )
    #-------------------------
    return overlap_outgs_for_PNs_df

In [43]:
def identify_dovs_overlaps_for_PN_i(
    best_ests_df_i, 
    dovs_df, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    PN_col_dovs='PREMISE_NB', 
    t_min_col_dovs='DT_OFF_TS_FULL', 
    t_max_col_dovs='DT_ON_TS', 
    outg_rec_nb_col_dovs = 'OUTG_REC_NB', 
    overlap_outg_col='overlap_DOVS', 
    overlap_times_col='overlap_times', 
    keep_col='keep'
):
    r"""
    Built with intention of being used in a .groupby(...).apply(lambda x: ) setting.
    So, be careful if using elsewhere.
    NOTE: dovs_df MUST NOT CONTAIN the outg_rec_nb for the data contained in best_ests_df_i.
            IF IT DOES, then all entries will be identified, because there will always be an overlap!
    -----
    This is a bit different from the previous functions (e.g., get_outgs_in_dovs_df_overlapping_interval, 
       get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN. etc.)
    Those are trying to find any other DOVS events whose posted beginning/ending time overlaps with the given
       interval/current outages's beg,end times.

    Here, we are dealing with the identified sub-outages found for each premise number.
    For each premise number:
      Determine if any of the identified sub-outages overlap with any of the potentially overlapping DOVS events.
      If no DOVS event overlaps with a given sub-outage, the sub-outage is kept.
      Which sub-outages to keep is tracked in the keep_srs_i series, which is updated at each iteration over
        the potential conflicting outages through keep_boolean_ij.
      The DOVS events overlapping the sub-outages are tracked in the overlap_df_i series
    Note:  At the end, keep_srs_i should equal overlap_df_i[overlap_outg_col].apply(lambda x: len(x)==0)
    """
    #-------------------------
    # Designed for single PN
    assert(best_ests_df_i[PN_col].nunique()==1)
    PN_i = best_ests_df_i[PN_col].unique().tolist()[0]
    #-------------------------
    # Make sure overlap_outg_col/overlap_times_col/keep_col aren't already contained in best_ests_df_i
    if overlap_outg_col in best_ests_df_i.columns.tolist():
        overlap_outg_col = overlap_outg_col + f'_{Utilities.generate_random_string(str_len=4)}'
        assert(overlap_outg_col not in best_ests_df_i.columns.tolist())
    #-----
    if overlap_times_col in best_ests_df_i.columns.tolist():
        overlap_times_col = overlap_times_col + f'_{Utilities.generate_random_string(str_len=4)}'
        assert(overlap_times_col not in best_ests_df_i.columns.tolist())
    #-----
    if keep_col in best_ests_df_i.columns.tolist():
        keep_col = keep_col + f'_{Utilities.generate_random_string(str_len=4)}'
        assert(keep_col not in best_ests_df_i.columns.tolist())
    #-------------------------
    # Grab relevant entries from dovs_df
    dovs_df_i = dovs_df[dovs_df[PN_col_dovs]==PN_i]
    
    # If no relevant entires, return
    if dovs_df_i.shape[0]==0:
        return_df = best_ests_df_i.copy()
        return_df[overlap_outg_col]  = [[] for _ in range(return_df.shape[0])]
        return_df[overlap_times_col] = [[] for _ in range(return_df.shape[0])]
        return_df[keep_col]          = True
        return return_df

    # Initiate the boolean slicing series
    keep_srs_i    = pd.Series(data=True, index=best_ests_df_i.index)
    overlap_df_i = pd.DataFrame(
        data={
            overlap_outg_col  : [[] for _ in range(best_ests_df_i.shape[0])], 
            overlap_times_col : [[] for _ in range(best_ests_df_i.shape[0])]
        }, 
        index=best_ests_df_i.index
    )

    # Iterate through (possibly) conflicting outages, finding the slicing boolean (keep_boolean_ij) for each
    #   and updating the overall slicing boolean (keep_srs_i)
    # In general, slicing booleans contain information regarding which sub-outages should be kept for this
    #   specific premise number (i.e., each row represents a sub-outage).
    for idx_j, cnflct_outg_j in dovs_df_i.iterrows():
        # I guess it's actually easier to essentially find those which overlap and then take the logical opposite.
        # To find those which don't overlap directly would need two separate and statements, I believe, whereas
        #   this only needs one
        keep_boolean_ij = ~(
            (best_ests_df_i[t_min_col]<=cnflct_outg_j[t_max_col_dovs]) & 
            (best_ests_df_i[t_max_col]>=cnflct_outg_j[t_min_col_dovs])
        )
        assert(all(keep_srs_i.index==keep_boolean_ij.index)) # Sanity, not really necessary...
        #----------
        # If cnflct_outg_j overlaps with any of the entries of best_ests_df_i, add the overlapping outg_rec_nb
        #   to the appropriate list in overlap_df_i
        # The overlap information is simply ~keep_boolean_ij (because we only keep if non-overlapping)
        overlap_df_i.loc[~keep_boolean_ij, overlap_outg_col] = overlap_df_i.loc[~keep_boolean_ij, overlap_outg_col].apply(
            lambda x: x+[cnflct_outg_j[outg_rec_nb_col_dovs]]
        )
        #-----
        overlap_df_i.loc[~keep_boolean_ij, overlap_times_col] = overlap_df_i.loc[~keep_boolean_ij, overlap_times_col].apply(
            lambda x: x+[(cnflct_outg_j[t_min_col_dovs], cnflct_outg_j[t_max_col_dovs])]
        )
        #----------
        keep_srs_i = keep_srs_i & keep_boolean_ij
        
    #-------------------------
    # Sanity check
    assert(overlap_df_i[overlap_outg_col].apply(lambda x: len(x)==0).equals(keep_srs_i))
    assert((overlap_df_i.index==keep_srs_i.index).all())
    #-------------------------
    df_to_merge = pd.merge(
        overlap_df_i, 
        keep_srs_i.to_frame(name=keep_col), 
        left_index=True, 
        right_index=True, 
        how='inner'
    )
    #-------------------------
    assert(set(best_ests_df_i.index).symmetric_difference(set(df_to_merge.index))==set())
    assert(set(best_ests_df_i.columns).intersection(set(df_to_merge.columns))==set())
    #-----
    return_df = pd.merge(
        best_ests_df_i, 
        df_to_merge, 
        left_index=True, 
        right_index=True, 
        how='inner'
    )
    #-------------------------        
    return return_df

In [44]:
def identify_dovs_overlaps_from_best_ests(
    best_ests_df, 
    outg_rec_nb, 
    dovs_df, 
    get_ptntl_ovrlp_dovs_kwargs=None, 
    assert_no_overlaps=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    PN_col_dovs='PREMISE_NB', 
    t_min_col_dovs='DT_OFF_TS_FULL', 
    t_max_col_dovs='DT_ON_TS', 
    outg_rec_nb_col_dovs='OUTG_REC_NB', 
    overlap_outg_col='overlap_DOVS', 
    overlap_times_col='overlap_times', 
    keep_col='keep'
):
    r"""
    If dovs_df is not supplied by user, it will built.
        Default behavior is to query using the premise number and time restrictions from best_ests_df.
        However, one could override this behavior by supplying different arguments for get_ptntl_ovrlp_dovs_kwargs.
        
    If dovs_df is supplied by the user:
        - it must include outg_rec_nb
        - it must contain a 'PREMISE_NB' column
        
    get_ptntl_ovrlp_dovs_kwargs:
        A dictionary with key/value pairs suitable for input into get_potential_overlapping_dovs.
        The keys and default values are:
            - PNs: 
                -- premise numbers to query
                -- default = best_ests_df[PN_col].unique().tolist()
            - outg_t_beg: 
                -- beginning time to look for overlaps
                -- default = best_ests_df[t_min_col].min()
            - outg_t_end: 
                -- ending time to look for overlaps
                -- default = best_ests_df[t_max_col].max()
            - dovs_sql_fcn: 
                -- SQL functin to use for query 
                -- default = DOVSOutages_SQL.build_sql_std_outage
            - addtnl_dovs_sql_kwargs: 
                -- Any additional kwargs to input into dovs_sql_fcn when running query
                -- default = None
                -- NOTE: premise_nbs, field_to_split, dt_on_ts, and dt_off_ts_full are handled by 
                         get_potential_overlapping_dovs and should therefore NOT be included in 
                         addtnl_dovs_sql_kwargs
    
    NOTE: Using build_sql_std_outage by default, so typical restrictions on outages types etc.
    """
    #--------------------------------------------------
    # Build dovs, if needed
    #-----
    # As mentioned above, the functionality here allows the user to override the default behavior by supplying
    #   different arguments for get_ptntl_ovrlp_dovs_kwargs.
    #-------------------------
    if dovs_df is None:
        dflt_get_ptntl_ovrlp_dovs_kwargs = dict(
            PNs                    = best_ests_df[PN_col].unique().tolist(), 
            outg_t_beg             = best_ests_df[t_min_col].min(), 
            outg_t_end             = best_ests_df[t_max_col].max(), 
            dovs_sql_fcn           = DOVSOutages_SQL.build_sql_std_outage, 
            addtnl_dovs_sql_kwargs = dict(include_DOVS_PREMISE_DIM=True)
        )
        #-----
        if get_ptntl_ovrlp_dovs_kwargs is None:
            get_ptntl_ovrlp_dovs_kwargs = dflt_get_ptntl_ovrlp_dovs_kwargs
        else:
            get_ptntl_ovrlp_dovs_kwargs = Utilities.supplement_dict_with_default_values(
                to_supplmnt_dict    = get_ptntl_ovrlp_dovs_kwargs, 
                default_values_dict = dflt_get_ptntl_ovrlp_dovs_kwargs, 
                extend_any_lists    = False, 
                inplace             = False
            )
        #-----
        dovs_df = get_potential_overlapping_dovs(**get_ptntl_ovrlp_dovs_kwargs)
        #-------------------------
        if outg_rec_nb_col_dovs not in dovs_df.columns.tolist():
            assert('OUTG_REC_NB' in dovs_df.columns.tolist())
            dovs_df = dovs_df.rename(columns={'OUTG_REC_NB':outg_rec_nb_col_dovs})
        #-----
        if t_min_col_dovs not in dovs_df.columns.tolist():
            assert('DT_OFF_TS_FULL' in dovs_df.columns.tolist())
            dovs_df = dovs_df.rename(columns={'DT_OFF_TS_FULL':t_min_col_dovs})
        #-----
        if t_max_col_dovs not in dovs_df.columns.tolist():
            assert('DT_ON_TS' in dovs_df.columns.tolist())
            dovs_df = dovs_df.rename(columns={'DT_ON_TS':t_max_col_dovs})
    #--------------------------------------------------
    # Mainly sanity checks....
    #-----
    nec_dovs_cols = [outg_rec_nb_col_dovs, PN_col_dovs, t_min_col_dovs, t_max_col_dovs]
    assert(set(nec_dovs_cols).difference(set(dovs_df.columns.tolist()))==set())
    #-----
    outg_rec_nbs_and_times = dovs_df[
        [outg_rec_nb_col_dovs, t_min_col_dovs, t_max_col_dovs]
    ].drop_duplicates().set_index(outg_rec_nb_col_dovs)
    #-----
    # Current outg_rec_nb should definitely be included!
    assert(outg_rec_nb in outg_rec_nbs_and_times.index.tolist())
    #-----
    # Should only be one entry per outg_rec_nb
    assert(outg_rec_nbs_and_times.index.nunique()==outg_rec_nbs_and_times.shape[0])
    #-------------------------
    # For a given PN, there definitely should not be any overlap between two DOVS OUTG_REC_NBs
    #   i.e., a given PN cannot belong to two outages at the same time!
    # ==========
    # OLD METHOD:
    #     Test this by checking whether Utilities.get_overlap_intervals returns an object with
    #       the same length as the input for each PN (if there were any overlaps, the returned
    #       object would have a shorter length)
    # REASON FOR CHANGE:
    #     This is actually a little bit too strict.
    #     For a given PN, if potentially overlapping DOVS outages overlap with each other but do not
    #       overlap with outg_rec_nb, they do not affect the processing of this outage, and therefore
    #       are not of issue.
    #     The old method was stricter, whereas the new method is more relaxed and matches the description above
    #-----
    # PNs_w_mult_outgs_smltnsly: multiple outages simultaneously
#     PNs_w_mult_outgs_smltnsly = dovs_df.groupby(PN_col_dovs, as_index=True, group_keys=False).apply(
#         lambda x: len(Utilities.get_overlap_intervals(x[[t_min_col_dovs, t_max_col_dovs]].values))!=x.shape[0]
#     )
#     if assert_no_overlaps:
#         assert(not PNs_w_mult_outgs_smltnsly.any())
    # END: OLD METHOD
    # ==========
    overlap_outgs_for_PNs_df = get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN(
        outg_rec_nb_i    = outg_rec_nb, 
        dovs_df          = dovs_df, 
        best_ests_df     = best_ests_df, 
        outg_rec_nb_col  = outg_rec_nb_col_dovs, 
        PN_col           = PN_col_dovs, 
        t_min_col        = t_min_col_dovs, 
        t_max_col        = t_max_col_dovs, 
        PN_col_best_ests = PN_col
    )
    if assert_no_overlaps:
        assert((overlap_outgs_for_PNs_df['n_overlap']>0).sum()==0)
    #--------------------------------------------------
    # Do not need current outg_rec_nb anymore, only the others
    dovs_df = dovs_df[dovs_df[outg_rec_nb_col_dovs] != outg_rec_nb].copy()
    #--------------------------------------------------
    # Simple-minded way would be to exclude any entries in best_ests_df which overlap 
    #   with the outage times in outg_rec_nbs_and_times.
    #   This would be correct in most all cases
    # HOWEVER, to be completely correct, this should be done at the PREMISE_NB level!
    #-----
    # I don't want to simply merge best_ests_df with dovs_df (by PN) because the former can have multiple sub-outages
    #   per PN, and the latter can have multiple (possibly) conflicting outages.
    # Merging the two would be sloppy.
    # Therefore, I'll use groupby
    return_df = best_ests_df.groupby(PN_col, as_index=False, group_keys=False).apply(
        lambda x: identify_dovs_overlaps_for_PN_i(
            best_ests_df_i       = x, 
            dovs_df              = dovs_df, 
            PN_col               = PN_col, 
            t_min_col            = t_min_col, 
            t_max_col            = t_max_col, 
            PN_col_dovs          = PN_col_dovs, 
            t_min_col_dovs       = t_min_col_dovs, 
            t_max_col_dovs       = t_max_col_dovs, 
            outg_rec_nb_col_dovs = outg_rec_nb_col_dovs, 
            overlap_outg_col     = overlap_outg_col, 
            overlap_times_col    = overlap_times_col, 
            keep_col             = keep_col
        )
    )
    #--------------------------------------------------
    return return_df

In [45]:
def remove_any_dovs_overlaps_from_best_ests(
    best_ests_df, 
    outg_rec_nb, 
    dovs_df, 
    get_ptntl_ovrlp_dovs_kwargs=None, 
    assert_no_overlaps=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_dovs='PREMISE_NB', 
    t_min_col_dovs='DT_OFF_TS_FULL', 
    t_max_col_dovs='DT_ON_TS', 
    outg_rec_nb_col_dovs='OUTG_REC_NB', 
    overlap_outg_col='overlap_DOVS', 
    overlap_times_col='overlap_times', 
    keep_col='keep'
):
    r"""
    If dovs_df is not supplied by user, it will built.
        Default behavior is to query using the premise number and time restrictions from best_ests_df.
        However, one could override this behavior by supplying different arguments for get_ptntl_ovrlp_dovs_kwargs.
        
    If dovs_df is supplied by the user:
        - it must include outg_rec_nb
        - it must contain a 'PREMISE_NB' column
        
    get_ptntl_ovrlp_dovs_kwargs:
        A dictionary with key/value pairs suitable for input into get_potential_overlapping_dovs.
        The keys and default values are:
            - PNs: 
                -- premise numbers to query
                -- default = best_ests_df[PN_col].unique().tolist()
            - outg_t_beg: 
                -- beginning time to look for overlaps
                -- default = best_ests_df[t_min_col].min()
            - outg_t_end: 
                -- ending time to look for overlaps
                -- default = best_ests_df[t_max_col].max()
            - dovs_sql_fcn: 
                -- SQL functin to use for query 
                -- default = DOVSOutages_SQL.build_sql_std_outage
            - addtnl_dovs_sql_kwargs: 
                -- Any additional kwargs to input into dovs_sql_fcn when running query
                -- default = None
                -- NOTE: premise_nbs, field_to_split, dt_on_ts, and dt_off_ts_full are handled by 
                         get_potential_overlapping_dovs and should therefore NOT be included in 
                         addtnl_dovs_sql_kwargs
    
    NOTE: Using build_sql_std_outage by default, so typical restrictions on outages types etc.
    """
    #--------------------------------------------------
    return_df = identify_dovs_overlaps_from_best_ests(
        best_ests_df                = best_ests_df, 
        outg_rec_nb                 = outg_rec_nb, 
        dovs_df                     = dovs_df, 
        get_ptntl_ovrlp_dovs_kwargs = get_ptntl_ovrlp_dovs_kwargs, 
        assert_no_overlaps          = assert_no_overlaps, 
        PN_col                      = PN_col, 
        t_min_col                   = t_min_col, 
        t_max_col                   = t_max_col, 
        PN_col_dovs                 = PN_col_dovs, 
        t_min_col_dovs              = t_min_col_dovs, 
        t_max_col_dovs              = t_max_col_dovs, 
        outg_rec_nb_col_dovs        = outg_rec_nb_col_dovs, 
        overlap_outg_col            = overlap_outg_col, 
        overlap_times_col           = overlap_times_col, 
        keep_col                    = keep_col
    )
    #--------------------------------------------------
    # Locate overlap_outg_col, overlap_times_col, and keep_col in return_df
    #-----
    found_overlap_outg_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(overlap_outg_col)
    )
    assert(len(found_overlap_outg_col)==1)
    overlap_outg_col = found_overlap_outg_col[0]
    #-----
    found_overlap_times_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(overlap_times_col)
    )
    assert(len(found_overlap_times_col)==1)
    overlap_times_col = found_overlap_times_col[0]
    #-----
    found_keep_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(keep_col)
    )
    assert(len(found_keep_col)==1)
    keep_col = found_keep_col[0]
    #--------------------------------------------------
    # Keep only those with keep_col values equal to True
    return_df = return_df[return_df[keep_col]==True].copy()
    #-----
    # No longer need overlap_outg_col, overlap_times_col, or keep_col, as they are trivially equal to [],[],True for all
    return_df = return_df.drop(columns=[overlap_outg_col, overlap_times_col, keep_col])
    #--------------------------------------------------
    # Re-build i_outg_col to account for any sub-outages removed
    return_df = set_i_outg_in_best_ests_df(
        best_ests_df = return_df, 
        groupby_cols = [PN_col], 
        sort_cols    = [t_min_col, t_max_col], 
        i_outg_col   = i_outg_col
    )
    #--------------------------------------------------
    return return_df

In [48]:
def expand_removed_srs(
    removed_srs, 
    ami_df_i, 
    best_ests_df, 
    PN_col_ami='aep_premise_nb', 
    time_col_ami='starttimeperiod_local', 
    PN_col_be='PN', 
    t_min_col_be='winner_min', 
    t_max_col_be='winner_max', 
    keep_col_be='keep', 
):
    r"""
    Expand the bounds of removed_srs in set_removed_due_to_overlap_in_ami_df_i.
    -----
    The point of building the 'removed_due_to_overlap' in ami_df_i is simply for plotting purposes.
    If there is a block in the middle of ami_df_i which is marked 'removed_due_to_overlap'=True, and the data with those
      values removed is plotted, the figure will still appear as if there are data points for the removed times.
    The reason being that markers are not drawn, so it is not obvious where datapoints are.
    The value to the left of the points to the removed will be connected with a straight line to those to the right of 
      the points to be removed.
    -----
    The purpose of this function is to make the plot a little clearer.
    If a removal period happens before the first sub-outage for a PN, expand the left point of the removal period to be
      equal to the minimum time value in ami_df_i for the PN.
    If a removal period happens after the last sub-outage for a PN, expand the right point of the removal period to be
      equal to the maximum time value in ami_df_i for the PN.
    """
    #-------------------------
    winner_minmax_by_PN = best_ests_df[best_ests_df[keep_col_be]==True].groupby(PN_col_be).agg(
        {t_min_col_be:'min', t_max_col_be:'max'}
    )
    #-----
    ami_minmax_times_by_PN = ami_df_i.groupby(PN_col_ami).agg(
        {time_col_ami:['min', 'max']}
    )
    assert(set(removed_srs.index).difference(set(ami_minmax_times_by_PN.index))==set())
    #-------------------------
    # Iterate through PNs in removed_srs and build expanded values
    return_srs = dict()
    for PN_j, overlap_times_j in removed_srs.items():
        # If PN_j did not actually lose power, there will be no entry in winner_minmax_by_PN
        # In such a case, leave values as they are
        if PN_j not in winner_minmax_by_PN.index:
            overlap_times_j_exp = overlap_times_j
        else:
            overlap_times_j_exp = []
            for overlap_pd_j_k in overlap_times_j:
                # If a removal period happens before the first sub-outage for a PN, expand the left point of 
                #   the removal period to be equal to the minimum time value in ami_df_i for the PN.
                if overlap_pd_j_k[1] < winner_minmax_by_PN.loc[PN_j, t_min_col_be]:
                    overlap_pd_j_k_0 = ami_minmax_times_by_PN.loc[PN_j, (time_col_ami, 'min')]
                else:
                    overlap_pd_j_k_0 = overlap_pd_j_k[0]
                #-----
                # If a removal period happens after the last sub-outage for a PN, expand the right point of
                #   the removal period to be equal to the maximum time value in ami_df_i for the PN.        
                if overlap_pd_j_k[0] > winner_minmax_by_PN.loc[PN_j, t_max_col_be]:
                    overlap_pd_j_k_1 = ami_minmax_times_by_PN.loc[PN_j, (time_col_ami, 'max')]
                else:
                    overlap_pd_j_k_1 = overlap_pd_j_k[1]
                #-----
                overlap_times_j_exp.append((overlap_pd_j_k_0, overlap_pd_j_k_1)) 
        assert(PN_j not in return_srs.keys())
        return_srs[PN_j] = overlap_times_j_exp
    #-------------------------
    return_srs = pd.Series(return_srs, name=removed_srs.name)
    return return_srs


def set_removed_due_to_overlap_helper(
    ami_df_i, 
    PN_j, 
    overlap_times_j, 
    time_col='starttimeperiod_local', 
    PN_col='aep_premise_nb', 
    removed_due_to_overlap_col='removed_due_to_overlap'
):
    r"""
    ONLY INTENDED FOR USE INSIDE OF set_removed_due_to_overlap_in_ami_df_i
    
    In order to set removed_due_to_overlap, the logic will be: set if PN is correct and time falls within
      any one of the overlap_times taken from best_ests_df
    In code, this amounts to:
        ami_df_i[
            (ami_df_i[PN_col]==PN_j) & 
            (    
                (        
                    (ami_df_i[time_col] >= overlap_time_min_j_0) &
                    (ami_df_i[time_col] <= overlap_time_max_j_0)
                )
                ||
                .
                .
                .
                (        
                    (ami_df_i[time_col] >= overlap_time_min_j_n) &
                    (ami_df_i[time_col] <= overlap_time_max_j_n)
                )    
            )
        ]
    Due to the unknown number of overlap times, this is easiest to accomplish using DFSlicers    
    """
    #-------------------------
    # Make sure all elements in overlap_times_j are lists/tuples of length 2
    assert(Utilities.are_all_list_elements_one_of_types(overlap_times_j, [list, tuple]))
    assert(Utilities.are_list_elements_lengths_homogeneous(overlap_times_j, 2))
    #-------------------------
    pn_slicer = slicer = DFSlicer(
        single_slicers = [dict(column=PN_col, value=PN_j, comparison_operator='==')]
    )
    #-----
    pn_slcr_bool_srs = pn_slicer.get_slicing_booleans(
        df=ami_df_i
    )
    #-------------------------
    time_slicers = []
    for overlap_time_min, overlap_time_max in overlap_times_j:
        ts_i = DFSlicer(
            single_slicers = [
                dict(column=time_col, value=overlap_time_min, comparison_operator='>='), 
                dict(column=time_col, value=overlap_time_max, comparison_operator='<=')
            ], 
            join_single_slicers='and'
        )
        time_slicers.append(ts_i)
    #-----
    time_slcr_bool_srs = DFSlicer.combine_slicers_and_get_slicing_booleans(
        df=ami_df_i, 
        slicers=time_slicers, 
        join_slicers='or', 
        apply_not=False
    )
    #-------------------------
    assert((time_slcr_bool_srs.index==pn_slcr_bool_srs.index).all())
    slcr_bool_srs = time_slcr_bool_srs & pn_slcr_bool_srs
    #-------------------------
    ami_df_i.loc[slcr_bool_srs, removed_due_to_overlap_col] = True
    #-------------------------
    return ami_df_i

def set_removed_due_to_overlap_in_ami_df_i(
    ami_df_i, 
    best_ests_df, 
    PN_col='aep_premise_nb', 
    time_idfr='starttimeperiod_local', 
    PN_col_be='PN', 
    t_min_col_be='winner_min', 
    t_max_col_be='winner_max', 
    keep_col_be='keep', 
    overlap_times_col_be='overlap_times', 
    removed_due_to_overlap_col='removed_due_to_overlap', 
    expand_removed_times=True
):
    r"""
    In order to set removed_due_to_overlap, the logic will be: set if PN is correct and time falls within
      any one of the overlap_times taken from best_ests_df
    In code, this amounts to:
        ami_df_i[
            (ami_df_i['aep_premise_nb']==PN_ij) & 
            (    
                (        
                    (ami_df_i[time_col] >= overlap_time_min_0) &
                    (ami_df_i[time_col] <= overlap_time_max_0)
                )
                ||
                .
                .
                .
                (        
                    (ami_df_i[time_col] >= overlap_time_min_n) &
                    (ami_df_i[time_col] <= overlap_time_max_n)
                )    
            )
        ]
    Due to the unknown number of overlap times, this is easiest to accomplish using DFSlicers    
    """
    #--------------------------------------------------
    # In order for method to work, time information must be in column, not index.
    # If found in index, call reset_index
    time_idfr_loc = Utilities_df.get_idfr_loc(
        df=ami_df_i, 
        idfr=time_idfr
    )
    #-------------------------
    if time_idfr_loc[1]:
        # For now, make sure ami_df_i has a single index (if MultiIndex, this functionality can be built out later)
        # time information is located in index, so reset_index must be called for methods to work
        # Looking back at final code: I'm not sure anything additional needs to be done for MultiIndex case
        assert(ami_df_i.index.nlevels==1)
        #-----
        if ami_df_i.index.names[time_idfr_loc[0]]:
            time_col = ami_df_i.index.names[time_idfr_loc[0]]
        else:
            time_col = 'time_idx_'+Utilities.generate_random_string(str_len=4)
            assert(time_col not in ami_df_i.columns.tolist())
            assert(time_col not in list(ami_df_i.index.names))
            ami_df_i.index = ami_df_i.index.set_names(time_col, level=time_idfr_loc[0])
        #-----
        idx_cols = list(ami_df_i.index.names)
        ami_df_i = ami_df_i.reset_index()
    else:
        time_col = time_idfr_loc[0]
        idx_cols=None
    #-------------------------
    assert(set([PN_col, time_col]).difference(set(ami_df_i.columns.tolist()))==set())
    #--------------------------------------------------
    # From best_ests_df, find all sub-outages to be removed by PN
    assert(set([PN_col_be, keep_col_be, overlap_times_col_be]).difference(set(best_ests_df.columns.tolist()))==set())
    removed_srs = best_ests_df[best_ests_df[keep_col_be]==False].copy()
    removed_srs = removed_srs.groupby(PN_col_be)[overlap_times_col_be].sum()
    if expand_removed_times:
        removed_srs = expand_removed_srs(
            removed_srs  = removed_srs, 
            ami_df_i     = ami_df_i, 
            best_ests_df = best_ests_df, 
            PN_col_ami   = PN_col, 
            time_col_ami = time_col, 
            PN_col_be    = PN_col_be, 
            t_min_col_be = t_min_col_be, 
            t_max_col_be = t_max_col_be, 
            keep_col_be  = keep_col_be 
        )
    #--------------------------------------------------
    # Iterate through each PN in removed_srs and use set_removed_due_to_overlap_helper to set the appropriate
    #   values in ami_df_i
    ami_df_i[removed_due_to_overlap_col] = False
    for PN_j, overlap_times_j in removed_srs.items():
        ami_df_i = set_removed_due_to_overlap_helper(
            ami_df_i                   = ami_df_i, 
            PN_j                       = PN_j, 
            overlap_times_j            = overlap_times_j, 
            time_col                   = time_col, 
            PN_col                     = PN_col, 
            removed_due_to_overlap_col = removed_due_to_overlap_col
        )
    #--------------------------------------------------
    # If .reset_index was called earlier, set index back to original values
    if idx_cols is not None:
        ami_df_i = ami_df_i.set_index(idx_cols)
    #--------------------------------------------------
    return ami_df_i

In [47]:
def build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
):
    r"""
    ami_df_i
        - Only needed for finding n_PNs_tot
    """
    #-------------------------
    df = best_ests_df[[PN_col, i_outg_col, t_min_col, t_max_col]].copy()
    #-------------------------
    all_times_sorted = natsorted(set(df[t_min_col].tolist()+df[t_max_col].tolist()))
    all_times_sorted = [all_times_sorted[0]-pd.Timedelta('1T')] + all_times_sorted
    #-----
    # For each unique time, determine how many PNs without power
    n_PNs_tot = ami_df_i[PN_col_ami_df].nunique()
    time_and_n_PNs_w_power=dict()
    for time_i in all_times_sorted:
        df_i = df[
            (df[t_min_col]<=time_i) & 
            (df[t_max_col]>time_i)
        ].copy()
        # Should find at most one outage for a given PN!
        assert(df_i.shape[0]==df_i[PN_col].nunique())
        n_PNs_out_i = df_i.shape[0]
        assert(time_i not in time_and_n_PNs_w_power.keys())
        if return_pct:
            time_and_n_PNs_w_power[time_i] = 100*(n_PNs_tot-n_PNs_out_i)/n_PNs_tot
        else:
            time_and_n_PNs_w_power[time_i] = n_PNs_tot-n_PNs_out_i
    #-------------------------
    n_PNs_w_power_srs = pd.Series(data=time_and_n_PNs_w_power)
    #-------------------------
    return n_PNs_w_power_srs

def simplify_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    freq='1T'
):
    r"""
    If there are a bunch of entries close together, this will keep only the one with the largest value.
    Intention: To be used when including data point information in text on plot for n_PNs_w_power_srs
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    if n_PNs_w_power_srs.name:
        name = n_PNs_w_power_srs.name
    else:
        name = 'pct_w_power'
    #-------------------------
    n_PNs_w_power_srs_simp = n_PNs_w_power_srs.to_frame(name=name).reset_index().groupby(
        pd.Grouper(freq=freq, key='index')
    ).apply(
        lambda x: x.loc[x[name].idxmax()] if x.shape[0]>0 else None
    )
    n_PNs_w_power_srs_simp = n_PNs_w_power_srs_simp.dropna()
    n_PNs_w_power_srs_simp=n_PNs_w_power_srs_simp.set_index('index').squeeze()
    #-------------------------
    return n_PNs_w_power_srs_simp


def get_periods_above_threshold(
    n_PNs_w_power_srs, 
    threshold, 
    return_indices=False
):
    r"""
    Given n_PNs_w_power_srs, returns the beginning and ending of periods during which the number
    of premises with power is above threshold
    
    n_PNs_w_power_srs:
        A series with index equal to timestamps and values equal to the percent of premises with power
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    n_PNs_w_power_srs = n_PNs_w_power_srs.copy()
    if n_PNs_w_power_srs.name:
        pct_w_power_col = n_PNs_w_power_srs.name
    else:
        pct_w_power_col = Utilities.generate_random_string()
        n_PNs_w_power_srs.name = pct_w_power_col
    #-------------------------
    above_thresh_col = 'above_thresh'
    above_thresh_df = (n_PNs_w_power_srs>threshold).astype(int).to_frame()
    above_thresh_df=above_thresh_df.rename(columns={pct_w_power_col:above_thresh_col})
    #-----
    tmp_diff_col = Utilities.generate_random_string()
    above_thresh_df[tmp_diff_col] = above_thresh_df[above_thresh_col].diff()
    #-------------------------
    # The .diff() operation always leaves the first element as a NaN
    # However, if the first element pct_w_power>threshold it represents the beginning
    #   of a black and the diff should be +1
    if above_thresh_df.iloc[0][above_thresh_col]==1:
        above_thresh_df.loc[above_thresh_df.index[0], tmp_diff_col] = 1
    #-------------------------
    # Continuous blocks of True (i.e., blocks with pct_w_power > threshold) begin with diff = +1 and
    #   end at the element preceding diff = -1
    block_beg_idxs = above_thresh_df.reset_index().index[above_thresh_df[tmp_diff_col]==1].tolist()
    #-----
    block_end_idxs = above_thresh_df.reset_index().index[above_thresh_df[tmp_diff_col]==-1].tolist()
    block_end_idxs = [x-1 for x in block_end_idxs]
    # If power is ongoing at the end of the data, the procedure above will miss the last end idx
    #   If single point above threshold at end of data ==> tmp_diff_col = +1
    #   If multiple points above threshold at the end of the data, then tmp_diff_col for the last value will be 0
    #     In this case, there does not exist a tmp_diff_col=-1 to signal the end of the block, so must add by hand
    if above_thresh_df.iloc[-1][above_thresh_col]==1:
        block_end_idxs.append(above_thresh_df.shape[0]-1)
    #--------------------------------------------------
    # periods of length one should have idx in both block_beg_idxs and block_end_idxs
    len_1_pd_idxs = natsorted(set(block_beg_idxs).intersection(set(block_end_idxs)))
    #-------------------------
    # Remove the len_1 idxs so the remaineders can be matched
    # NOTE: The following procedure relies on block_beg(end)_idxs being sorting
    block_beg_idxs = natsorted(set(block_beg_idxs).difference(len_1_pd_idxs))
    block_end_idxs = natsorted(set(block_end_idxs).difference(len_1_pd_idxs))
    assert(len(block_beg_idxs)==len(block_end_idxs))
    block_begend_idxs = list(zip(block_beg_idxs, block_end_idxs))
    #-------------------------
    # Include the length 1 blocks!
    block_begend_idxs.extend([(x,x) for x in len_1_pd_idxs])
    #-------------------------
    # Sort
    block_begend_idxs = natsorted(block_begend_idxs, key=lambda x: x[0])
    #-------------------------
    # Sanity check!
    for i in range(len(block_begend_idxs)):
        assert(len(block_begend_idxs[i])==2)
        assert(block_begend_idxs[i][1]>=block_begend_idxs[i][0])
        if i>0:
            assert(block_begend_idxs[i][0]>block_begend_idxs[i-1][1])
    #-------------------------
    # Convert indices to actual values
    block_begend = [(above_thresh_df.index[x[0]], above_thresh_df.index[x[1]]) for x in block_begend_idxs]
    #-------------------------
    if return_indices:
        return block_begend_idxs
    else:
        return block_begend
    
def get_first_last_above_threshold(
    n_PNs_w_power_srs, 
    threshold
):
    r"""
    Get the first and last time (after initial power loss) that power was regained.
    For simple outages, these should be approximately equal
    """
    #-------------------------
    pds_above_thresh = get_periods_above_threshold(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        threshold=threshold, 
        return_indices=False
    )
    #----------------------------------------------------------------------------------------------------
    # len(pds_above_thresh)==2:
    #     For simple outages, where all initially have power, power is lost suddenly for all, and power is regained 
    #       suddenly for all, there should be two periods above threshold (len(pds_above_thresh)==2)
    #
    # len(pds_above_thresh)==1:
    #     This can arise from a few different situations (#2 is the only usable of the 3):
    #       1: - 'Power' is never lost (i.e., the percent of premises with power never dips below threshold)
    #          -  The single period spans the data
    #          -  The period data is of no use
    #       2: - 'Power' is already out at the beginning of the data 
    #          -  The beginning of the single period may be used to signify when power is restored
    #       3: - 'Power' is on at the beginning of the data, goes out, but is never restored
    #          -  The period data is of no use
    #
    # len(pds_above_thresh)==0:
    #     'Power' is out for the entirety of the data
    #
    # len(pds_above_thresh)>2:
    #     More complicated structure with sub-outages
    #
    #-------------------------
    # How we interpret the data depends on whether there is power at the beginng and end of the data
    power_at_beg = n_PNs_w_power_srs.iloc[0]>threshold
    power_at_end = n_PNs_w_power_srs.iloc[-1]>threshold
    #-------------------------
    if len(pds_above_thresh)==0:
        assert(not power_at_beg and not power_at_end)
        frst_above=None
        last_above=None
    #-------------------------
    elif len(pds_above_thresh)==1:
        assert(power_at_beg or power_at_end)
        last_above=None
        # If power_at_beg and power_at_end, then 'power' is never lost ==> situation #1 (described above)
        # If power_at_beg==False and power_at_end==True  ==> situation #2 (described above)
        # If power_at_beg==True  and power_at_end==False ==> situation #3 (described above)
        if power_at_beg==False and power_at_end==True:
            frst_above = pds_above_thresh[0][0]
        else:
            frst_above = None
    #-------------------------
    else:
        assert(power_at_beg or power_at_end)
        #-----
        # First time above threshold:
        #   If first data point is above threshold, then return beginning of the second period (at index=1)
        #     - i.e., if there is 'power' at the beginning of the data, return the first time power was restored
        #         after it was lost (where 'power' is defined as a minimum threshold of premises having power)
        #   If first data point is not above threshold, the power was initially out, so return beginning of first 
        #     period (at index=0)
        if power_at_beg:
            frst_above = pds_above_thresh[1][0]
        else:
            frst_above = pds_above_thresh[0][0]
        #-----
        # Last time above threshold:
        # If last data point is above threshold, then return beginning of last period (at index=-1)
        #     - i.e., if there is 'power' at the end of the data, return the beginning of the last period
        # If the last data point is not above threshold, the 'power' is out at the end of the data, so
        #   in this case return None
        if power_at_end:
            last_above = pds_above_thresh[-1][0]
        else:
            last_above = None
    #-------------------------
    return frst_above, last_above
    
def get_n_PNs_w_power_srs_time_to_print(
    n_PNs_w_power_srs, 
    threshold
):
    r"""
    In figure, entries with n_PNs_w_power>threshold are typically circled and have both their pct and time
    printed next to the data point.
    However, if there are multiple in a row, this can be messy.
    In such cases, we only want to print the time for those at the beginning and end of each grouping.
    This function identifies which entries should have their times printed
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    pds_above_thresh = get_periods_above_threshold(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        threshold=threshold, 
        return_indices=True
    )
    # Get unique idxs from pds_above_thresh
    print_times = natsorted(set(itertools.chain.from_iterable(pds_above_thresh)))
    #-------------------------
    # Always print the first and last times!
    if 0 not in print_times:
        print_times = [0]+print_times
    if n_PNs_w_power_srs.shape[0]-1 not in print_times:
        print_times = print_times+[n_PNs_w_power_srs.shape[0]-1]
    #-------------------------
    return_print_times_srs = n_PNs_w_power_srs.iloc[print_times]
    #-------------------------
    return return_print_times_srs


def plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=None, 
    fig_num=0, 
    title=None, 
    adjust_texts=True, 
    fig_ax=None, 
    **kwargs
):
    r"""
    """
    #-------------------------
    threshold_color = kwargs.get('threshold_color', 'red')
    #-------------------------
    if not n_PNs_w_power_srs.name:
        n_PNs_w_power_srs.name = 'pct_w_power'
    #-------------------------
    if fig_ax is None:
        fig,ax = Plot_General.default_subplots(fig_num=fig_num)
    else:
        fig = fig_ax[0]
        ax  = fig_ax[1]
    #-------------------------
    # Line plot of the data
    sns.lineplot(
        ax=ax, 
        data=n_PNs_w_power_srs.to_frame().reset_index(), 
        x='index', 
        y=n_PNs_w_power_srs.name, 
        marker='o'
    )
    #-----
    # Grab txt_srs (depending on simp_freq) to be used when printing
    #   points data on plot
    if simp_freq is not None:
        txt_srs = simplify_n_PNs_w_power_srs(
            n_PNs_w_power_srs, 
            freq=simp_freq
        )
    else:
        txt_srs = n_PNs_w_power_srs

    #-------------------------
    if threshold is not None:
        # Draw red circles around any data above threshold
        sns.scatterplot(
            ax=ax, 
            data=n_PNs_w_power_srs[n_PNs_w_power_srs>threshold].to_frame().reset_index(), 
            x='index', 
            y=n_PNs_w_power_srs.name, 
            marker='o', 
            edgecolor=threshold_color, 
            facecolor='none', 
            s=100
        )
        #-----
        # Draw horizontal line indicating threshold value
        ax.axhline(threshold, color=threshold_color, linestyle='dashed', alpha=0.5)
        #-----
        # Grab print_times_srs, used to print full time information next to 
        #   some of the above-threshold values (printed at beginning and end of blocks)
        print_times_srs = get_n_PNs_w_power_srs_time_to_print(
            n_PNs_w_power_srs=n_PNs_w_power_srs, 
            threshold=threshold
        )
        #-----
        # Don't want any repeats between print_times_srs and txt_srs
        txt_srs = txt_srs.loc[list(set(txt_srs.index).difference(set(print_times_srs.index)))]           

    #----- Printing points data on plot ---------------
    texts = []
    for time_i, pct_i in txt_srs.items():
        texts.append(ax.annotate(np.round(pct_i, decimals=2), (time_i, pct_i)))
    #-----
    if threshold is not None:
        for time_i, pct_i in print_times_srs.items():
            texts.append(
                ax.annotate(
                    f"{np.round(pct_i, decimals=2)}, {time_i.strftime('%d %H:%M:%S')}", 
                (time_i, pct_i), 
                color=threshold_color
                )
            )
    if adjust_texts:
        adjustText.adjust_text(texts, ax=ax)
    #--------------------------------------------------
    if title is not None:
        ax.set_title(title, fontdict=dict(fontsize=24))
    #--------------------------------------------------
    return fig,ax


def build_n_PNs_w_power_srs_and_plot(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    simp_freq='1T', 
    threshold=None, 
    fig_num=0, 
    fig_ax=None, 
    title=None, 
    adjust_texts=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb', 
    **plot_kwargs
):
    r"""
    ami_df_i
        - Only needed for finding n_PNs_tot
    """
    #-------------------------
    n_PNs_w_power_srs = build_n_PNs_w_power_srs(
        best_ests_df  = best_ests_df, 
        ami_df_i      = ami_df_i, 
        return_pct    = return_pct, 
        PN_col        = PN_col, 
        t_min_col     = t_min_col, 
        t_max_col     = t_max_col, 
        i_outg_col    = i_outg_col, 
        PN_col_ami_df = PN_col_ami_df
    )
    #-----
    if not n_PNs_w_power_srs.name:
        n_PNs_w_power_srs.name = 'pct_w_power'
    #-------------------------    
    fig,ax = plot_n_PNs_w_power_srs(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        simp_freq=simp_freq, 
        threshold=threshold, 
        fig_num=fig_num, 
        title=title, 
        adjust_texts=adjust_texts, 
        fig_ax=fig_ax, 
        **plot_kwargs
    )
    #-------------------------
    return n_PNs_w_power_srs, fig, ax